In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import datetime, timedelta, date
import xlrd
import glob
import warnings
from pulp import *
import pytz
from utils import *
import math
from rectpack import newPacker
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
def calculate_stock_available_sr(load, stock, open_so, open_sto_in, open_sto_out, pre_load_sto_out_df, production, actual_production, inventory_policy, run_time, run_type):
    
    # changes 5/27 : Pre-load changes
    sku = load['material_sk']
    source = load['Source']
    slot_booking_time = load['Slot Booked From']
    priority_flag = load['Priority Flag']

    try:
        # Total Stock at hand for this SKU and source location, also depends on the refresh rate
        stock_at_hand = stock.loc[
            (stock['material_sk'] == sku) & (stock['Source'] == source), 'opening_stock_hl'].values[0]
    
    except IndexError:
        stock_at_hand = 0


    # Outgoing Open SO - total outgoing quantity of this SKU from this source for run date
    outgoing_so = open_so[
        (open_so['material_sk'] == sku) &
        (open_so['Source'] == source) &
        # moving out of source (slot_arrival_time.date)
        (open_so['Delivery Date'] == run_time.normalize())
    ]['open_so_out_hl'].sum()

    # Incoming Open STO - total incoming quantity of this SKU to this source between model run time and (1 hour before slot booking time or truck arrival time)
    # # this is will be commented out
    # incoming_sto = open_sto_in[
    #     (open_sto_in['material_sk'] == sku) &
    #     (open_sto_in['Destination'] == source) &
    #     (open_sto_in['Slot Booked From'] >= run_time) &
    #     #above condition should be greater than run_time.normalize() but less than slot_booking_time 
    #     (open_sto_in['Slot Booked From'] <= slot_booking_time)
    # ]['total_quantity_hl'].sum()


    # Outgoing Preloaded Open STO

    # SAP stock on hand account for it so it needs to be excluded
    outgoing_preloaded_sto = pre_load_sto_out_df[

        (pre_load_sto_out_df['material_sk'] == sku) &

        (pre_load_sto_out_df['Source'] == source) &
        #origin slot arrival date
        (pre_load_sto_out_df['Delivery Date'] == run_time.normalize())

    ]['pre_load_sto_out(HL)'].sum()


    # total planned production of this SKU at this source after truck arrival time
    planned_production = production[
        (production['material_sk'] == sku) &
        (production['plant_code'] == source) &
        (production['release_ts'] <= slot_booking_time)
    ]['production_hl'].sum()


    # Actual production for the whole day for this SKU and destination -- destination var is not defined always leading to error
    #check with Rachana on this
    actual_prod = actual_production[
        (actual_production['material_sk'] == sku) &
        (actual_production['plant_code'] == source)
    ]['production_hl'].sum()



    #assuming stock refresh rate accounts for sto that has already been dispatched before model run time
    if run_type == 'INIT':
        outgoing_sto = open_sto_out[
            (open_sto_out['material_sk'] == sku) &
            (open_sto_out['Source'] == source) &
            (open_sto_out['Slot Booked From'] > run_time) &
            (open_sto_out['Slot Booked From'] < slot_booking_time)
            # check if we need to include a filter for priority_flag
            #(open_sto_out['Priority Flag'] <= priority_flag)
            ]['total_quantity_hl'].sum()
    elif run_type == 'TOP-UP':
        outgoing_sto = open_sto_out[
            (open_sto_out['material_sk'] == sku) &
            (open_sto_out['Source'] == source) &
            (open_sto_out['Slot Booked From'] > run_time) &
            (open_sto_out['Slot Booked From'] < run_time.normalize() + timedelta(days = 1))
        ]['total_quantity_hl'].sum()

    try: 
        # Safety stock
        safety_stock = inventory_policy.loc[(inventory_policy['material_sk'] == sku)&(inventory_policy['Source']==source), 'safety_stock_hl'].values[0]
    except IndexError:
        safety_stock = 0
    
    ## adding IF statement here so that for initial assignment, there is no Safety Stock, but for top-ups there are
    if run_type == 'INIT':
        safety_stock = 0
    elif run_type == 'TOP-UP':
        safety_stock = safety_stock
        #stock at hand update for LCP top-ups
        #check if this needs to be moved to general scope
        stock_at_hand = stock_at_hand - open_sto_out[
                                        (open_sto_out['material_sk'] == sku) &
                                        (open_sto_out['Source'] == source)
                                        ]['total_feasible_order_qty_hl'].sum()



    # Calculate the stock available
    # stock_available = (stock_at_hand - outgoing_so + incoming_sto + planned_production + actual_prod - outgoing_sto - safety_stock)
    stock_available = (stock_at_hand - outgoing_so - outgoing_preloaded_sto + planned_production + actual_prod - outgoing_sto - safety_stock)
    
    incoming_sto = 0


    return stock_available, stock_at_hand, planned_production, actual_prod, outgoing_so + outgoing_sto, incoming_sto, safety_stock

In [3]:
def calculate_stock_available_dest(load, stock, open_so, open_sto_in, open_sto_out,pre_load_sto_out_df, production,actual_production, inventory_policy, run_time):

    # changes 5/27 : Pre-load changes
    sku = load['material_sk']
    destination = load['Destination']
    priority_flag = load['Priority Flag']

    try:
        # Stock at hand at the destination
        stock_at_hand = stock.loc[
            (stock['material_sk'] == sku) & (stock['Source'] == destination), 'opening_stock_hl'].values[0]
    except IndexError:
        stock_at_hand = 0

    # Outgoing SO orders for the whole day
    outgoing_so = open_so[
        (open_so['material_sk'] == sku) &
        (open_so['Source'] == destination) &
        (open_so['Delivery Date'] == run_time.normalize())
    ]['open_so_out_hl'].sum()

    # Incoming Open STO orders for the whole day
    # changes 5/27 : Pre-load changes
    incoming_sto = open_sto_in[
        (open_sto_in['material_sk'] == sku) &
        (open_sto_in['Destination'] == destination) &
        (open_sto_in['planned_movement_ts'].dt.normalize() == run_time.normalize())
    ]['total_quantity_hl'].sum()

    # Outgoing STO orders for the whole day
    # total_quantity_hl here should be replaced by total_feasible_order_qty_hl
    outgoing_sto = open_sto_out[
        (open_sto_out['material_sk'] == sku) &
        (open_sto_out['Source'] == destination) &
        (open_sto_out['Slot Booked From'].dt.normalize() == run_time.normalize())
    ]['total_feasible_order_qty_hl'].sum()


     # Outgoing Preloaded Open STO orders for the whole day

    outgoing_preloaded_sto = pre_load_sto_out_df[

        (pre_load_sto_out_df['material_sk'] == sku) &

        (pre_load_sto_out_df['Source'] == destination) &

        (pre_load_sto_out_df['Delivery Date'] == run_time.normalize())

    ]['pre_load_sto_out(HL)'].sum()


    planned_production = production[
        (production['material_sk'] == sku) &
        (production['plant_code'] == destination) &
        (production['release_ts'].dt.normalize() == run_time.normalize())
    ]['production_hl'].sum()



    # Actual production for the whole day
    actual_prod = actual_production[
        (actual_production['material_sk'] == sku) &
        (actual_production['plant_code'] == destination)
    ]['production_hl'].sum()


    # Calculate the stock available at the destination
    stock_available = (stock_at_hand - outgoing_so  - outgoing_preloaded_sto + incoming_sto + planned_production + actual_prod - outgoing_sto)

    try:
        # Maximum stock (inventory policy)
        max_stock = inventory_policy.loc[
            (inventory_policy['material_sk'] == sku) & 
            (inventory_policy['Source'] == destination), 
            'max_stock_hl'
        ].values[0]
    except IndexError:
        max_stock = 0

    # Calculate the demand at the destination
    demand = max_stock - stock_available

    try:
        # safety stock (inventory policy)
        safety_stock = inventory_policy.loc[
            (inventory_policy['material_sk'] == sku) & 
            (inventory_policy['Source'] == destination), 
            'safety_stock_hl'
        ].values[0]
    except IndexError:
        safety_stock = 0

    oos_qty = (safety_stock - stock_available) if (safety_stock - stock_available>=0) else 0
    oos_per = oos_qty / demand
    oos_per = oos_per if oos_per>0 else 0

    return stock_available, demand, oos_per

In [4]:
def calculate_end_of_day_stock(stock, open_so, open_sto_out, pre_load_sto_out_df, production, actual_production, open_sto_out_swaps_combined, run_time):
    # changes 5/27 : Pre-load changes
    date_str = run_time.normalize().strftime('%Y-%m-%d')
    run_date = run_time.normalize()

    ## Prepare keys for merging
    sto_out_src = open_sto_out[['material_sk', 'Source', 'material_code', 'origin_shipping_location_sk']].rename(columns={'Source': 'plant_code', 'origin_shipping_location_sk':'plant_sk'})
    sto_out_dest = open_sto_out[['material_sk', 'Destination', 'material_code', 'destination_shipping_location_sk']].rename(columns={'Destination': 'plant_code', 'destination_shipping_location_sk':'plant_sk'})
    swaps_src = open_sto_out_swaps_combined[['material_sk', 'Source', 'material_code', 'origin_shipping_location_sk']].rename(columns={'Source': 'plant_code', 'origin_shipping_location_sk':'plant_sk'})
    swaps_dest = open_sto_out_swaps_combined[['material_sk', 'Destination', 'material_code', 'destination_shipping_location_sk']].rename(columns={'Destination': 'plant_code', 'destination_shipping_location_sk':'plant_sk'})
    so_keys = open_so[['material_sk', 'Source', 'material_code', 'origin_shipping_location_sk']].rename(columns={'Source': 'plant_code',  'origin_shipping_location_sk':'plant_sk'})
    preload_keys = pre_load_sto_out_df[['material_sk', 'Source', 'material_code', 'origin_shipping_location_sk']].rename(columns={'Source': 'plant_code',  'origin_shipping_location_sk':'plant_sk'})
    prod_keys = production[['material_sk', 'plant_code', 'material_code', 'plant_sk']]
    act_prod_keys = actual_production[['material_sk', 'plant_code', 'material_code', 'plant_sk']]
    stock_keys = stock[['material_sk', 'Source', 'material_code', 'plant_sk']].rename(columns={'Source': 'plant_code'})

    all_keys = pd.concat([
        sto_out_src, sto_out_dest, swaps_src, swaps_dest,
        so_keys, preload_keys, prod_keys, act_prod_keys, stock_keys
    ]).drop_duplicates()

    ## Merge with stock
    base = all_keys.copy()
    base['opening_stock_hl'] = 0
    base = base.merge(
        stock.rename(columns={'Source': 'plant_code'}),
        on=['material_sk', 'plant_code'], how='left', suffixes=('', '_stock')
    )
    base['opening_stock_hl'] = base['opening_stock_hl_stock'].fillna(0)
    base.drop(columns=['opening_stock_hl_stock'], inplace=True)

    ## Get groupby sums for all relevant dataframes
    def aggregate(df, group_cols, value_col, filters=None, rename_dict=None):
        if filters:
            for f in filters:
                df = df[f]
        agg = df.groupby(group_cols)[value_col].sum().reset_index()
        if rename_dict:
            agg = agg.rename(columns=rename_dict)
        return agg

    so_agg = aggregate(
        open_so, ['material_sk', 'Source'], 'open_so_out_hl',
        filters=[open_so['Delivery Date'] == date_str],
        rename_dict={'Source': 'plant_code', 'open_so_out_hl': 'outgoing_so'}
    )

    preload_agg = aggregate(
        pre_load_sto_out_df, ['material_sk', 'Source'], 'pre_load_sto_out(HL)',
        filters=[pre_load_sto_out_df['Delivery Date'] == date_str],
        rename_dict={'Source': 'plant_code', 'pre_load_sto_out(HL)': 'outgoing_preloaded_sto'}
    )

    incoming_swap_agg = aggregate(
        open_sto_out_swaps_combined, ['material_sk', 'Destination'], 'suggested_deployment_sr_hl',
        filters=[open_sto_out_swaps_combined['suggested_deployment_sr_hl'] > 0],
        rename_dict={'Destination': 'plant_code', 'suggested_deployment_sr_hl': 'incoming_sto'}
    )

    outgoing_swap_agg = aggregate(
        open_sto_out_swaps_combined, ['material_sk', 'Source'], 'suggested_deployment_sr_hl',
        filters=[open_sto_out_swaps_combined['suggested_deployment_sr_hl'] > 0],
        rename_dict={'Source': 'plant_code', 'suggested_deployment_sr_hl': 'outgoing_sto'}
    )

    incoming_og_agg = aggregate(
        open_sto_out, ['material_sk', 'Destination'], 'total_quantity_hl',
        filters=[open_sto_out['at_risk_flag'] == False],
        rename_dict={'Destination': 'plant_code', 'total_quantity_hl': 'incoming_og_sto'}
    )

    outgoing_og_agg = aggregate(
        open_sto_out, ['material_sk', 'Source'], 'total_quantity_hl',
        filters=[open_sto_out['at_risk_flag'] == False],
        rename_dict={'Source': 'plant_code', 'total_quantity_hl': 'outgoing_og_sto'}
    )

    planned_prod_agg = aggregate(
        production, ['material_sk', 'plant_code'], 'production_hl',
        filters=[production['release_ts'].dt.normalize() == run_date],
        rename_dict={'production_hl': 'planned_production'}
    )

    actual_prod_agg = aggregate(
        actual_production, ['material_sk', 'plant_code'], 'production_hl',
        rename_dict={'production_hl': 'actual_production'}
    )

    
    for df in [
        so_agg, preload_agg, incoming_swap_agg, outgoing_swap_agg,
        incoming_og_agg, outgoing_og_agg, planned_prod_agg, actual_prod_agg
    ]:
        base = base.merge(df, on=['material_sk', 'plant_code'], how='left')

    base.fillna(0, inplace=True)

    
    base['Closing_Stock'] = (
        base['opening_stock_hl']
        - base['outgoing_so']
        - base['outgoing_preloaded_sto']
        + base['incoming_sto']
        + base['incoming_og_sto']
        - base['outgoing_sto']
        - base['outgoing_og_sto']
        + base['planned_production']
        + base['actual_production']
    )

    return base.rename(columns={'plant_code': 'Source'})

In [5]:
weight_6 = 1

In [6]:
def optimise_loads(loads_at_risk_or_light_lcp_obs_enriched):
    problem = LpProblem('Load Exchanging', LpMaximize)

    # Decision Variable for Quantity
    #changes 4/20 : Area based optimization
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_Name'] = 'qty_' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Name'] = 'qty_pal_' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Continuous"))
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))
    #changes 5/12 rectpack
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Top_Name'] = 'qty_pal_top' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Bottom_Name'] = 'qty_pal_bottom' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_TOP'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Top_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))
    loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_BOTTOM'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Bottom_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))

    # OBJECTIVE FUNCTION
    # shipment_value = lpSum((data['lcp_rank_1'] + (0.5 * data['%_OOS_1'] + 0.5 * data['%_At_Risk_1'])/100 + data['Priority Flag_1'] + (1 - data['Waiting_time'])) * data['Qty_LPVar'] * data['HL_weight'])
    shipment_value = lpSum((loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] + 1 + (loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] + loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) + loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled'] + weight_6 * loads_at_risk_or_light_lcp_obs_enriched['truck_utilization_score_rescaled']) * loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'])
    problem +=shipment_value
    problem += lpSum(loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'])


    # # CONSTRAINT: Sum of all HL quantity recommendations for a material_sk should be less than total HL stock on hand at the source
    for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['material_sk', 'Source']):
        problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['stock_on_hand_sr_hl'].iloc[0]
    
    # # CONSTRAINT: Sum of all HL quantity recommendations for a material_sk should be less than Demand at the Destination
    for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['material_sk', 'Destination']):
        problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['demand_at_dt_hl'].iloc[0]

    # # CONSTRAINT: Sum of all recommended weights for a load should be less than the weight left on the truck(load)
    for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id']):
        problem += lpSum(qty * conv for qty, conv in zip(grp_df['Qty_LPVar'], grp_df['HL_weight'])) <= grp_df['available_weight'].iloc[0]

    # CONSTRAINT: Should be less than the area left on the truck
    # Check what happens when HL_PAL is NA for any material_sk
    #changes 4/20 : Area based optimization
    #changes 5/12 rectpack
    for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id']):
        problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL_TOP'], grp_df['area_per_pal'])) <= grp_df['available_area_top_row'].iloc[0]
        problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL_BOTTOM'], grp_df['area_per_pal'])) <= grp_df['available_area_bottom_row'].iloc[0]
        problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL'], grp_df['area_per_pal'])) <= grp_df['available_area'].iloc[0]

    #changes 4/20 : Area based optimization
    # individual constraint for each load_id and shipment
    for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id','material_sk']):
        problem += lpSum(grp_df['Qty_LPVar']) == lpSum(pal_qty * hl_per_pal for pal_qty, hl_per_pal in zip(grp_df['Qty_LPVar_PAL'], grp_df['PAL_HL']))
        #constraint: sum of suggested pallets in top and bottom row should be less than the total suggested pallets
        problem += lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL_TOP']) + lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL_BOTTOM']) == lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL'])
   # Soft constraint to avoid unbounded solution but still keep it relaxed
    for i, row in loads_at_risk_or_light_lcp_obs_enriched.iterrows():
        problem += row['Qty_LPVar'] <= 9999  # arbitrary large limit

    print('Started optimization')
    problem.solve(PULP_CBC_CMD(timeLimit = 600, threads = None, msg = 0))
    print('completed optimization')
    print(LpStatus[problem.status])
    return problem

In [7]:
def process_loads(main_outbound_df, main_inbound_df, main_load_details, stock, open_so, open_sto, pre_load_sto_out_df, production, actual_production, inventory_policy, lcp_data,no_forecast_df, load_details_df, run_time, result_path, tag):

    # Initialize a dictionary to store the results
    kpi_results    = {}
    print('Optimizing for '+tag)


    # Sorting the outbound loads as per slot booked from and priority flag
    main_outbound_df = main_outbound_df.sort_values(['Priority Flag', 'Slot Booked From']).reset_index(drop=True)
    print('## Total Number of Loads: ', main_outbound_df['load_id'].nunique())
    kpi_results['Total number of loads'] = {
        'Value': main_outbound_df['load_id'].nunique(),
        'Percentage': 100
    }

    if main_outbound_df['load_id'].nunique() == 0:
        print('## No loads to process')
        swaps_df = pd.DataFrame()
        main_load_details = pd.DataFrame()
        if tag == 'D0':
            return swaps_df, main_load_details, stock
        else:
            return swaps_df, main_load_details

    # Merging the outbound loads with the Open STO loads (to get material level details)
    # changes 5/27 : Pre-load changes
    open_sto_out = pd.merge(main_outbound_df, open_sto.loc[:,~open_sto.columns.isin(['trailer_equipment_type_code','actual_loading_start_ts','planned_movement_ts'])], on=['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 'Priority Flag'], how='inner')
    open_sto_in = pd.merge(main_inbound_df, open_sto.loc[:,~open_sto.columns.isin(['trailer_equipment_type_code','actual_loading_start_ts','planned_movement_ts'])], on=['load_id', 'Source', 'RFRC_NUM12', 'movement_type', 'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 'Priority Flag'] ,how='inner')
    
    # changes 5/27 : Pre-load changes
    open_sto_out[['stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl']] = \
            open_sto_out.apply(lambda row: pd.Series(calculate_stock_available_sr(row, stock, open_so, open_sto_in, open_sto_out, pre_load_sto_out_df, production, actual_production, inventory_policy, run_time, 'INIT')), axis=1)

    open_sto_out['at_risk_flag'] = np.where(open_sto_out['stock_on_hand_sr_hl'] < open_sto_out['total_quantity_hl'], True, False)
    #In cases where there is no SOH at source, no incoming but outgoing due
    open_sto_out['stock_on_hand_sr_hl'] = np.where(open_sto_out['stock_on_hand_sr_hl'] < 0, 0, open_sto_out['stock_on_hand_sr_hl'])
    open_sto_out['qty_at_risk_hl'] = np.where(open_sto_out['total_quantity_hl'] - open_sto_out['stock_on_hand_sr_hl'] <= 0, 0, open_sto_out['total_quantity_hl'] - open_sto_out['stock_on_hand_sr_hl'])
    open_sto_out['total_feasible_order_qty_hl'] = np.where(open_sto_out['at_risk_flag'] == True, open_sto_out['total_quantity_hl'] - open_sto_out['qty_at_risk_hl'], open_sto_out['total_quantity_hl'])


    # Getting UOM for weights and for HL to PAL conversion
    #changes 4/14 : keg conversion and rounding
    open_sto_out['total_feasible_order_qty_pc'] = open_sto_out['total_feasible_order_qty_hl'] / open_sto_out['material_sk'].map(pc_to_hl_dict)
    open_sto_out['total_feasible_order_qty_pal'] = np.where(open_sto_out['container_type_description'].str.upper() == 'KEG',open_sto_out['total_feasible_order_qty_pc'] * open_sto_out['material_sk'].map(pc_to_pal_dict) / open_sto_out['PAL_STACKING_FACTOR'], open_sto_out['total_feasible_order_qty_pc'] * open_sto_out['material_sk'].map(pc_to_pal_dict))
    #changes 4/20 : Area based optimization
    open_sto_out['total_feasible_order_qty_pal'] = open_sto_out['total_feasible_order_qty_pal'].apply(custom_round)
    open_sto_out['total_feasible_order_qty_weight'] = np.where(open_sto_out['container_type_description'].str.upper() == 'KEG', open_sto_out['total_feasible_order_qty_pal'] * open_sto_out['material_sk'].map(pal_weight_dict) * open_sto_out['PAL_STACKING_FACTOR'], open_sto_out['total_feasible_order_qty_pal'] * open_sto_out['material_sk'].map(pal_weight_dict))
    #changes 4/20 : Area based optimization
    open_sto_out['total_feasible_order_area'] = open_sto_out['total_feasible_order_qty_pal'] * open_sto_out['pal_length'] * open_sto_out['pal_width']

    #changes 5/12 rectpack
    load_level_rectpack_responses = extract_free_area(open_sto_out,ROTATABLE_DIMS, truck_length, truck_width, min_pallet_length, min_pallet_width, result_path, tag)


    # changes 5/21 
    print('% of loads with original load plan as rectpack feasible', load_level_rectpack_responses.loc[load_level_rectpack_responses['feasible'] == True, 'load_id'].nunique() / open_sto_out['load_id'].nunique() * 100.00)
    
    #changes 5/12 rectpack
    load_level_rectpack_responses['available_area'] = load_level_rectpack_responses['available_area_top_row'] + load_level_rectpack_responses['available_area_bottom_row']
    load_level_rectpack_responses = load_level_rectpack_responses[['load_id','feasible','available_area_top_row','available_area_bottom_row','available_area']]

    #changes 4/20 : Area based optimization
    #changes 5/12 rectpack
    load_level_feasible_order_details = open_sto_out.groupby(['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination', 'origin_shipping_location_sk',
        'destination_shipping_location_sk', 'Slot Booked From', 'Slot Booked To'], as_index=False).agg({'total_feasible_order_qty_hl': 'sum','total_feasible_order_qty_pc': 'sum','total_feasible_order_qty_pal': 'sum', 'total_feasible_order_qty_weight': 'sum','total_feasible_order_area': 'sum'})
    
    # changes 5/27 : Pre-load changes
    main_load_details = main_load_details[(main_load_details['actual_loading_start_ts']=='1900-01-01 00:00:00')|(main_load_details['actual_loading_start_ts'].isna())]
    
    main_load_details = pd.merge(main_load_details, load_level_feasible_order_details[['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination', 'total_feasible_order_qty_hl', 'total_feasible_order_qty_pc','total_feasible_order_qty_pal', 'total_feasible_order_qty_weight','total_feasible_order_area']],\
                on = ['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination'], how = 'left')
    main_load_details = pd.merge(main_load_details, load_level_rectpack_responses,on = ['load_id'], how = 'left')


    #changes 4/20 : Area based optimization
    #changes 5/12 rectpack
    #commenting below line below top vs bottom split cannot be calculated for infeasible loads
    # main_load_details['available_area'] = np.where(main_load_details['feasible']!=True,MAX_AVAILABLE_AREA - main_load_details['total_feasible_order_area'],main_load_details['available_area'])


    #changes 4/20 : Area based optimization
    #below statement needs to be changed as available_pal takes into consideration standard pallet area
    main_load_details['available_pal'] = main_load_details['available_area'] / STANDARD_PALLET_AREA
    main_load_details['available_pal'] = np.where(main_load_details['available_pal'] < 0, 0, main_load_details['available_pal'])
    main_load_details['available_pal'] = main_load_details['available_pal'].apply(custom_round)
    main_load_details['available_weight'] = standard_weights.loc[standard_weights['Country'] == 'GB', 'weight_limit'].values[0] - (main_load_details['total_feasible_order_qty_weight']/1000)
 

    ### Getting the labels for each load:
    # Define a tolerance level for floating-point comparison
    tolerance = 1e-5

    # Initialize the 'Action' column with default empty strings
    main_load_details['Action'] = ''

    # Apply 'Load not at risk' condition
    main_load_details.loc[
        (main_load_details['original_quantity_ordered_pal'] - main_load_details['total_feasible_order_qty_pal']).abs() < tolerance, 
        'Action'] = 'Load not at risk'

    # Apply 'At risk' condition
    main_load_details.loc[
        (main_load_details['total_feasible_order_qty_pal'] < main_load_details['original_quantity_ordered_pal']) & 
        (main_load_details['Action'] == ''),
        'Action'] = 'At risk'

    # Apply 'Light load' condition only if the other two are not satisfied
    #changes 4/14 : keg conversion and rounding
    #changes 4/20 : Area based optimization
    main_load_details.loc[
        (main_load_details['available_pal'] >= MIN_AVAILABLE_STD_PAL_FOR_LIGHT_LOAD_CLASS) & 
        (main_load_details['available_weight'] > MIN_AVAILABLE_WEIGHT_IN_TONNES_FOR_LIGHT_LOAD_CLASS) & 
        (main_load_details['Action'] == 'Load not at risk'), 
        'Action'] = 'Light load'

    ### new_load_data => only the light loads or the ones at risk
    loads_at_risk_or_light = main_load_details[main_load_details['Action'].isin(['Light load', 'At risk'])]
    #changes 4/20 : Area based optimization
    loads_at_risk_or_light = loads_at_risk_or_light[(loads_at_risk_or_light['available_pal']>=MIN_AVAILABLE_STD_PAL_FOR_LIGHT_LOAD_CLASS)&(loads_at_risk_or_light['available_weight']>MIN_AVAILABLE_WEIGHT_IN_TONNES_FOR_LIGHT_LOAD_CLASS)]
    print('## Loads that are at risk or have a light load: ', loads_at_risk_or_light['load_id'].nunique())
    kpi_results['Loads that are at risk or have a light load'] = {
        'Value': loads_at_risk_or_light['load_id'].nunique(),
        'Percentage': loads_at_risk_or_light['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    print('## Loads that are at risk: ', loads_at_risk_or_light[loads_at_risk_or_light['Action']=='At risk']['load_id'].nunique())
    kpi_results['Loads that are at risk'] = {
        'Value': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='At risk']['load_id'].nunique(),
        'Percentage': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='At risk']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    print('## Loads that are a light load: ', loads_at_risk_or_light[loads_at_risk_or_light['Action']=='Light load']['load_id'].nunique())
    kpi_results['Loads that are a light load'] = {
        'Value': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='Light load']['load_id'].nunique(),
        'Percentage': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='Light load']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    if loads_at_risk_or_light['load_id'].nunique()==0:
        print('## No Loads at Risk')
        swaps_df = pd.DataFrame()
        main_load_details = pd.DataFrame()
        if tag == 'D0':
            return swaps_df, main_load_details, stock
        else:
            return swaps_df, main_load_details

    ## remove since this has been added to preprocessing
    # lcp_data.rename(columns={'origin_location_code':'Source', 'destination_location_code':'Destination', 'origin_plant_sk':'origin_shipping_location_sk', 'destination_plant_sk':'destination_shipping_location_sk'}, inplace=True)
    # lcp_data.drop(columns=['origin_shipping_location_sk', 'destination_shipping_location_sk'], inplace=True)

    # Change 28/04
    print("LCP data combinations before removing zero-forecast SKUs with hist. loads: ", lcp_data.shape[0])
    ## Dropping those combinations where SKUs have zero forecast at the source but have been
    lcp_data = pd.merge(lcp_data, no_forecast_df[['Source', 'material_sk', 'material_code', 'actual_quantity']], on=['Source', 'material_sk', 'material_code'], how = 'left').fillna(0)
    lcp_data = lcp_data[lcp_data['actual_quantity']==0].reset_index(drop=True).drop(columns=['actual_quantity'])
    print("LCP data combinations after removing zero-forecast SKUs with hist. loads: ", lcp_data.shape[0])

    ### Gotta add the origin and the destination sk
    loads_at_risk_or_light_lcp_enriched = pd.merge(loads_at_risk_or_light.loc[:,~loads_at_risk_or_light.columns.str.contains('_shipping_location_sk')], lcp_data.loc[:,~lcp_data.columns.str.contains('_shipping_location_sk')], on=['Source', 'Destination'], how='inner')
    loads_at_risk_or_light_lcp_enriched.rename(columns={'origin_slot_arrival':'Slot Booked From', 'origin_slot_departure':'Slot Booked To'}, inplace=True)


    if loads_at_risk_or_light_lcp_enriched.shape[0]==0:
        print('## No possible replacement for the loads at risk')
        loads_at_risk_or_light_lcp_enriched.to_excel(f"{result_path}{tag}_pre_opti_model.xlsx", index=False)

        return None


    # changes 5/27 : Pre-load changes
    loads_at_risk_or_light_lcp_enriched[['stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl']] = \
            loads_at_risk_or_light_lcp_enriched.apply(lambda row: pd.Series(calculate_stock_available_sr(row, stock, open_so, open_sto_in, open_sto_out, pre_load_sto_out_df, production, actual_production, inventory_policy, run_time, 'TOP-UP')), axis=1)
    loads_at_risk_or_light_lcp_enriched = loads_at_risk_or_light_lcp_enriched[loads_at_risk_or_light_lcp_enriched['stock_on_hand_sr_hl'] > 0]
    
    
    # changes 5/27 : Pre-load changes
    loads_at_risk_or_light_lcp_enriched[['stock_on_hand_dt_hl', 'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl']] = loads_at_risk_or_light_lcp_enriched.apply(lambda row: pd.Series(calculate_stock_available_dest(row, stock, open_so, open_sto_in, open_sto_out, pre_load_sto_out_df, production, actual_production, inventory_policy, run_time)), axis=1)
    loads_at_risk_or_light_lcp_enriched = loads_at_risk_or_light_lcp_enriched[(loads_at_risk_or_light_lcp_enriched['demand_at_dt_hl'] > 0.0)&(loads_at_risk_or_light_lcp_enriched['stock_on_hand_sr_hl'] > 0.0)]


    print('## Loads with LCP swaps available: ', loads_at_risk_or_light_lcp_enriched['load_id'].nunique())
    kpi_results['Loads with LCP swaps available'] = {
        'Value':  loads_at_risk_or_light_lcp_enriched['load_id'].nunique(),
        'Percentage':  loads_at_risk_or_light_lcp_enriched['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }



    if loads_at_risk_or_light_lcp_enriched.shape[0]==0:
        print('## No possible replacement for the loads at risk')
        loads_at_risk_or_light_lcp_enriched.to_excel(f"{result_path}{tag}_pre_opti_model.xlsx", index=False)

        return None


    #changes 4/14 : keg conversion and rounding
    loads_at_risk_or_light_lcp_enriched['container_type_description'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(container_type_dict)
    loads_at_risk_or_light_lcp_enriched['PC_HL'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_hl_dict)

    #changes 4/15 pal_stacking_factor_map
    #changes 4/15 pal_stacking_factor_map
    mapped_values = loads_at_risk_or_light_lcp_enriched.apply(lambda row: pal_stacking_factor_map.get(row['Destination'][:2], {}).get(row['PC_HL'], np.nan),axis=1)
    loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',mapped_values, 1)


    #changes 4/20 : Area based optimization
    loads_at_risk_or_light_lcp_enriched['pal_length'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_length_dict)
    loads_at_risk_or_light_lcp_enriched['pal_width'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_width_dict)
    loads_at_risk_or_light_lcp_enriched['pal_height'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_height_dict)
    loads_at_risk_or_light_lcp_enriched['area_per_pal'] = loads_at_risk_or_light_lcp_enriched['pal_length'] * loads_at_risk_or_light_lcp_enriched['pal_width']


    #removing aggregated ordered vs feasible columns at load level . Retaining only available PAL/weight
    #changes 4/20 : Area based optimization
    #changes 5/12 rectpack
    loads_at_risk_or_light_lcp_enriched = loads_at_risk_or_light_lcp_enriched[['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Slot Booked From',
                'Slot Booked To', 'Priority Flag', 'available_pal', 'available_weight','available_area_top_row','available_area_bottom_row','available_area','material_sk', 'material_code', 'lcp_rank',
                'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl','actual_production_sr_hl',
                'outgoing_so_sto_sr_hl','incoming_sto_sr_hl', 'safety_stock_sr_hl', 'stock_on_hand_dt_hl',
                'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl', 'Action','pal_length', 'pal_width', 'pal_height', 'area_per_pal']]
    
    #changes 4/14 : keg conversion and rounding
    loads_at_risk_or_light_lcp_enriched['container_type_description'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(container_type_dict)
    loads_at_risk_or_light_lcp_enriched['PC_PAL'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_pal_dict)
    loads_at_risk_or_light_lcp_enriched['PC_HL'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_hl_dict)
    #changes 4/15 pal_stacking_factor_map
    #changes 4/20 : Area based optimization
    mapped_values = loads_at_risk_or_light_lcp_enriched.apply(lambda row: pal_stacking_factor_map.get(row['Destination'][:2], {}).get(row['PC_HL'], np.nan),axis=1)
    loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',mapped_values, 1)
    loads_at_risk_or_light_lcp_enriched.dropna(subset = ['PAL_STACKING_FACTOR'], inplace=True)
    loads_at_risk_or_light_lcp_enriched['HL_PAL'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',loads_at_risk_or_light_lcp_enriched['PC_PAL'] / (loads_at_risk_or_light_lcp_enriched['PC_HL'] * loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR']),loads_at_risk_or_light_lcp_enriched['PC_PAL'] / loads_at_risk_or_light_lcp_enriched['PC_HL'])
    loads_at_risk_or_light_lcp_enriched['PAL_HL'] = 1 / loads_at_risk_or_light_lcp_enriched['HL_PAL']
    loads_at_risk_or_light_lcp_enriched['HL_weight'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_weight_dict) * loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR'] * loads_at_risk_or_light_lcp_enriched['HL_PAL'] / 1000,loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_weight_dict) * loads_at_risk_or_light_lcp_enriched['HL_PAL'] / 1000)


    print('% LCP SKUs with missing HL_PAL',(loads_at_risk_or_light_lcp_enriched['HL_PAL'].isnull().sum() / len(loads_at_risk_or_light_lcp_enriched['HL_PAL']))*100.00)
    print('% LCP SKUs with missing HL_weight',(loads_at_risk_or_light_lcp_enriched['HL_weight'].isnull().sum() / len(loads_at_risk_or_light_lcp_enriched['HL_weight']))*100.00)
    print('% LCP SKUs with missing PAL_STACKING_FACTOR',(loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR'].isnull().sum() / len(loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR']))*100.00)

    loads_at_risk_or_light_lcp_obs_enriched = pd.merge(loads_at_risk_or_light_lcp_enriched, obs_df[['material_code', 'Source', 'material_sk', 'perc_obsolescence_risk_at_sr_hl']], on=['material_sk', 'material_code', 'Source'], how='left').fillna(0)

    loads_at_risk_or_light_lcp_obs_enriched.drop_duplicates(subset=['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Slot Booked From',
        'Slot Booked To', 'Priority Flag', 'material_sk', 'material_code'], inplace = True)



    # Operations to calculate the weights
    loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'] = loads_at_risk_or_light_lcp_obs_enriched['Slot Booked From'] - run_time
    #code for normalization
    loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'] = loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'] / loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'].max()

    #### Change this: priority Flag?
    # Create a dictionary with the formula (15 - x) * 3 + 1 for Priority Flag values 0 to 16
    priority_mapping = {x: (15 - x) * 3 + 1 for x in range(17)}
    # Apply the dictionary to map the 'Priority Flag' column
    loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled'] = loads_at_risk_or_light_lcp_obs_enriched['Priority Flag'].map(priority_mapping)
    #bug here, %OOS returned is always <1, not multiplied by 100 in the function
    loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] = loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl'] * 100.00
    # Check why this is 200
    loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] = np.where(loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] >= 200, 200, loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'])

    # % At Risk refers to obsolescence risk - already scaled to 100
    loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'] = np.where(loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'] >= 100, 100, loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'])
    # Define the dictionary for mapping lcp_rank values
    lcp_rank_mapping = {
        1: 5,
        2: 4,  # You can assign 2 to other values if needed
        3: 3,
        4: 2,  # Example mapping for the remaining values
        5: 1
    }

    # Apply the dictionary to map the 'lcp_rank' column
    loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] = loads_at_risk_or_light_lcp_obs_enriched['lcp_rank'].map(lcp_rank_mapping)

    loads_at_risk_or_light_lcp_obs_enriched['PAL_WEIGHT_KG'] = loads_at_risk_or_light_lcp_obs_enriched['material_sk'].map(pal_weight_dict)

    
    # Define truck weight limit
    TRUCK_WEIGHT_LIMIT = 44000

# Function to compute truck utilization score
    def compute_truck_utilization_score(loads_at_risk_or_light_lcp_obs_enriched, truck_weight_limit):
        utilization_dict = {}
        for load_id in loads_at_risk_or_light_lcp_obs_enriched['load_id'].unique():
            load_weight = loads_at_risk_or_light_lcp_obs_enriched[loads_at_risk_or_light_lcp_obs_enriched['load_id'] == load_id]['PAL_WEIGHT_KG'].sum()
            score = load_weight / truck_weight_limit if truck_weight_limit else 0
            utilization_dict[load_id] = min(score, 1.0)
        return utilization_dict

# Compute and map to new column
    truck_utilization_score_dict = compute_truck_utilization_score(loads_at_risk_or_light_lcp_obs_enriched, TRUCK_WEIGHT_LIMIT)
    loads_at_risk_or_light_lcp_obs_enriched['truck_utilization_score'] = loads_at_risk_or_light_lcp_obs_enriched['load_id'].map(truck_utilization_score_dict)

    from sklearn.preprocessing import MinMaxScaler

# Rescale truck utilization score just like other features
    scaler = MinMaxScaler()
    loads_at_risk_or_light_lcp_obs_enriched['truck_utilization_score_rescaled'] = scaler.fit_transform(
        loads_at_risk_or_light_lcp_obs_enriched[['truck_utilization_score']]
    )


    loads_at_risk_or_light_lcp_obs_enriched['Weights'] = ((loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled']) + (1 * loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] + 1 * loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) + (loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled']) + (weight_6 * loads_at_risk_or_light_lcp_obs_enriched['truck_utilization_score_rescaled']))

    loads_at_risk_or_light_lcp_obs_enriched.to_excel(f"{result_path}{tag}_pre_opti_model.xlsx", index=False)
    temp = loads_at_risk_or_light[loads_at_risk_or_light['load_id'].isin(loads_at_risk_or_light_lcp_obs_enriched['load_id'].unique())]

    # Model run here
    
    problem = optimise_loads(loads_at_risk_or_light_lcp_obs_enriched)

    # POST PROCESSING 

    #changes 4/20 : Area based optimization
    loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_hl'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'].apply(lambda x: x.value())
    loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_pal'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL'].apply(lambda x: x.value())
    loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_pal_top'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_TOP'].apply(lambda x: x.value())
    loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_pal_bottom'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_BOTTOM'].apply(lambda x: x.value())
    loads_at_risk_or_light_lcp_obs_enriched['LP_Result_Status'] = LpStatus[problem.status]

    #changes 4/20 : Area based optimization
    optimization_results = loads_at_risk_or_light_lcp_obs_enriched[loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_hl'] > 0]
    optimization_results = optimization_results[['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Priority Flag', \
        'material_sk', 'material_code', 'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl'
        , 'Action', 'suggested_deployment_sr_hl','suggested_deployment_sr_pal', 'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl','pal_length','pal_width']]
    

    # Swaps file creation
    #changes 4/20 : Area based optimization
    open_sto_out_swaps = open_sto_out[['load_id', 'RFRC_NUM12', 'movement_type', 'sales_document_item_code', 'Source', 'Destination', 'Priority Flag', 'material_sk', 'material_code',
                            'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl', 'qty_at_risk_hl', 'total_feasible_order_qty_hl', 'at_risk_flag','pal_length','pal_width']]
    open_sto_out_swaps = open_sto_out_swaps[open_sto_out_swaps['at_risk_flag'] == True]
    open_sto_out_swaps['Action'] = 'Swap-Out'
    open_sto_out_swaps = open_sto_out_swaps.rename(columns={'qty_at_risk_hl': 'swap_out_qty_hl', 'total_feasible_order_qty_hl': 'suggested_deployment_sr_hl'})

    open_sto_out_swaps_combined = pd.concat([open_sto_out_swaps, optimization_results])

    if len(open_sto_out_swaps_combined) == 0:
        print('## No optimization suggested')
        swaps_df = pd.DataFrame()
        if tag == 'D0':
            stock['Closing_Stock'] = stock['opening_stock_hl']
            return swaps_df, main_load_details, stock
        else:
            return swaps_df, main_load_details


    #changes 4/14 : keg conversion and rounding
    # enriching metadata to enable correct HL forward conversion
    open_sto_out_swaps_combined['container_type_description'] = open_sto_out_swaps_combined['material_sk'].map(container_type_dict)
    open_sto_out_swaps_combined['PC_PAL'] = open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict)
    open_sto_out_swaps_combined['PC_HL'] = open_sto_out_swaps_combined['material_sk'].map(pc_to_hl_dict)
    #changes 4/15 pal_stacking_factor_map
    mapped_values = open_sto_out_swaps_combined.apply(lambda row: pal_stacking_factor_map.get(row['Destination'][:2], {}).get(row['PC_HL'], np.nan),axis=1)
    open_sto_out_swaps_combined['PAL_STACKING_FACTOR'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',mapped_values, 1)
    #changes 4/20 : Area based optimization
    open_sto_out_swaps_combined.dropna(subset = ['PAL_STACKING_FACTOR'], inplace=True)
    open_sto_out_swaps_combined['HL_PAL'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',open_sto_out_swaps_combined['PC_PAL'] / (open_sto_out_swaps_combined['PC_HL'] * open_sto_out_swaps_combined['PAL_STACKING_FACTOR']),open_sto_out_swaps_combined['PC_PAL'] / open_sto_out_swaps_combined['PC_HL'])
    open_sto_out_swaps_combined['HL_weight'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',open_sto_out_swaps_combined['material_sk'].map(pal_weight_dict) * open_sto_out_swaps_combined['PAL_STACKING_FACTOR'] * open_sto_out_swaps_combined['HL_PAL'] / 1000,open_sto_out_swaps_combined['material_sk'].map(pal_weight_dict) * open_sto_out_swaps_combined['HL_PAL'] / 1000)


    #changes 4/17 : keg conversion and rounding
    #changes 4/20 : Area based optimization
    open_sto_out_swaps_combined['suggested_deployment_sr_pal'] = np.where(open_sto_out_swaps_combined['Action']=='Swap-Out',open_sto_out_swaps_combined['suggested_deployment_sr_hl'] * open_sto_out_swaps_combined['HL_PAL'],open_sto_out_swaps_combined['suggested_deployment_sr_pal'])
    open_sto_out_swaps_combined['suggested_deployment_sr_weight'] = open_sto_out_swaps_combined['suggested_deployment_sr_hl'] * open_sto_out_swaps_combined['HL_weight']
    open_sto_out_swaps_combined['suggested_deployment_sr_pc'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',(open_sto_out_swaps_combined['suggested_deployment_sr_pal'] * open_sto_out_swaps_combined['PAL_STACKING_FACTOR']) / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict),open_sto_out_swaps_combined['suggested_deployment_sr_pal'] / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict))
    open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] = open_sto_out_swaps_combined['suggested_deployment_sr_pal'].apply(custom_round)
    open_sto_out_swaps_combined['rounded_suggested_deployment_sr_hl'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] / open_sto_out_swaps_combined['HL_PAL']
    open_sto_out_swaps_combined['rounded_suggested_deployment_sr_weight'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_hl'] * open_sto_out_swaps_combined['HL_weight']
    open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pc'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',(open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] * open_sto_out_swaps_combined['PAL_STACKING_FACTOR']) / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict),open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict))
    open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pc'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pc'].apply(custom_round)


    #changes 4/20 : Area based optimization
    open_sto_out_swaps_combined['rounded_suggested_deployment_sr_area'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] * open_sto_out_swaps_combined['pal_length'] * open_sto_out_swaps_combined['pal_width']

    #changes 4/20 : Area based optimization
    swapped_load_details = open_sto_out_swaps_combined.loc[open_sto_out_swaps_combined['Action']!= 'Swap-Out',:].groupby(['load_id'], as_index = False).agg({'rounded_suggested_deployment_sr_hl':'sum', 'rounded_suggested_deployment_sr_pal':'sum', 'rounded_suggested_deployment_sr_weight':'sum', 'rounded_suggested_deployment_sr_area':'sum'})
    
    main_load_details = pd.merge(main_load_details, swapped_load_details[['load_id', 'rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_weight', 'rounded_suggested_deployment_sr_area']], on='load_id', how='left')
    #changes 4/20 : Area based optimization
    main_load_details[['rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_weight', 'rounded_suggested_deployment_sr_area']] = main_load_details[['rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_weight','rounded_suggested_deployment_sr_area']].fillna(0)
    main_load_details['rounded_suggested_deployment_sr_hl'] = main_load_details['rounded_suggested_deployment_sr_hl'] + main_load_details['total_feasible_order_qty_hl']
    main_load_details['rounded_suggested_deployment_sr_pal'] = main_load_details['rounded_suggested_deployment_sr_pal'] + main_load_details['total_feasible_order_qty_pal']
    # changes 05/12 : x 1000 for rounded weight because STO grain df is in tonnes
    main_load_details['rounded_suggested_deployment_sr_weight'] = main_load_details['rounded_suggested_deployment_sr_weight']*1000 + main_load_details['total_feasible_order_qty_weight']
    main_load_details['rounded_suggested_deployment_sr_area'] = main_load_details['rounded_suggested_deployment_sr_area'] + main_load_details['total_feasible_order_area']
    main_load_details['Cancel_load'] = np.where((26-main_load_details['total_feasible_order_qty_pal']-main_load_details['rounded_suggested_deployment_sr_pal'])/26 > cancel_load_threshold, 'Yes', 'No')


    main_load_details['Agreement to Recommendation(Yes/No)']= ''
    main_load_details['Recommendation Executed(Yes/No)']= ''
    main_load_details['Reason for non-agreement/non-execution']= ''

    main_load_details.drop_duplicates(subset=['load_id'], inplace=True)

    #changes 4/20 : Area based optimization
    main_load_details = main_load_details[['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source',
        'Destination', 'origin_shipping_location_sk',
        'destination_shipping_location_sk', 'origin_slot_arrival',
        'origin_slot_departure', 'destination_slot_arrival',
        'destination_slot_departure', 'Action', 'original_quantity_ordered_hl',
        'original_quantity_ordered_pal', 'original_quantity_ordered_kg','total_area_in_cm2',
        'total_feasible_order_qty_hl','total_feasible_order_qty_pal', 'total_feasible_order_qty_weight','total_feasible_order_area',
        'available_pal', 'available_weight', 'available_area',
        'rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal',
        'rounded_suggested_deployment_sr_weight','rounded_suggested_deployment_sr_area', 'Cancel_load',
        'Agreement to Recommendation(Yes/No)',
        'Recommendation Executed(Yes/No)',
        'Reason for non-agreement/non-execution']]


    main_load_details['available_pal'] = np.where(main_load_details['available_pal']<0, 0, main_load_details['available_pal'])
    main_load_details['available_weight'] = np.where(main_load_details['available_weight']<0, 0, main_load_details['available_weight'])
    main_load_details['Day_tag'] = tag
        
    # Generating output file 1 with the at-risk loads
    main_load_details['model_run_time_bst'] = run_time
    main_load_details.to_excel(f"{result_path}{tag}_load_level_report.xlsx", index=False)

    #check if this merge at RFRCNUM12 is correct
    open_sto_out_swaps_combined = pd.merge(open_sto_out_swaps_combined, main_load_details[['RFRC_NUM12', 'origin_slot_arrival', 'origin_slot_departure', 'destination_slot_arrival', 'destination_slot_departure', 'total_feasible_order_qty_pal', 'total_feasible_order_qty_weight','origin_shipping_location_sk', 'destination_shipping_location_sk', 'Cancel_load']], on = ['RFRC_NUM12'], how= 'left')
    #Changes 05/12 converting feasible weight at STO level to tonnes
    open_sto_out_swaps_combined['total_feasible_order_qty_weight'] = open_sto_out_swaps_combined['total_feasible_order_qty_weight']/1000
    open_sto_out_swaps_combined['swap_out_qty_hl'] = open_sto_out_swaps_combined['swap_out_qty_hl'].fillna(0)

    open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['swap_out_qty_hl']>0)&(open_sto_out_swaps_combined['suggested_deployment_sr_hl']>0), 'Swap-out (Update)', open_sto_out_swaps_combined['Action'])
    open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['swap_out_qty_hl']>0)&(open_sto_out_swaps_combined['suggested_deployment_sr_hl']==0), 'Swap-out (Delete)', open_sto_out_swaps_combined['Action'])

    open_sto_out_swaps_combined = pd.merge(open_sto_out_swaps_combined, open_sto_out[['load_id', 'material_sk', 'total_quantity_hl']], on = ['load_id', 'material_sk'], how = 'left').fillna(0)

    open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['Action']=='Light load')&(open_sto_out_swaps_combined['total_quantity_hl']==0), 'Top-up (New)', open_sto_out_swaps_combined['Action'])
    open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['Action']=='Light load')&(open_sto_out_swaps_combined['total_quantity_hl']!=0), 'Top-up (Update)', open_sto_out_swaps_combined['Action'])
    open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['Action']=='At risk'), 'Swap-in', open_sto_out_swaps_combined['Action'])


    open_sto_out_swaps_combined['Agreement to Recommendation(Yes/No)']= ''
    open_sto_out_swaps_combined['Recommendation Executed(Yes/No)']= ''
    open_sto_out_swaps_combined['Reason for non-agreement/non-execution']= ''
    open_sto_out_swaps_combined = open_sto_out_swaps_combined.fillna(0)

    open_sto_out_swaps_combined = open_sto_out_swaps_combined[['RFRC_NUM12', 'load_id', 'movement_type', 'sales_document_item_code', 'Priority Flag', 'origin_slot_arrival', 'origin_slot_departure', 'Source',
    'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 
    'material_sk', 'material_code', 'Action', 'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl', 
    'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl', 'swap_out_qty_hl', 'suggested_deployment_sr_hl',
    'suggested_deployment_sr_pal', 'suggested_deployment_sr_pc',
    'suggested_deployment_sr_weight', 'rounded_suggested_deployment_sr_hl',
    'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_pc',
    'rounded_suggested_deployment_sr_weight', 'Cancel_load',
    'Agreement to Recommendation(Yes/No)',
    'Recommendation Executed(Yes/No)',
    'Reason for non-agreement/non-execution']]

    open_sto_out_swaps_combined['Day_tag'] = tag
        
    open_sto_out_swaps_combined.rename(columns={'RFRC_NUM12':'STO Number'}, inplace=True)

    ### Dropping rows where the rounded recommendations for Swap-in or top-up are equal to zero
    open_sto_out_swaps_combined = open_sto_out_swaps_combined[(~open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Top-up (Update)', 'Swap-in']))|(open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal']!=0)]

    # Writing the swaps file
    open_sto_out_swaps_combined['model_run_time_bst'] = run_time
    open_sto_out_swaps_combined.to_excel(f"{result_path}{tag}_Swaps.xlsx", index=False)
    
    print('### Number of loads with swap-ins or top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique())
    kpi_results['Number of loads with swap-ins or top-ups'] = {
        'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique(),
        'Percentage':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    print('### Number of swap-ins or top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])].shape[0])
    kpi_results['Number of swap-ins or top-ups'] = {
        'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])].shape[0],
        'Percentage':  '-'
    }

    print('### Number of loads with swap-ins: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in']['load_id'].nunique())
    kpi_results['Number of loads with swap-ins'] = {
        'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in']['load_id'].nunique(),
        'Percentage':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }
    
    print('### Number of swap-ins: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in'].shape[0])
    kpi_results['Number of swap-ins'] = {
        'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in'].shape[0],
        'Percentage':  '-'
    }

    print('### Number of loads with top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])]['load_id'].nunique())
    kpi_results['Number of loads with top-ups'] = {
        'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])]['load_id'].nunique(),
        'Percentage':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])]['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
    }

    print('### Number of top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])].shape[0])
    kpi_results['Number of top-ups'] = {
        'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])].shape[0],
        'Percentage':  '-'
    }
    #convert from kg to Tonnes
    main_load_details[['total_feasible_order_qty_weight','rounded_suggested_deployment_sr_weight']] =  np.round(main_load_details[['total_feasible_order_qty_weight','rounded_suggested_deployment_sr_weight']]/1000,3)


    # changes 5/29 : scorecard customization
    wps_improvement_summary_overall = main_load_details.loc[main_load_details['Action']!='Load not at risk',['RFRC_NUM12', 'load_id','Day_tag', 'Action','Cancel_load','movement_type','Source','Destination','total_feasible_order_qty_weight','rounded_suggested_deployment_sr_weight']]
    wps_improvement_summary_overall['market'] = 'GB'
    wps_improvement_summary_overall['max_payload_weight'] = standard_weights.loc[standard_weights['Country'] == 'GB', 'weight_limit'].values[0]
    wps_improvement_summary_overall['max_vs_deployment_weight'] = round(wps_improvement_summary_overall['max_payload_weight'] - wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'],3)
    wps_improvement_summary_overall['deployment_weight_percent_of_max_weight'] = round((wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'] / wps_improvement_summary_overall['max_payload_weight'])*100.00,2)

    wps_improvement_summary_overall['deployment_vs_feasible_weight'] = round(wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'] - wps_improvement_summary_overall['total_feasible_order_qty_weight'],3)
    wps_improvement_summary_overall['deployment_weight_percent_of_feasible_weight'] = round((wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'] / wps_improvement_summary_overall['total_feasible_order_qty_weight'])*100.00,2)
    wps_improvement_summary_overall['wps_savings_per_tonne_in_usd'] = wps_savings_per_tonne_in_usd
    wps_improvement_summary_overall['wps_cost_savings_in_usd'] = round(wps_improvement_summary_overall['deployment_vs_feasible_weight'] * wps_improvement_summary_overall['wps_savings_per_tonne_in_usd'],2)

    wps_improvement_summary_overall.to_excel(f"{result_path}{tag}_wps_improvement_summary_overall.xlsx")

    wps_improvement_summary_light_loads = wps_improvement_summary_overall.loc[wps_improvement_summary_overall['Action']=='Light load',:]


    wps_improvement_summary_light_loads.to_excel(f"{result_path}{tag}_wps_improvement_summary_light_loads.xlsx")

    # changes 5/29 : scorecard customization
    if tag != 'D0':
        cancellation_cost_potential_summary = main_load_details.loc[(main_load_details['total_feasible_order_qty_pal']==0) & (main_load_details['rounded_suggested_deployment_sr_pal']>0),['RFRC_NUM12', 'load_id','Day_tag', 'Action','Cancel_load','movement_type','Source','Destination','total_feasible_order_qty_pal','rounded_suggested_deployment_sr_pal']]
        cancellation_cost_potential_summary['average_shipment_price'] = average_shipment_price
        cancellation_cost_potential_summary['perc_trip_price_as_cancellation_cost'] = perc_trip_price_as_cancellation_cost
        cancellation_cost_potential_summary['average_cancellation_cost'] = round(cancellation_cost_potential_summary['average_shipment_price'] * cancellation_cost_potential_summary['perc_trip_price_as_cancellation_cost'],2)
        cancellation_cost_potential_summary['market'] = 'GB'
        cancellation_cost_potential_summary.to_excel(f"{result_path}{tag}_cancellation_cost_potential_summary.xlsx")

    ### initial volume:
    metric = main_load_details['original_quantity_ordered_pal'].sum()
    perc_metric = main_load_details['original_quantity_ordered_pal'].sum()/main_load_details['original_quantity_ordered_pal'].sum() * 100
    print('### Total planned initial volume in PAL: ',metric )
    kpi_results['Total planned initial volume in PAL'] = {
        'Value':  metric,
        'Percentage':  perc_metric
    }
    ### Volumne at risk
    metric = main_load_details['original_quantity_ordered_pal'].sum() - main_load_details['total_feasible_order_qty_pal'].sum()
    perc_metric = (main_load_details['original_quantity_ordered_pal'].sum() - main_load_details['total_feasible_order_qty_pal'].sum())/main_load_details['original_quantity_ordered_pal'].sum() * 100
    print('### Volume at risk in PAL due to insufficient stock at source: ', metric)
    kpi_results['Volume at risk in PAL due to insufficient stock at source (% of total planned volume)'] = {
        'Value':  metric,
        'Percentage':  perc_metric
    }
    ### Total optimised volume:
    #check if the denominator base should be unavailable volume - yes
    metric = main_load_details.loc[main_load_details['Action']=='At risk','rounded_suggested_deployment_sr_pal'].sum() - main_load_details.loc[main_load_details['Action']=='At risk','total_feasible_order_qty_pal'].sum()
    perc_metric = (main_load_details.loc[main_load_details['Action']=='At risk','rounded_suggested_deployment_sr_pal'].sum() - main_load_details.loc[main_load_details['Action']=='At risk','total_feasible_order_qty_pal'].sum())/(main_load_details['original_quantity_ordered_pal'].sum() - main_load_details['total_feasible_order_qty_pal'].sum()) * 100
    print('### Volume at risk optimized in PAL due to swap-ins', metric)
    kpi_results['Volume at risk optimized in PAL due to swap-ins (% of volume at risk)'] = {
        'Value':  metric,
        'Percentage':  perc_metric
    }

    metric = main_load_details[main_load_details['Action']=='Light load']['rounded_suggested_deployment_sr_pal'].sum() - main_load_details[main_load_details['Action']=='Light load']['total_feasible_order_qty_pal'].sum()
    perc_metric = ((main_load_details[main_load_details['Action']=='Light load']['rounded_suggested_deployment_sr_pal'].sum() - main_load_details[main_load_details['Action']=='Light load']['total_feasible_order_qty_pal'].sum())/main_load_details[main_load_details['Action']=='Light load']['available_pal'].sum()) * 100
    print('### Light load top-ups performed in PAL: ', metric)
    kpi_results['Light load top-ups performed in PAL (% of available PAL for light loads)'] = {
        'Value':  metric,
        'Percentage':  perc_metric
    }

    metric = np.round(main_load_details['total_feasible_order_qty_weight'].mean(),3)
    print('### Average feasible weight per shipment in Tonnes pre optimization: ', metric)
    kpi_results['Average feasible weight per shipment in Tonnes pre optimization'] = {
        'Value':  metric,
        'Percentage':  '-'
    }
    metric = np.round(main_load_details['rounded_suggested_deployment_sr_weight'].mean(),3)
    print('### Average weight per shipment in Tonnes post optimization: ', metric)
    kpi_results['Average weight per shipment in Tonnes post optimization'] = {
        'Value':  metric,
        'Percentage':  '-'
    }
    metric = np.round((main_load_details['rounded_suggested_deployment_sr_weight'].mean()) - (main_load_details['total_feasible_order_qty_weight'].mean()),3)
    print('### Improvement in average weight per shipment in Tonnes due to optimization: ', metric)
    kpi_results['Improvement in average weight per shipment in Tonnes due to optimization'] = {
        'Value': metric,
        'Percentage':  '-'
    }
    metric = np.round((main_load_details.loc[main_load_details['Action']=='Light load','total_feasible_order_qty_weight'].mean()),3)
    print('### Average feasible weight per shipment in Tonnes pre optimization (only for light loads): ', metric)
    kpi_results['Average feasible weight per shipment in Tonnes pre optimization (only for light loads)'] = {
        'Value': metric,
        'Percentage':  '-'
    }
    metric = np.round((main_load_details.loc[main_load_details['Action']=='Light load','rounded_suggested_deployment_sr_weight'].mean()),3)
    print('### Average weight per shipment in Tonnes post optimization (only for light loads): ',  metric)
    kpi_results['Average weight per shipment in Tonnes post optimization (only for light loads)'] = {
        'Value': metric,
        'Percentage':  '-'
    }
    metric = np.round(((main_load_details.loc[main_load_details['Action']=='Light load','rounded_suggested_deployment_sr_weight']) - (main_load_details.loc[main_load_details['Action']=='Light load','total_feasible_order_qty_weight'])).mean(),3)
    print('### Improvement in average weight per shipment in Tonnes due to optimization (only for light loads): ', metric)
    kpi_results['Improvement in average weight per shipment in Tonnes due to optimization (only for light loads)'] = {
        'Value': metric,
        'Percentage':  '-'
    }

    # changes 5/29 : scorecard customization
    metric = wps_improvement_summary_overall['wps_cost_savings_in_usd'].sum()
    print('### USD Benefits generated from WPS improvement across all loads: ', metric)
    kpi_results['USD Benefits generated from WPS improvement across all loads:'] = {
        'Value': metric,
        'Percentage':  '-'
    }
    metric = wps_improvement_summary_light_loads['wps_cost_savings_in_usd'].sum()
    print('### USD Benefits generated from WPS improvement across light loads: ', metric)
    kpi_results['USD Benefits generated from WPS improvement across light loads:'] = {
        'Value': metric,
        'Percentage':  '-'
    }
    if tag!='D0':
        metric = round(cancellation_cost_potential_summary['average_cancellation_cost'].sum(),2)
        print('### Potential D1 cancellation avoidance cost savings across all loads: ', metric)
        kpi_results['Potential D1 cancellation avoidance cost savings across all loads:'] = {
            'Value': round(metric,2),
            'Percentage':  '-'
        }

    # Convert KPI results into a DataFrame for saving as a table
    kpi_df = pd.DataFrame.from_dict(kpi_results, orient='index')


    kpi_df['model_run_time_bst'] = run_time

    # Save to excel file (or any other format)
    kpi_df.to_excel(f"{result_path}{tag}_Score_card.xlsx")

    if tag =='D1':
        return open_sto_out_swaps_combined, main_load_details
        
    ## Need to recheck the open_sto_in and open_sto_out part.
    updated_stock = calculate_end_of_day_stock(stock, open_so, open_sto_out, pre_load_sto_out_df, production, actual_production, open_sto_out_swaps_combined, run_time)

    return open_sto_out_swaps_combined, main_load_details, updated_stock

In [8]:
def extract_free_area(sto_details,ROTATABLE_DIMS, truck_length, truck_width, min_pallet_length, min_pallet_width,result_path, tag):
    
    result = {}

    for load_id, group in sto_details.groupby("load_id"):
        #print(load_id)
        packer = newPacker(rotation=False)
        packer.add_bin(truck_length, truck_width)

        # Add rectangles with unique IDs
        rect_id = 0
        rect_map = {}

        # Add each pallet to the bin
        for _, row in group.iterrows():
            # changes 5/21 : non-orthogonal loading for 132x120 pallets
            rotate_it = (
            (row["pal_length"], row["pal_width"]) in ROTATABLE_DIMS
            )

            pal_len  = row["pal_length"]
            pal_wid  = row["pal_width"]

            if rotate_it:
                pal_len, pal_wid = pal_wid, pal_len

            for _ in range(int(row['total_feasible_order_qty_pal'])):
                packer.add_rect( pal_wid,pal_len, rect_id)
                rect_map[rect_id] = (row['material_sk'], pal_wid,pal_len)
                rect_id += 1
        packer.pack()
        rectangles = packer.rect_list()
        total_pallets = group['total_feasible_order_qty_pal'].sum()

        if total_pallets == 0:
            result[load_id] = {
                'pallets_requested': 0,
                'pallets_packed': 0,
                'feasible': True,
                'remaining_length_bottom_row': truck_length,
                'remaining_length_top_row': truck_length,
                'remaining_width_bottom_row': truck_width/2,
                'remaining_width_top_row': truck_width/2,
                'available_area_top_row': truck_length * (truck_width/2),
                'available_area_bottom_row': truck_length * (truck_width/2)
            }
            continue

        #get the packed pallets
        packed_pallets = len(packer[0])

        #get the remaining length in top & bottom row separately
        bottom_row_rectangles = [rect for rect in rectangles if rect[2]==0]
        top_row_rectangles = [rect for rect in rectangles if rect[2]!=0]

        # Calculate max X + width to determine used truck length (bottom row)
        #changes 5/20 : list end error
        try:
            max_x_end_bottom_row = max(x + w for b, x, y, w, h, rid in bottom_row_rectangles)
        except Exception:
            max_x_end_bottom_row = 0
        remaining_length_bottom_row = truck_length - max_x_end_bottom_row

        # Calculate max X + width to determine used truck length (bottom row)
        #changes 5/20 : list end error
        try:
            max_x_end_top_row = max(x + w for b, x, y, w, h, rid in top_row_rectangles)
        except Exception:
            max_x_end_top_row = 0
        remaining_length_top_row = truck_length - max_x_end_top_row

        #changes 5/20 : list end error
        try:
            max_x_end = max(x + w for b, x, y, w, h, rid in rectangles)
        except Exception:
            max_x_end = 0


        pallet_placed_at_end_bottom_row = [placed_rect for placed_rect in bottom_row_rectangles if placed_rect[1] + placed_rect[3] == max_x_end_bottom_row]
        pallet_placed_at_end_top_row = [placed_rect for placed_rect in top_row_rectangles if placed_rect[1] + placed_rect[3] == max_x_end_top_row]
        pallets_placed_at_end = [placed_rect for placed_rect in rectangles if placed_rect[1] + placed_rect[3] == max_x_end]
        
        if sum(pal[4] for pal in pallets_placed_at_end) == truck_width:
            remaining_width_top_row = truck_width / 2
            remaining_width_bottom_row = truck_width / 2
        #changes 5/20 : list end error
        elif (len(pallets_placed_at_end) == 1) & (((0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][1]) + (0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][3])) > ((0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][1]) + (0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][3]))):
            remaining_width_bottom_row = pallet_placed_at_end_bottom_row[0][4]
            remaining_width_top_row = truck_width - remaining_width_bottom_row
        #changes 5/20 : list end error
        elif (len(pallets_placed_at_end) == 1) & (((0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][1]) + (0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][3])) < ((0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][1]) + (0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][3]))):
            remaining_width_top_row = pallet_placed_at_end_top_row[0][4]
            remaining_width_bottom_row = truck_width - remaining_width_top_row
        else:
            remaining_width_top_row = truck_width / 2
            remaining_width_bottom_row = truck_width / 2

        
        if remaining_length_bottom_row < min_pallet_width or remaining_width_bottom_row < min_pallet_length:
            available_area_bottom_row = 0
        else:
            available_area_bottom_row = remaining_length_bottom_row * remaining_width_bottom_row

        if remaining_length_top_row < min_pallet_width or remaining_width_top_row < min_pallet_length:
            available_area_top_row = 0
        else:
            available_area_top_row = remaining_length_top_row * remaining_width_top_row


        result[load_id] = {
                'pallets_requested': int(total_pallets),
                'pallets_packed': int(packed_pallets),
                'feasible': packed_pallets == total_pallets,
                'remaining_length_bottom_row': remaining_length_bottom_row,
                'remaining_length_top_row': remaining_length_top_row,
                'remaining_width_bottom_row': remaining_width_bottom_row,
                'remaining_width_top_row': remaining_width_top_row,
                'available_area_top_row': available_area_top_row,
                'available_area_bottom_row': available_area_bottom_row
            }
        # Plot setup
        fig, ax = plt.subplots(figsize=(20, 3))
        ax.set_xlim(0, truck_length)
        ax.set_ylim(0, truck_width)
        ax.set_aspect('equal')
        ax.set_title(f"Packed Layout for Shipment {load_id}")

        # Draw each packed pallet
        for b, x, y, w, h, rid in rectangles:
            # print(x, y, w, h, rid )
            material, plen, pwid = rect_map[rid]
            ax.add_patch(plt.Rectangle((x, y), w, h, edgecolor='black', facecolor='skyblue', alpha=0.6))
            ax.text(x + w/2, y + h/2, f"{material}\n{plen}×{pwid}", ha='center', va='center', fontsize=8)

        plt.xlabel("Truck Length (cm)")
        plt.ylabel("Truck Width (cm)")
        plt.grid(True)
        plt.savefig(f"{result_path}{tag}_{load_id}_pre_opt.png")
        # plt.show()
        plt.close()

    result = pd.DataFrame.from_dict(result, orient='index').reset_index()
    result.rename(columns={'index': 'load_id'}, inplace=True)
    return(result)
   

In [9]:
def custom_round(decimal_value):
    """
    Rounds a decimal value based on the difference with the next integer.
    If the difference is less than 0.00001, ceil the value; otherwise, floor it.

    Args:
        decimal_value (float): The decimal value to round.

    Returns:
        int: The rounded integer.
    """
    if math.ceil(decimal_value) - decimal_value < 0.00001:
        return math.ceil(decimal_value)
    else:
        return math.floor(decimal_value)

In [10]:
cancel_load_threshold = 0.8
MIN_AVAILABLE_WEIGHT_IN_TONNES_FOR_LIGHT_LOAD_CLASS = 0.3
MIN_AVAILABLE_STD_PAL_FOR_LIGHT_LOAD_CLASS=1
# MAX_AVAILABLE_AREA = 26*120*100
# STANDARD_PALLET_AREA = 120*100
MAX_AVAILABLE_AREA = 1360*240
truck_length=1360
truck_width=240
min_pallet_width = 80
min_pallet_length = 120
STANDARD_PALLET_AREA = min_pallet_length*min_pallet_width
ROTATABLE_DIMS = {(132, 120)}
wps_savings_per_tonne_in_usd = 10
average_shipment_price = 500
perc_trip_price_as_cancellation_cost = 0.7
#stacking factor for products are in KEGs 
pal_stacking_factor_map = {

    'AT' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'BE' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'CH' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'FR' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'GB' : {
        0.06 : 1,
        0.1 : 4,
        0.15 : 4,
        0.20 : 4,
        0.30 : 4,
        0.50 : 3,
        1 : 3,
        1.5 : 3 
        },
    'IT' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'LU' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'NL' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'PL' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'DE' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'DK' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'SE' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'FI' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        },
    'NO' : {
        0.06 : 1,
        0.1 : 3,
        0.15 : 3,
        0.20 : 3,
        0.30 : 3,
        0.50 : 2,
        1 : 0,
        1.5 : 0
        }
}

In [11]:
#load the location variables from the file
with open('location_variables.txt', 'r') as f:
    for line in f:
        exec(line)

run_date = pd.to_datetime(run_date)
run_time_naive = pd.to_datetime(run_time_naive)
run_time = pd.to_datetime(run_time)

In [12]:
run_time

Timestamp('2025-06-05 13:27:24')

In [13]:
preprocessed_path

'./PRE_PROCESSED_DATA/06. June 2025/05_06_2025 preprocessed data/2025_06_05_13_27_24 preprocessed data/'

In [14]:
### Reading the preprocessed files
stock =  pd.read_csv(preprocessed_path + 'stock.csv' )

load_details_df =  pd.read_csv(preprocessed_path + 'load_details.csv' )

obs_df =  pd.read_csv(preprocessed_path + 'obs_stock.csv' )

open_sto = pd.read_csv(preprocessed_path + 'open_sto.csv' )

inventory_policy = pd.read_csv(preprocessed_path + 'inventory_policy.csv' )

open_so = pd.read_csv(preprocessed_path + 'open_so.csv' )

uom_df = pd.read_csv(preprocessed_path + 'uom.csv' )

production = pd.read_csv(preprocessed_path + 'planned_production.csv' )

actual_production = pd.read_csv(preprocessed_path + 'actual_production.csv' )

planned_loads_df = pd.read_csv(preprocessed_path + 'planned_loads.csv' )

outbound_loads_df = pd.read_csv(preprocessed_path + 'outbound_loads.csv' )

inbound_loads_df = pd.read_csv(preprocessed_path + 'inbound_loads.csv' )

lcp_data = pd.read_csv(preprocessed_path + 'lcp_data.csv' )

standard_weights = pd.read_csv(preprocessed_path + 'standard_weights.csv' )

no_forecast_df = pd.read_csv(preprocessed_path + 'no_forecast.csv' )

pre_load_sto_out_df = pd.read_csv(preprocessed_path + 'pre_load_sto_out.csv' )
#Remove duplicates if any
apply_function_to_all_dfs_in_memory(ensure_level_of_data)

In [15]:
# % of total STO loads that have original quantity ordered in PAL > 26
((load_details_df.loc[load_details_df['original_quantity_ordered_pal']>26,'load_id'].nunique() / load_details_df['load_id'].nunique())*100.00)

8.333333333333332

In [16]:
#loads that do not contain mixed pallets
load_level_pal_config_details = open_sto.groupby(['load_id','trailer_equipment_type_code'],as_index=False).agg({'total_quantity_pal' : 'sum','total_quantity_kg' : 'sum','total_area_in_cm2' : sum , 'pal_config' : set}).reset_index(drop=True)

In [17]:
load_level_pal_config_details.loc[load_level_pal_config_details['pal_config'] != set(['120.0x100.0']),:]

,load_id,trailer_equipment_type_code,total_quantity_pal,total_quantity_kg,total_area_in_cm2,pal_config
0,34800585,ZGBTR26,15,24977.00,144000.0,{120.0x80.0}
25,34800880,ZGBTR26,26,21891.20,266400.0,"{120.0x100.0, 120.0x80.0}"
34,34801263,ZGBTR26,16,24489.52,245760.0,"{120.0x100.0, 132.0x120.0, 120.0x80.0}"
36,34801265,ZGBTR26,16,25920.00,253440.0,{132.0x120.0}
41,34801564,ZGBTR26,17,27540.00,269280.0,{132.0x120.0}
44,34801567,ZGBTR26,17,25456.80,269280.0,{132.0x120.0}
45,34801568,ZGBTR26,15,23839.20,237600.0,{132.0x120.0}
47,34801613,ZGBTR26,19,27342.80,293280.0,"{120.0x100.0, 132.0x120.0}"
49,34801654,ZGBTR26,20,23820.00,192000.0,{120.0x80.0}


In [18]:
loads_with_mixed_pallets = load_level_pal_config_details.loc[load_level_pal_config_details['pal_config'] != set(['120.0x100.0']), 'load_id'].unique()

In [19]:
# % of total STO loads that have mixed pallets
((len(loads_with_mixed_pallets) / load_details_df['load_id'].nunique())*100.00)

12.5

In [20]:
loads_with_mixed_pallets

array([34800585, 34800880, 34801263, 34801265, 34801564, 34801567,
       34801568, 34801613, 34801654], dtype=int64)

In [21]:
### Creating dictionaries for all units of conversion

#changes 4/14 : keg conversion and rounding
pc_to_hl_dict = dict(zip(uom_df['material_sk'], uom_df['PC_HL']))
pc_to_pal_dict = dict(zip(uom_df['material_sk'], uom_df['PC_PAL']))
pal_weight_dict = dict(zip(uom_df['material_sk'], uom_df['PAL_WEIGHT_KG']))
container_type_dict = dict(zip(uom_df['material_sk'], uom_df['container_type_description']))
pal_length_dict = dict(zip(uom_df['material_sk'], uom_df['pal_length']))
pal_width_dict = dict(zip(uom_df['material_sk'], uom_df['pal_width']))
pal_height_dict = dict(zip(uom_df['material_sk'], uom_df['pal_height']))


In [22]:
cols_to_convert_to_dt = ['origin_slot_arrival','origin_slot_departure','actual_loading_start_ts','planned_movement_ts']
outbound_loads_df[cols_to_convert_to_dt] = outbound_loads_df[cols_to_convert_to_dt].apply(ensure_datetime)
cols_to_convert_to_dt = ['destination_slot_arrival','destination_slot_departure','actual_loading_start_ts','planned_movement_ts']
inbound_loads_df[cols_to_convert_to_dt] = inbound_loads_df[cols_to_convert_to_dt].apply(ensure_datetime)
cols_to_convert_to_dt = ['origin_slot_arrival','origin_slot_departure','destination_slot_arrival','destination_slot_departure','actual_loading_start_ts','planned_movement_ts']
load_details_df[cols_to_convert_to_dt] = load_details_df[cols_to_convert_to_dt].apply(ensure_datetime)
cols_to_convert_to_dt = ['start_inflow_ts','end_outflow_ts','release_ts']
production[cols_to_convert_to_dt] = production[cols_to_convert_to_dt].apply(ensure_datetime)

outbound_loads_df.rename(columns={'origin_slot_arrival':'Slot Booked From', 'origin_slot_departure':'Slot Booked To'}, inplace=True)
inbound_loads_df.rename(columns={'destination_slot_arrival':'Slot Booked From', 'destination_slot_departure':'Slot Booked To'}, inplace=True)

In [23]:
print('D0 is between' ,run_time.normalize(), 'and', run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))
print('D1 is between',run_time.normalize()+timedelta(days=1), 'and', run_time.normalize()+ timedelta(days=2))

D0 is between 2025-06-05 00:00:00 and 2025-06-05 23:59:59
D1 is between 2025-06-06 00:00:00 and 2025-06-07 00:00:00


In [24]:
run_time

Timestamp('2025-06-05 13:27:24')

In [25]:
### Getting the outbound and inbound STO loads for D0 and D+1
outbound_loads_df_d0 = outbound_loads_df[outbound_loads_df['Slot Booked From'].between(run_time.normalize(), run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))]
outbound_loads_df_d1 = outbound_loads_df[outbound_loads_df['Slot Booked From'].between(run_time.normalize()+timedelta(days=1), run_time.normalize()+ timedelta(days=2))]

# Changes 5/27 : Pre-load changes
# inbound_loads_df_d0 = inbound_loads_df[inbound_loads_df['Slot Booked From'].between(run_time.normalize(), run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))]
# inbound_loads_df_d1 = inbound_loads_df[inbound_loads_df['Slot Booked From'].between(run_time.normalize()+timedelta(days=1), run_time.normalize()+ timedelta(days=2))]
inbound_loads_df_d0 = inbound_loads_df[inbound_loads_df['planned_movement_ts'].between(run_time.normalize(), run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))]
inbound_loads_df_d1 = inbound_loads_df[inbound_loads_df['planned_movement_ts'].between(run_time.normalize()+timedelta(days=1), run_time.normalize()+ timedelta(days=2))]


load_details_df_d0 = load_details_df[load_details_df['origin_slot_arrival'].between(run_time.normalize(), run_time.normalize()+ timedelta(days=1) - timedelta(seconds= 1))]
load_details_df_d1 = load_details_df[load_details_df['origin_slot_arrival'].between(run_time.normalize()+timedelta(days=1), run_time.normalize()+ timedelta(days=2))]


### Running for D0
## Filtering for future loads from the time of the run
main_outbound_df = outbound_loads_df_d0[outbound_loads_df_d0['Slot Booked From']>run_time]
main_inbound_df = inbound_loads_df_d0
main_load_details = load_details_df_d0[load_details_df_d0['origin_slot_arrival']>run_time].reset_index(drop=True)


swaps_df_d0, main_load_details_d0, updated_stock = process_loads(main_outbound_df, main_inbound_df, main_load_details, stock, open_so, open_sto, pre_load_sto_out_df, production, actual_production, inventory_policy, lcp_data, no_forecast_df, load_details_df, run_time, result_path, 'D0')

Optimizing for D0
## Total Number of Loads:  25


% of loads with original load plan as rectpack feasible 100.0
## Loads that are at risk or have a light load:  10
## Loads that are at risk:  1
## Loads that are a light load:  9
LCP data combinations before removing zero-forecast SKUs with hist. loads:  11224
LCP data combinations after removing zero-forecast SKUs with hist. loads:  11168
## Loads with LCP swaps available:  10
% LCP SKUs with missing HL_PAL 0.0
% LCP SKUs with missing HL_weight 0.0
% LCP SKUs with missing PAL_STACKING_FACTOR 0.0
Started optimization
completed optimization
Optimal
### Number of loads with swap-ins or top-ups:  7
### Number of swap-ins or top-ups:  13
### Number of loads with swap-ins:  1
### Number of swap-ins:  5
### Number of loads with top-ups:  6
### Number of top-ups:  8
### Total planned initial volume in PAL:  616
### Volume at risk in PAL due to insufficient stock at source:  26
### Volume at risk optimized in PAL due to swap-ins 25.0
### Light load top-ups performed in PAL:  12.0
### Average f

In [26]:
swaps_df_d0

,STO Number,load_id,movement_type,sales_document_item_code,Priority Flag,origin_slot_arrival,origin_slot_departure,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,material_sk,material_code,Action,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,demand_at_dt_hl,perc_oos_risk_at_dt_hl,swap_out_qty_hl,suggested_deployment_sr_hl,suggested_deployment_sr_pal,suggested_deployment_sr_pc,suggested_deployment_sr_weight,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_pc,rounded_suggested_deployment_sr_weight,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag,model_run_time_bst
0,4.508683e+09,34800873,STO,10.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,138.996,0.0000,0.0,0.0,0.0000,0.0000,0,0,0.0000,No,,,,D0,2025-06-05 13:27:24
1,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1903370,103436,Swap-in,189.240000,189.24000,0.000000,0.000000,0.000,0.0,0.000000,11207.013344,0.115001,0.000,140.4000,13.0,1170.0,15.2464,140.4000,13,1170,15.2464,No,,,,D0,2025-06-05 13:27:24
2,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1870722,101002,Swap-in,58.320000,58.32000,0.000000,0.000000,0.000,0.0,0.000000,2480.581000,0.000000,0.000,19.2000,2.0,160.0,2.0892,19.2000,2,160,2.0892,No,,,,D0,2025-06-05 13:27:24
3,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,35600,82765,Swap-in,8298.059560,3148.78080,1914.580800,5743.742400,0.000,0.0,2509.044440,908.950864,0.000000,0.000,10.5600,1.0,100.0,1.1520,10.5600,1,100,1.1520,No,,,,D0,2025-06-05 13:27:24
4,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1886284,101371,Swap-in,64.800000,64.80000,0.000000,0.000000,0.000,0.0,0.000000,1341.153200,0.000000,0.000,54.0000,5.0,450.0,5.8640,54.0000,5,450,5.8640,No,,,,D0,2025-06-05 13:27:24
5,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1896054,107451,Swap-in,2044.273123,3791.60448,0.000000,0.000000,0.000,0.0,1747.331357,6759.407386,0.000000,0.000,38.1696,4.0,280.0,4.1480,38.1696,4,280,4.1480,No,,,,D0,2025-06-05 13:27:24
6,4.508683e+09,34801564,STO,0.0,12,2025-06-05 16:00:00,2025-06-05 16:30:00,GB01,GB02,526605,1365010,83898,83855,Top-up (New),627.652200,3890.89680,0.000000,0.000000,101.184,0.0,3162.060600,6233.386604,0.000000,0.000,5.9520,1.0,80.0,0.9490,5.9520,1,80,0.9490,No,,,,D0,2025-06-05 13:27:24
7,4.508683e+09,34801567,STO,0.0,12,2025-06-05 20:00:00,2025-06-05 20:30:00,GB01,GB02,526605,1365010,878558,96797,Top-up (New),644.515900,779.48640,0.000000,0.000000,0.000,0.0,134.970500,118.116123,0.000000,0.000,5.5440,1.0,70.0,0.8746,5.5440,1,70,0.8746,No,,,,D0,2025-06-05 13:27:24
8,4.508683e+09,34801567,STO,0.0,12,2025-06-05 20:00:00,2025-06-05 20:30:00,GB01,GB02,526605,1365010,1954342,98865,Top-up (New),91.158254,539.35200,0.000000,0.000000,0.000,0.0,448.193746,1121.496193,0.028548,0.000,10.2960,1.0,130.0,1.1210,10.2960,1,130,1.1210,No,,,,D0,2025-06-05 13:27:24
9,4.508683e+09,34801567,STO,0.0,12,2025-06-05 20:00:00,2025-06-05 20:30:00,GB01,GB02,526605,1365010,1896054,107451,Top-up (New),2044.273123,3791.60448,0.000000,0.000000,0.000,0.0,1747.331357,17228.538243,0.000000,0.000,9.5424,1.0,70.0,1.0370,9.5424,1,70,1.0370,No,,,,D0,2025-06-05 13:27:24


In [27]:
main_load_details_d0

,RFRC_NUM12,load_id,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,Action,original_quantity_ordered_hl,original_quantity_ordered_pal,original_quantity_ordered_kg,total_area_in_cm2,total_feasible_order_qty_hl,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area,available_pal,available_weight,available_area,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_area,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag,model_run_time_bst
0,4.508683e+09,34800873,STO,12,GB01,GB28,526605,1537973,2025-06-05 23:00:00,2025-06-05 23:30:00,2025-06-06 00:00:00,2025-06-06 01:00:00,At risk,138.9960,26,24713.00,312000.0,0.0000,0,0.000,0.0,34,28.50000,326400.0,262.3296,25.0,28.500,300000.0,No,,,,D0,2025-06-05 13:27:24
1,4.508683e+09,34801204,STO,12,GB01,GB02,526605,1365010,2025-06-05 16:00:00,2025-06-05 16:30:00,2025-06-05 21:00:00,2025-06-05 21:30:00,Load not at risk,154.4400,26,26962.00,312000.0,154.4400,26,26.962,312000.0,0,1.53800,0.0,154.4400,26.0,26.962,312000.0,No,,,,D0,2025-06-05 13:27:24
2,4.508683e+09,34801205,STO,12,GB02,GB01,1365010,526605,2025-06-05 21:00:00,2025-06-05 21:30:00,NaT,NaT,Load not at risk,181.9224,26,27563.56,312000.0,181.9224,26,27.564,312000.0,0,0.93644,0.0,181.9224,26.0,27.564,312000.0,No,,,,D0,2025-06-05 13:27:24
3,4.508683e+09,34800586,STO,12,GB02,GB28,1365010,1537973,2025-06-05 18:00:00,2025-06-05 18:30:00,2025-06-05 21:00:00,2025-06-05 22:00:00,Light load,221.6706,25,28168.40,300000.0,221.6706,25,28.168,300000.0,2,0.33160,19200.0,221.6706,25.0,28.168,300000.0,No,,,,D0,2025-06-05 13:27:24
4,4.508683e+09,34800875,STO,12,GB01,GB28,526605,1537973,2025-06-05 19:30:00,2025-06-05 20:00:00,2025-06-05 23:00:00,2025-06-06 00:00:00,Load not at risk,168.3000,26,27036.00,312000.0,168.3000,26,27.036,312000.0,0,1.46400,0.0,168.3000,26.0,27.036,312000.0,No,,,,D0,2025-06-05 13:27:24
5,4.508683e+09,34800878,STO,12,GB01,GB28,526605,1537973,2025-06-05 20:00:00,2025-06-05 20:30:00,2025-06-05 21:00:00,2025-06-05 22:00:00,Load not at risk,196.2240,26,26618.00,312000.0,196.2240,26,26.618,312000.0,0,1.88200,0.0,196.2240,26.0,26.618,312000.0,No,,,,D0,2025-06-05 13:27:24
6,4.508683e+09,34801564,STO,12,GB01,GB02,526605,1365010,2025-06-05 16:00:00,2025-06-05 16:30:00,2025-06-05 20:00:00,2025-06-05 20:30:00,Light load,210.4000,17,27540.00,269280.0,210.4000,17,27.540,269280.0,5,0.96000,57120.0,216.3520,18.0,28.489,281280.0,No,,,,D0,2025-06-05 13:27:24
7,4.508683e+09,34801566,STO,12,GB02,GB01,1365010,526605,2025-06-05 16:00:00,2025-06-05 16:30:00,2025-06-05 20:00:00,2025-06-05 20:30:00,Load not at risk,246.1824,24,28403.60,288000.0,246.1824,24,28.404,288000.0,4,0.09640,38400.0,246.1824,24.0,28.404,288000.0,No,,,,D0,2025-06-05 13:27:24
8,4.508683e+09,34801565,STO,12,GB02,GB01,1365010,526605,2025-06-05 20:00:00,2025-06-05 20:30:00,2025-06-06 03:00:00,2025-06-06 03:30:00,Light load,231.8622,25,27546.30,300000.0,231.8622,25,27.546,300000.0,2,0.95370,19200.0,237.8142,26.0,28.495,312000.0,No,,,,D0,2025-06-05 13:27:24
9,4.508683e+09,34800633,STO,12,GB01,GB02,526605,1365010,2025-06-05 17:00:00,2025-06-05 17:30:00,2025-06-06 21:00:00,2025-06-06 21:30:00,Load not at risk,247.1040,26,26962.00,312000.0,247.1040,26,26.962,312000.0,0,1.53800,0.0,247.1040,26.0,26.962,312000.0,No,,,,D0,2025-06-05 13:27:24


In [28]:
main_load_details_d0[main_load_details_d0['Action']!= 'Load not at risk'].sort_values(['Action'])

,RFRC_NUM12,load_id,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,Action,original_quantity_ordered_hl,original_quantity_ordered_pal,original_quantity_ordered_kg,total_area_in_cm2,total_feasible_order_qty_hl,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area,available_pal,available_weight,available_area,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_area,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag,model_run_time_bst
0,4.508683e+09,34800873,STO,12,GB01,GB28,526605,1537973,2025-06-05 23:00:00,2025-06-05 23:30:00,2025-06-06 00:00:00,2025-06-06 01:00:00,At risk,138.9960,26,24713.0,312000.0,0.0000,0,0.000,0.0,34,28.5000,326400.0,262.3296,25.0,28.500,300000.0,No,,,,D0,2025-06-05 13:27:24
3,4.508683e+09,34800586,STO,12,GB02,GB28,1365010,1537973,2025-06-05 18:00:00,2025-06-05 18:30:00,2025-06-05 21:00:00,2025-06-05 22:00:00,Light load,221.6706,25,28168.4,300000.0,221.6706,25,28.168,300000.0,2,0.3316,19200.0,221.6706,25.0,28.168,300000.0,No,,,,D0,2025-06-05 13:27:24
6,4.508683e+09,34801564,STO,12,GB01,GB02,526605,1365010,2025-06-05 16:00:00,2025-06-05 16:30:00,2025-06-05 20:00:00,2025-06-05 20:30:00,Light load,210.4000,17,27540.0,269280.0,210.4000,17,27.540,269280.0,5,0.9600,57120.0,216.3520,18.0,28.489,281280.0,No,,,,D0,2025-06-05 13:27:24
8,4.508683e+09,34801565,STO,12,GB02,GB01,1365010,526605,2025-06-05 20:00:00,2025-06-05 20:30:00,2025-06-06 03:00:00,2025-06-06 03:30:00,Light load,231.8622,25,27546.3,300000.0,231.8622,25,27.546,300000.0,2,0.9537,19200.0,237.8142,26.0,28.495,312000.0,No,,,,D0,2025-06-05 13:27:24
12,4.508683e+09,34801567,STO,12,GB01,GB02,526605,1365010,2025-06-05 20:00:00,2025-06-05 20:30:00,2025-06-06 03:00:00,2025-06-06 03:30:00,Light load,204.8000,17,25456.8,269280.0,204.8000,17,25.457,269280.0,5,3.0432,57120.0,230.1824,20.0,28.489,305280.0,No,,,,D0,2025-06-05 13:27:24
14,4.508683e+09,34800752,STO,11,GB02,GB67,1365010,1565895,2025-06-05 23:00:00,2025-06-05 23:30:00,2025-06-06 06:00:00,2025-06-06 07:00:00,Light load,250.6680,25,27460.5,300000.0,250.6680,25,27.460,300000.0,2,1.0395,19200.0,260.2104,26.0,28.498,312000.0,No,,,,D0,2025-06-05 13:27:24
20,4.508683e+09,34801431,STO,12,GB02,GB67,1365010,1565895,2025-06-05 23:00:00,2025-06-05 23:30:00,2025-06-06 06:00:00,2025-06-06 07:00:00,Light load,256.6080,24,27864.0,288000.0,256.6080,24,27.864,288000.0,4,0.6360,38400.0,256.6080,24.0,27.864,288000.0,No,,,,D0,2025-06-05 13:27:24
21,4.508683e+09,34801432,STO,12,GB02,GB67,1365010,1565895,2025-06-05 16:00:00,2025-06-05 16:30:00,2025-06-06 07:00:00,2025-06-06 08:00:00,Light load,256.6080,24,27864.0,288000.0,256.6080,24,27.864,288000.0,4,0.6360,38400.0,256.6080,24.0,27.864,288000.0,No,,,,D0,2025-06-05 13:27:24
23,4.508688e+09,34806473,STO,0,GB28,IE06,1537973,5229203,2025-06-05 14:00:00,2025-06-05 15:00:00,2025-06-09 15:00:00,2025-06-09 15:30:00,Light load,130.6800,22,23584.0,264000.0,130.6800,22,23.584,264000.0,6,4.9160,62400.0,173.8800,26.0,28.275,312000.0,No,,,,D0,2025-06-05 13:27:24
24,4.508688e+09,34806474,STO,0,GB28,IE06,1537973,5229203,2025-06-05 15:00:00,2025-06-05 16:00:00,2025-06-09 16:00:00,2025-06-09 16:30:00,Light load,142.9200,24,25731.8,288000.0,142.9200,24,25.732,288000.0,4,2.7682,38400.0,164.5200,26.0,28.077,312000.0,No,,,,D0,2025-06-05 13:27:24


In [29]:
swaps_df_d0[['load_id','material_sk','Action','swap_out_qty_hl','rounded_suggested_deployment_sr_hl','rounded_suggested_deployment_sr_pal','rounded_suggested_deployment_sr_weight', 'Cancel_load']]

,load_id,material_sk,Action,swap_out_qty_hl,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,Cancel_load
0,34800873,1870515,Swap-out (Delete),138.996,0.0000,0,0.0000,No
1,34800873,1903370,Swap-in,0.000,140.4000,13,15.2464,No
2,34800873,1870722,Swap-in,0.000,19.2000,2,2.0892,No
3,34800873,35600,Swap-in,0.000,10.5600,1,1.1520,No
4,34800873,1886284,Swap-in,0.000,54.0000,5,5.8640,No
5,34800873,1896054,Swap-in,0.000,38.1696,4,4.1480,No
6,34801564,83898,Top-up (New),0.000,5.9520,1,0.9490,No
7,34801567,878558,Top-up (New),0.000,5.5440,1,0.8746,No
8,34801567,1954342,Top-up (New),0.000,10.2960,1,1.1210,No
9,34801567,1896054,Top-up (New),0.000,9.5424,1,1.0370,No


In [30]:
swaps_df_d0.sort_values('load_id')

,STO Number,load_id,movement_type,sales_document_item_code,Priority Flag,origin_slot_arrival,origin_slot_departure,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,material_sk,material_code,Action,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,demand_at_dt_hl,perc_oos_risk_at_dt_hl,swap_out_qty_hl,suggested_deployment_sr_hl,suggested_deployment_sr_pal,suggested_deployment_sr_pc,suggested_deployment_sr_weight,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_pc,rounded_suggested_deployment_sr_weight,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag,model_run_time_bst
11,4.508683e+09,34800752,STO,0.0,11,2025-06-05 23:00:00,2025-06-05 23:30:00,GB02,GB67,1365010,1565895,1896054,107451,Top-up (New),2947.095200,1278.68160,850.596706,2041.432094,0.000,0.0,1223.615200,2175.513112,0.000000,0.000,9.5424,1.0,70.0,1.0370,9.5424,1,70,1.0370,No,,,,D0,2025-06-05 13:27:24
0,4.508683e+09,34800873,STO,10.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,138.996,0.0000,0.0,0.0,0.0000,0.0000,0,0,0.0000,No,,,,D0,2025-06-05 13:27:24
1,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1903370,103436,Swap-in,189.240000,189.24000,0.000000,0.000000,0.000,0.0,0.000000,11207.013344,0.115001,0.000,140.4000,13.0,1170.0,15.2464,140.4000,13,1170,15.2464,No,,,,D0,2025-06-05 13:27:24
2,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1870722,101002,Swap-in,58.320000,58.32000,0.000000,0.000000,0.000,0.0,0.000000,2480.581000,0.000000,0.000,19.2000,2.0,160.0,2.0892,19.2000,2,160,2.0892,No,,,,D0,2025-06-05 13:27:24
3,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,35600,82765,Swap-in,8298.059560,3148.78080,1914.580800,5743.742400,0.000,0.0,2509.044440,908.950864,0.000000,0.000,10.5600,1.0,100.0,1.1520,10.5600,1,100,1.1520,No,,,,D0,2025-06-05 13:27:24
4,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1886284,101371,Swap-in,64.800000,64.80000,0.000000,0.000000,0.000,0.0,0.000000,1341.153200,0.000000,0.000,54.0000,5.0,450.0,5.8640,54.0000,5,450,5.8640,No,,,,D0,2025-06-05 13:27:24
5,4.508683e+09,34800873,STO,0.0,12,2025-06-05 23:00:00,2025-06-05 23:30:00,GB01,GB28,526605,1537973,1896054,107451,Swap-in,2044.273123,3791.60448,0.000000,0.000000,0.000,0.0,1747.331357,6759.407386,0.000000,0.000,38.1696,4.0,280.0,4.1480,38.1696,4,280,4.1480,No,,,,D0,2025-06-05 13:27:24
6,4.508683e+09,34801564,STO,0.0,12,2025-06-05 16:00:00,2025-06-05 16:30:00,GB01,GB02,526605,1365010,83898,83855,Top-up (New),627.652200,3890.89680,0.000000,0.000000,101.184,0.0,3162.060600,6233.386604,0.000000,0.000,5.9520,1.0,80.0,0.9490,5.9520,1,80,0.9490,No,,,,D0,2025-06-05 13:27:24
10,4.508683e+09,34801565,STO,0.0,12,2025-06-05 20:00:00,2025-06-05 20:30:00,GB02,GB01,1365010,526605,83898,83855,Top-up (New),76.921294,1368.14160,0.000000,0.000000,0.000,0.0,1291.220306,7816.740512,0.000000,0.000,5.9520,1.0,80.0,0.9490,5.9520,1,80,0.9490,No,,,,D0,2025-06-05 13:27:24
7,4.508683e+09,34801567,STO,0.0,12,2025-06-05 20:00:00,2025-06-05 20:30:00,GB01,GB02,526605,1365010,878558,96797,Top-up (New),644.515900,779.48640,0.000000,0.000000,0.000,0.0,134.970500,118.116123,0.000000,0.000,5.5440,1.0,70.0,0.8746,5.5440,1,70,0.8746,No,,,,D0,2025-06-05 13:27:24


In [31]:
### Running for D1:
run_time_d1 = run_date +timedelta(days =1 )
print('run_time_d1 : ', run_time_d1)
main_outbound_df = outbound_loads_df_d1
main_inbound_df = inbound_loads_df_d1
main_load_details = load_details_df_d1[load_details_df_d1['origin_slot_arrival']>run_time_d1].reset_index(drop=True)
updated_stock['opening_stock_hl'] = updated_stock['Closing_Stock']

swaps_df_d1, main_load_details_d1 = process_loads(main_outbound_df, main_inbound_df, main_load_details, updated_stock, open_so, open_sto, pre_load_sto_out_df, production, actual_production, inventory_policy, lcp_data, no_forecast_df, load_details_df, run_time_d1, result_path, 'D1')


run_time_d1 :  2025-06-06 00:00:00
Optimizing for D1
## Total Number of Loads:  18
% of loads with original load plan as rectpack feasible 100.0
## Loads that are at risk or have a light load:  12
## Loads that are at risk:  4
## Loads that are a light load:  8
LCP data combinations before removing zero-forecast SKUs with hist. loads:  11224
LCP data combinations after removing zero-forecast SKUs with hist. loads:  11168
## Loads with LCP swaps available:  12
% LCP SKUs with missing HL_PAL 0.0
% LCP SKUs with missing HL_weight 0.0
% LCP SKUs with missing PAL_STACKING_FACTOR 0.0
Started optimization
completed optimization
Optimal
### Number of loads with swap-ins or top-ups:  12
### Number of swap-ins or top-ups:  28
### Number of loads with swap-ins:  4
### Number of swap-ins:  17
### Number of loads with top-ups:  8
### Number of top-ups:  11
### Total planned initial volume in PAL:  438
### Volume at risk in PAL due to insufficient stock at source:  87
### Volume at risk optimized in

In [32]:
print('Completed Running optimizer')

Completed Running optimizer


In [33]:
main_load_details_d1

,RFRC_NUM12,load_id,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,Action,original_quantity_ordered_hl,original_quantity_ordered_pal,original_quantity_ordered_kg,total_area_in_cm2,total_feasible_order_qty_hl,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area,available_pal,available_weight,available_area,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_area,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag,model_run_time_bst
0,4.508681e+09,34798240,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,Light load,141.1344,33,25773.00,316800.0,141.1344,33,25.773,316800.0,1,2.72700,9600.0,145.4112,34.0,26.545,326400.0,No,,,,D1,2025-06-06
1,4.508681e+09,34798241,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,Light load,141.1344,33,25773.00,316800.0,141.1344,33,25.773,316800.0,1,2.72700,9600.0,145.4112,34.0,26.545,326400.0,No,,,,D1,2025-06-06
2,4.508681e+09,34798242,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,Light load,141.1344,33,25773.00,316800.0,141.1344,33,25.773,316800.0,1,2.72700,9600.0,145.4112,34.0,26.545,326400.0,No,,,,D1,2025-06-06
3,4.508683e+09,34800871,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,At risk,138.9960,26,24713.00,312000.0,0.0000,0,0.000,0.0,34,28.50000,326400.0,262.1760,25.0,28.494,300000.0,No,,,,D1,2025-06-06
4,4.508683e+09,34800872,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,At risk,138.9960,26,24713.00,312000.0,0.0000,0,0.000,0.0,34,28.50000,326400.0,262.2672,25.0,28.498,300000.0,No,,,,D1,2025-06-06
5,4.508683e+09,34801610,STO,12,GB02,GB01,1365010,526605,2025-06-06 20:00:00,2025-06-06 20:30:00,2025-06-07 03:00:00,2025-06-07 03:30:00,Load not at risk,204.7536,26,27651.92,312000.0,204.7536,26,27.652,312000.0,0,0.84808,0.0,204.7536,26.0,27.652,312000.0,No,,,,D1,2025-06-06
6,4.508683e+09,34801622,STO,12,GB02,GB01,1365010,526605,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,Light load,237.6768,23,27554.40,276000.0,237.6768,23,27.554,276000.0,5,0.94560,50400.0,243.3792,24.0,28.482,288000.0,No,,,,D1,2025-06-06
7,4.508683e+09,34800876,STO,12,GB01,GB28,526605,1537973,2025-06-06 07:00:00,2025-06-06 07:30:00,NaT,NaT,At risk,140.0652,26,24889.58,312000.0,17.1072,3,3.028,36000.0,30,25.47192,290400.0,251.0256,25.0,28.466,300000.0,No,,,,D1,2025-06-06
8,4.508683e+09,34800877,STO,12,GB01,GB28,526605,1537973,2025-06-06 04:00:00,2025-06-06 04:30:00,2025-06-06 05:00:00,2025-06-06 06:00:00,Load not at risk,244.8000,26,27297.10,312000.0,244.8000,26,27.297,312000.0,0,1.20290,0.0,244.8000,26.0,27.297,312000.0,No,,,,D1,2025-06-06
9,4.508683e+09,34801568,STO,12,GB01,GB02,526605,1365010,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,Light load,212.0000,15,23839.20,237600.0,212.0000,15,23.839,237600.0,9,4.66080,88800.0,254.7680,19.0,28.499,285600.0,No,,,,D1,2025-06-06


In [34]:
swaps_df_d1

,STO Number,load_id,movement_type,sales_document_item_code,Priority Flag,origin_slot_arrival,origin_slot_departure,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,material_sk,material_code,Action,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,demand_at_dt_hl,perc_oos_risk_at_dt_hl,swap_out_qty_hl,suggested_deployment_sr_hl,suggested_deployment_sr_pal,suggested_deployment_sr_pc,suggested_deployment_sr_weight,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_pc,rounded_suggested_deployment_sr_weight,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag,model_run_time_bst
0,4.508690e+09,34808282,STO,10.0,0,2025-06-06 14:00:00,2025-06-06 15:00:00,GB28,IE06,1537973,5229203,2052964,99803,Swap-out (Update),68.587200,68.58720,0.00000,0.0000,0.000,0.0,0.000000,0.000000,0.000000,68.0328,68.5872,11.546667,1732.0,12.585867,65.3400,11,1650,11.99000,No,,,,D1,2025-06-06
1,4.508683e+09,34800871,STO,10.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.00000,0.0000,0.000,0.0,0.000000,0.000000,0.000000,138.9960,0.0000,0.000000,0.0,0.000000,0.0000,0,0,0.00000,No,,,,D1,2025-06-06
2,4.508683e+09,34800872,STO,10.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.00000,0.0000,0.000,0.0,0.000000,0.000000,0.000000,138.9960,0.0000,0.000000,0.0,0.000000,0.0000,0,0,0.00000,No,,,,D1,2025-06-06
3,4.508683e+09,34800876,STO,20.0,12,2025-06-06 07:00:00,2025-06-06 07:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.00000,0.0000,277.992,0.0,0.000000,0.000000,0.000000,122.9580,0.0000,0.000000,0.0,0.000000,0.0000,0,0,0.00000,No,,,,D1,2025-06-06
4,4.508681e+09,34798240,STO,0.0,11,2025-06-06 01:00:00,2025-06-06 02:00:00,GB28,IT12,1537973,2967078,1869530,105531,Top-up (New),547.984800,547.98480,0.00000,0.0000,0.000,0.0,0.000000,16848.498400,0.676409,0.0000,4.2768,1.000000,54.0,0.772360,4.2768,1,54,0.77236,No,,,,D1,2025-06-06
5,4.508681e+09,34798241,STO,0.0,11,2025-06-06 01:00:00,2025-06-06 02:00:00,GB28,IT12,1537973,2967078,1869530,105531,Top-up (New),547.984800,547.98480,0.00000,0.0000,0.000,0.0,0.000000,16848.498400,0.676409,0.0000,4.2768,1.000000,54.0,0.772360,4.2768,1,54,0.77236,No,,,,D1,2025-06-06
6,4.508681e+09,34798242,STO,0.0,11,2025-06-06 01:00:00,2025-06-06 02:00:00,GB28,IT12,1537973,2967078,1869530,105531,Top-up (New),547.984800,547.98480,0.00000,0.0000,0.000,0.0,0.000000,16848.498400,0.676409,0.0000,4.2768,1.000000,54.0,0.772360,4.2768,1,54,0.77236,No,,,,D1,2025-06-06
7,4.508683e+09,34800871,STO,0.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1903370,103436,Swap-in,48.840000,48.84000,0.00000,0.0000,0.000,0.0,0.000000,11304.213344,0.122611,0.0000,21.6000,2.000000,180.0,2.345600,21.6000,2,180,2.34560,No,,,,D1,2025-06-06
8,4.508683e+09,34800871,STO,0.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,76116,83038,Swap-in,20.135520,20.13552,0.00000,0.0000,0.000,0.0,0.000000,2818.178840,0.000000,0.0000,10.2384,1.000000,90.0,1.130500,10.2384,1,90,1.13050,No,,,,D1,2025-06-06
9,4.508683e+09,34800871,STO,0.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1871005,103666,Swap-in,222.142700,397.90080,0.00000,0.0000,0.000,0.0,175.758100,108.294943,0.000000,0.0000,10.2960,1.000000,130.0,1.121000,10.2960,1,130,1.12100,No,,,,D1,2025-06-06


General Testing - Start checkpoint - 
All code below this cell is only for manual testing

In [35]:
#testing start for process_loads
# swaps_df_d0, main_load_details_d0, updated_stock = process_loads(main_outbound_df, main_inbound_df, main_load_details, stock, open_so, open_sto, production, actual_production, inventory_policy, lcp_data, load_details_df, run_time, result_path, 'D0')

In [36]:
main_outbound_df_backup = main_outbound_df.copy()
main_inbound_df_backup = main_inbound_df.copy()
main_load_details_backup = main_load_details.copy()
open_so_backup = open_so.copy()
open_sto_backup = open_sto.copy()
stock_backup = stock.copy()
production_backup = production.copy()
actual_production_backup = actual_production.copy()
inventory_policy_backup = inventory_policy.copy()
lcp_data_backup = lcp_data.copy()
load_details_df_backup = load_details_df.copy()

# main_outbound_df = main_outbound_df_backup.copy()
# main_inbound_df = main_inbound_df_backup.copy()
# main_load_details = main_load_details_backup.copy()
# open_so = open_so_backup.copy()
# open_sto = open_sto_backup.copy()
# stock = stock_backup.copy()
# production = production_backup.copy()
# actual_production = actual_production_backup.copy()
# inventory_policy = inventory_policy_backup.copy()
# lcp_data = lcp_data_backup.copy()
# load_details_df = load_details_df_backup.copy()

In [37]:
tag = 'D0'
kpi_results    = {}
print('Optimizing for '+tag)

Optimizing for D0


In [38]:
main_outbound_df = main_outbound_df.sort_values(['Priority Flag', 'Slot Booked From']).reset_index(drop=True)
print('## Total Number of Loads: ', main_outbound_df['load_id'].nunique())
kpi_results['Total number of loads'] = {
    'Value': main_outbound_df['load_id'].nunique(),
    'Percentage': 100
}

## Total Number of Loads:  18


In [39]:
# changes 5/27 : Pre-load changes
open_sto_out = pd.merge(main_outbound_df, open_sto.loc[:,~open_sto.columns.isin(['trailer_equipment_type_code','actual_loading_start_ts','planned_movement_ts'])], on=['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 'Priority Flag'], how='inner')
open_sto_in = pd.merge(main_inbound_df, open_sto.loc[:,~open_sto.columns.isin(['trailer_equipment_type_code','actual_loading_start_ts','planned_movement_ts'])], on=['load_id', 'Source', 'RFRC_NUM12', 'movement_type', 'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 'Priority Flag'] ,how='inner')

In [40]:
#testing start for calculate_stock_available_sr
#runs row wise
# open_sto_out[['Stock_on_hand_sr(HL)', 'Stock_sr', 'Planned_production_sr', 'Actual_production_sr', 'Outgoing_SO_STO_sr', 'Incoming_STO_sr', 'Safety_Stock_sr']] = \
#         open_sto_out.apply(lambda row: pd.Series(calculate_stock_available_sr(row, stock, open_so, open_sto_in, open_sto_out, production, actual_production, inventory_policy, run_time, 'INIT')), axis=1)


In [41]:
load = open_sto_out.iloc[0,:]
#load = open_sto_out.loc[open_sto_out['load_id'] == 34776223].iloc[0,:]
# 34776221
# 34776223

In [42]:
load

RFRC_NUM12                                 4508690315.0
load_id                                        34808282
trailer_equipment_type_code                     ZGBTR26
movement_type                                       STO
Priority Flag                                         0
Source                                             GB28
Destination                                        IE06
origin_shipping_location_sk                     1537973
destination_shipping_location_sk                5229203
Slot Booked From                    2025-06-06 14:00:00
Slot Booked To                      2025-06-06 15:00:00
actual_loading_start_ts                             NaT
planned_movement_ts                 2025-06-06 00:00:00
sales_document_item_code                             10
origin_slot_arrival                 2025-06-06 14:00:00
origin_slot_departure               2025-06-06 15:00:00
destination_slot_arrival            2025-06-10 15:00:00
destination_slot_departure          2025-06-10 1

In [43]:
sku = load['material_sk']
source = load['Source']
slot_booking_time = load['Slot Booked From']
priority_flag = load['Priority Flag']

In [44]:
stock_at_hand = stock.loc[
            (stock['material_sk'] == sku) & (stock['Source'] == source), 'opening_stock_hl'].values[0]

In [45]:
stock_at_hand

341.8272

In [46]:
outgoing_so = open_so[
        (open_so['material_sk'] == sku) &
        (open_so['Source'] == source) &
        (open_so['Delivery Date'] == run_time.normalize())
    ]['open_so_out_hl'].sum()

In [47]:
outgoing_so

0.0

In [48]:
#assumption on 1 hour lag required for movement of stock after incoming_sto and before outgoing_sto
#removed buffer of 1 hour
incoming_sto = open_sto_in[
        (open_sto_in['material_sk'] == sku) &
        (open_sto_in['Destination'] == source) &
        (open_sto_in['Slot Booked From'] >= run_time) &
        (open_sto_in['Slot Booked From'] <= slot_booking_time - timedelta(hours=1))
    ]['total_quantity_hl'].sum()

In [49]:
incoming_sto

0.0

In [50]:
#change done to account for delay between end_outflow_ts and release_ts
#also removed conditions on start_inflow_ts
#removed filter for 1 hour
planned_production = production[
        (production['material_sk'] == sku) &
        (production['plant_code'] == source) &
        # (production['release_ts'] >= run_time) &
        (production['release_ts'] <= slot_booking_time)
    ]['production_hl'].sum()

In [51]:
planned_production

0.0

In [52]:
actual_prod = actual_production[
    (actual_production['material_sk'] == sku) &
    (actual_production['plant_code'] == source)
]['production_hl'].sum()


In [53]:
actual_prod

0.0

In [54]:
#with caveat on priority lesser than current priority
#removed priority flag filter
outgoing_sto = open_sto_out[
        (open_sto_out['material_sk'] == sku) &
        (open_sto_out['Source'] == source) &
        (open_sto_out['Slot Booked From'] > run_time) &
        (open_sto_out['Slot Booked From'] < slot_booking_time)
    ]['total_quantity_hl'].sum()

In [55]:
outgoing_sto

0.0

In [56]:
run_type = 'INIT'

In [57]:
try: 
    # Safety stock
    safety_stock = inventory_policy.loc[(inventory_policy['material_sk'] == sku)&(inventory_policy['Source']==source), 'safety_stock_hl'].values[0]
except IndexError:
    safety_stock = 0

In [58]:
safety_stock

496.452

In [59]:
if run_type == 'INIT':
    safety_stock = 0
elif run_type == 'TOP-UP':
    safety_stock = safety_stock

In [60]:
stock_at_hand, outgoing_so, incoming_sto, planned_production, actual_prod, outgoing_sto, safety_stock

(341.8272, 0.0, 0.0, 0.0, 0.0, 0.0, 0)

In [61]:
stock_available = (stock_at_hand - outgoing_so + incoming_sto + planned_production + actual_prod - outgoing_sto - safety_stock)

In [62]:
stock_available

341.8272

In [63]:
#testing end for calculate_stock_available_sr
# return stock_available, stock_at_hand, planned_production, actual_prod, outgoing_so + outgoing_sto, incoming_sto, safety_stock

In [64]:
#changes 4/20 : Area based optimization
# changes 5/27 : Pre-load changes
open_sto_out[['stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl']] = \
        open_sto_out.apply(lambda row: pd.Series(calculate_stock_available_sr(row, stock, open_so, open_sto_in, open_sto_out, pre_load_sto_out_df, production, actual_production, inventory_policy, run_time, 'INIT')), axis=1)

In [65]:
open_sto_out.sort_values(['Source','material_sk','Slot Booked From'],inplace=True)

In [66]:
open_sto_out[['planned_production_sr_hl','actual_production_sr_hl','outgoing_so_sto_sr_hl','incoming_sto_sr_hl','safety_stock_sr_hl']].apply(lambda x:x.sum(),axis=0)

planned_production_sr_hl    1322.894925
actual_production_sr_hl        0.000000
outgoing_so_sto_sr_hl        674.914400
incoming_sto_sr_hl             0.000000
safety_stock_sr_hl             0.000000
dtype: float64

In [67]:
open_sto_out.loc[open_sto_out['actual_production_sr_hl']==0,['Source','material_sk']].drop_duplicates().sort_values(['Source','material_sk'])

,Source,material_sk
5,GB01,25894
8,GB01,31402
35,GB01,43883
7,GB01,47482
34,GB01,48299
33,GB01,49844
21,GB01,52804
13,GB01,57482
15,GB01,68916
36,GB01,72351


In [68]:
open_sto_out['at_risk_flag'] = np.where(open_sto_out['stock_on_hand_sr_hl'] < open_sto_out['total_quantity_hl'], True, False)
#In cases where there is no SOH at source, no incoming but outgoing due
open_sto_out['stock_on_hand_sr_hl'] = np.where(open_sto_out['stock_on_hand_sr_hl'] < 0, 0, open_sto_out['stock_on_hand_sr_hl'])
open_sto_out['qty_at_risk_hl'] = np.where(open_sto_out['total_quantity_hl'] - open_sto_out['stock_on_hand_sr_hl'] <= 0, 0, open_sto_out['total_quantity_hl'] - open_sto_out['stock_on_hand_sr_hl'])
open_sto_out['total_feasible_order_qty_hl'] = np.where(open_sto_out['at_risk_flag'] == True, open_sto_out['total_quantity_hl'] - open_sto_out['qty_at_risk_hl'], open_sto_out['total_quantity_hl'])

In [69]:
kpi_results

{'Total number of loads': {'Value': 18, 'Percentage': 100}}

In [70]:
#changes 4/14 : keg conversion and rounding
open_sto_out['total_feasible_order_qty_pc'] = open_sto_out['total_feasible_order_qty_hl'] / open_sto_out['material_sk'].map(pc_to_hl_dict)
open_sto_out['total_feasible_order_qty_pal'] = np.where(open_sto_out['container_type_description'].str.upper() == 'KEG',open_sto_out['total_feasible_order_qty_pc'] * open_sto_out['material_sk'].map(pc_to_pal_dict) / open_sto_out['PAL_STACKING_FACTOR'], open_sto_out['total_feasible_order_qty_pc'] * open_sto_out['material_sk'].map(pc_to_pal_dict))
#changes 4/20 : Area based optimization
open_sto_out['total_feasible_order_qty_pal'] = open_sto_out['total_feasible_order_qty_pal'].apply(custom_round)
open_sto_out['total_feasible_order_qty_weight'] = np.where(open_sto_out['container_type_description'].str.upper() == 'KEG', open_sto_out['total_feasible_order_qty_pal'] * open_sto_out['material_sk'].map(pal_weight_dict) * open_sto_out['PAL_STACKING_FACTOR'], open_sto_out['total_feasible_order_qty_pal'] * open_sto_out['material_sk'].map(pal_weight_dict))
#changes 4/20 : Area based optimization
open_sto_out['total_feasible_order_area'] = open_sto_out['total_feasible_order_qty_pal'] * open_sto_out['pal_length'] * open_sto_out['pal_width']

In [71]:
sto_cols_to_show = ['load_id','material_sk','container_type_description','container_size_description','at_risk_flag','total_quantity_hl','total_quantity_pal', 'qty_at_risk_hl', 'total_feasible_order_qty_hl','total_feasible_order_qty_pal']

In [72]:
open_sto_out.loc[open_sto_out['at_risk_flag']==True,sto_cols_to_show]

,load_id,material_sk,container_type_description,container_size_description,at_risk_flag,total_quantity_hl,total_quantity_pal,qty_at_risk_hl,total_feasible_order_qty_hl,total_feasible_order_qty_pal
11,34800871,1870515,BOTTLE,"0,330 L",True,138.996,26,138.996,0.0,0
12,34800872,1870515,BOTTLE,"0,330 L",True,138.996,26,138.996,0.0,0
23,34800876,1870515,BOTTLE,"0,330 L",True,122.958,23,122.958,0.0,0


In [73]:
open_sto_out.loc[:,sto_cols_to_show]

,load_id,material_sk,container_type_description,container_size_description,at_risk_flag,total_quantity_hl,total_quantity_pal,qty_at_risk_hl,total_feasible_order_qty_hl,total_feasible_order_qty_pal
5,34800720,25894,BOTTLE,"0,330 L",False,148.5000,25,0.000,148.5000,25
6,34800698,25894,BOTTLE,"0,330 L",False,154.4400,26,0.000,154.4400,26
8,34800874,31402,CAN,"0,440 L",False,158.4000,16,0.000,158.4000,16
35,34801568,43883,KEG,"50,000 L",False,92.0000,7,0.000,92.0000,7
48,34801613,43883,KEG,"50,000 L",False,48.0000,4,0.000,48.0000,4
7,34800874,47482,CAN,"0,440 L",False,39.6000,4,0.000,39.6000,4
34,34801568,48299,KEG,"30,000 L",False,2.4000,0,0.000,2.4000,0
33,34801568,49844,KEG,"30,000 L",False,2.4000,0,0.000,2.4000,0
21,34800877,52804,BOTTLE,"0,250 L",False,6.2400,1,0.000,6.2400,1
13,34800880,57482,BOTTLE,"0,330 L",False,5.7024,1,0.000,5.7024,1


In [74]:
open_sto_out.sort_values('load_id')

,RFRC_NUM12,load_id,trailer_equipment_type_code,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,Slot Booked From,Slot Booked To,actual_loading_start_ts,planned_movement_ts,sales_document_item_code,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,material_sk,material_code,unit_of_measure_code,container_type_description,container_size_description,pal_length,pal_width,pal_height,total_quantity_pc,total_quantity_hl,PC_HL,PC_PAL,PAL_WEIGHT_KG,PAL_STACKING_FACTOR,total_quantity_pal,total_quantity_kg,total_area_in_cm2,pal_config,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,at_risk_flag,qty_at_risk_hl,total_feasible_order_qty_hl,total_feasible_order_qty_pc,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area
2,4.508681e+09,34798240,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,2025-06-06,10,2025-06-06 01:00:00,2025-06-06 02:00:00,NaN,NaN,2073737,99923,PC,BOTTLE,"0,330 L",120.0,80.0,149.40,1782.0,141.1344,0.07920,0.018519,781.00,1.0,33,25773.00,316800.0,120.0x80.0,641.757600,641.75760,0.000000,0.0,0.0000,0.0,0.0,False,0.000,141.1344,1782.0,33,25773.00,316800.0
3,4.508681e+09,34798241,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,2025-06-06,10,2025-06-06 01:00:00,2025-06-06 02:00:00,NaN,NaN,2073737,99923,PC,BOTTLE,"0,330 L",120.0,80.0,149.40,1782.0,141.1344,0.07920,0.018519,781.00,1.0,33,25773.00,316800.0,120.0x80.0,641.757600,641.75760,0.000000,0.0,0.0000,0.0,0.0,False,0.000,141.1344,1782.0,33,25773.00,316800.0
4,4.508681e+09,34798242,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,2025-06-06,10,2025-06-06 01:00:00,2025-06-06 02:00:00,NaN,NaN,2073737,99923,PC,BOTTLE,"0,330 L",120.0,80.0,149.40,1782.0,141.1344,0.07920,0.018519,781.00,1.0,33,25773.00,316800.0,120.0x80.0,641.757600,641.75760,0.000000,0.0,0.0000,0.0,0.0,False,0.000,141.1344,1782.0,33,25773.00,316800.0
6,4.508683e+09,34800698,ZGBTR26,STO,11,GB01,GB67,526605,1565895,2025-06-06 08:00:00,2025-06-06 08:30:00,NaT,2025-06-06,10,2025-06-06 08:00:00,2025-06-06 08:30:00,2025-06-06 14:00:00,2025-06-06 15:00:00,25894,70909,PC,BOTTLE,"0,330 L",120.0,100.0,141.12,3900.0,154.4400,0.03960,0.006667,974.00,1.0,26,25324.00,312000.0,120.0x100.0,6190.628400,6339.12840,0.000000,0.0,148.5000,0.0,0.0,False,0.000,154.4400,3900.0,26,25324.00,312000.0
5,4.508683e+09,34800720,ZGBTR26,STO,11,GB01,GB67,526605,1565895,2025-06-06 03:00:00,2025-06-06 03:30:00,NaT,2025-06-06,10,2025-06-06 03:00:00,2025-06-06 03:30:00,2025-06-06 14:00:00,2025-06-06 15:00:00,25894,70909,PC,BOTTLE,"0,330 L",120.0,100.0,141.12,3750.0,148.5000,0.03960,0.006667,974.00,1.0,25,24350.00,300000.0,120.0x100.0,6339.128400,6339.12840,0.000000,0.0,0.0000,0.0,0.0,False,0.000,148.5000,3750.0,25,24350.00,300000.0
11,4.508683e+09,34800871,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,NaT,2025-06-06,10,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,1870515,105537,PC,BOTTLE,"0,330 L",120.0,100.0,147.60,2340.0,138.9960,0.05940,0.011111,950.50,1.0,26,24713.00,312000.0,120.0x100.0,0.000000,0.00000,0.000000,0.0,0.0000,0.0,0.0,True,138.996,0.0000,0.0,0,0.00,0.0
12,4.508683e+09,34800872,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,NaT,2025-06-06,10,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,1870515,105537,PC,BOTTLE,"0,330 L",120.0,100.0,147.60,2340.0,138.9960,0.05940,0.011111,950.50,1.0,26,24713.00,312000.0,120.0x100.0,0.000000,0.00000,0.000000,0.0,0.0000,0.0,0.0,True,138.996,0.0000,0.0,0,0.00,0.0
9,4.508683e+09,34800874,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 00:00:00,2025-06-06 00:30:00,NaT,2025-06-05,30,2025-06-06 00:00:00,2025-06-06 00:30:00,2025-06-06 01:00:00,2025

In [75]:
##testing start for extract_free_area

In [76]:
#changes 5/7 : rectpack
from rectpack import newPacker
import matplotlib.pyplot as plt
df = open_sto_out
df = open_sto_out.loc[open_sto_out['load_id']==34776448,['load_id','material_sk','total_feasible_order_qty_pal','pal_length','pal_width']]

In [77]:
df

,load_id,material_sk,total_feasible_order_qty_pal,pal_length,pal_width


In [78]:
# df = pd.DataFrame({'load_id': [34753530,34753530,34753530],'material_sk': ['A1','A2','A3'], 'total_feasible_order_qty_pal': [11,5,3], 'pal_length': [120,120,120], 'pal_width': [100,100,80]})
# df = pd.DataFrame({'load_id': [34753530,34753530],'material_sk': ['A1','A2'], 'total_feasible_order_qty_pal': [32,1], 'pal_length': [120,120], 'pal_width': [80,80]})

In [79]:
# rectangles
# total_pallets
# packed_pallets
# bottom_row_rectangles
# top_row_rectangles
# remaining_length_bottom_row
# max_x_end_top_row
# try:
#     max_x_end_top_row = max(x + w for b, x, y, w, h, rid in top_row_rectangles)
# except Exception:
#     max_x_end_top_row = 0
# len(pallets_placed_at_end)
# rectangles
# bottom_row_rectangles
# pallet_placed_at_end_bottom_row
# pallet_placed_at_end_top_row
# total_pallets

In [80]:
result = {}

for load_id, group in df.groupby("load_id"):
    print(load_id)
    packer = newPacker(rotation=False)
    packer.add_bin(truck_length, truck_width)

    # Add rectangles with unique IDs
    rect_id = 0
    rect_map = {}

    # Add each pallet to the bin
    for _, row in group.iterrows():

        # changes 5/21 : non-orthogonal loading for 132x120 pallets
        rotate_it = (
        (row["pal_length"], row["pal_width"]) in ROTATABLE_DIMS
        )

        pal_len  = row["pal_length"]
        pal_wid  = row["pal_width"]

        if rotate_it:
            pal_len, pal_wid = pal_wid, pal_len
        for _ in range(int(row['total_feasible_order_qty_pal'])):
            packer.add_rect( pal_wid,pal_len, rect_id)
            rect_map[rect_id] = (row['material_sk'], pal_wid,pal_len)
            rect_id += 1
    packer.pack()
    rectangles = packer.rect_list()
    total_pallets = group['total_feasible_order_qty_pal'].sum()

    if total_pallets == 0:
        result[load_id] = {
            'pallets_requested': 0,
            'pallets_packed': 0,
            'feasible': True,
            'remaining_length_bottom_row': truck_length,
            'remaining_length_top_row': truck_length,
            'remaining_width_bottom_row': truck_width/2,
            'remaining_width_top_row': truck_width/2,
            'available_area_top_row': truck_length * (truck_width/2),
            'available_area_bottom_row': truck_length * (truck_width/2)
        }
        continue

    #get the packed pallets
    packed_pallets = len(packer[0])

    #get the remaining length in top & bottom row separately
    bottom_row_rectangles = [rect for rect in rectangles if rect[2]==0]
    top_row_rectangles = [rect for rect in rectangles if rect[2]!=0]

    # Calculate max X + width to determine used truck length (bottom row)
    #changes 5/20 : list end error
    try:
        max_x_end_bottom_row = max(x + w for b, x, y, w, h, rid in bottom_row_rectangles)
    except Exception:
        max_x_end_bottom_row = 0
    remaining_length_bottom_row = truck_length - max_x_end_bottom_row

    # Calculate max X + width to determine used truck length (bottom row)
    try:
        max_x_end_top_row = max(x + w for b, x, y, w, h, rid in top_row_rectangles)
    except Exception:
        max_x_end_top_row = 0
    remaining_length_top_row = truck_length - max_x_end_top_row

    try:
        max_x_end = max(x + w for b, x, y, w, h, rid in rectangles)
    except Exception:
        max_x_end = 0


    pallet_placed_at_end_bottom_row = [placed_rect for placed_rect in bottom_row_rectangles if placed_rect[1] + placed_rect[3] == max_x_end_bottom_row]
    pallet_placed_at_end_top_row = [placed_rect for placed_rect in top_row_rectangles if placed_rect[1] + placed_rect[3] == max_x_end_top_row]
    pallets_placed_at_end = [placed_rect for placed_rect in rectangles if placed_rect[1] + placed_rect[3] == max_x_end]
    
    if sum(pal[4] for pal in pallets_placed_at_end) == truck_width:
        remaining_width_top_row = truck_width / 2
        remaining_width_bottom_row = truck_width / 2
    elif (len(pallets_placed_at_end) == 1) & (((0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][1]) + (0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][3])) > ((0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][1]) + (0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][3]))):
        remaining_width_bottom_row = pallet_placed_at_end_bottom_row[0][4]
        remaining_width_top_row = truck_width - remaining_width_bottom_row
    elif (len(pallets_placed_at_end) == 1) & (((0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][1]) + (0 if len(pallet_placed_at_end_bottom_row)==0 else pallet_placed_at_end_bottom_row[0][3])) < ((0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][1]) + (0 if len(pallet_placed_at_end_top_row)==0 else pallet_placed_at_end_top_row[0][3]))):
        remaining_width_top_row = pallet_placed_at_end_top_row[0][4]
        remaining_width_bottom_row = truck_width - remaining_width_top_row
    else:
        remaining_width_top_row = truck_width / 2
        remaining_width_bottom_row = truck_width / 2

    
    if remaining_length_bottom_row < min_pallet_width or remaining_width_bottom_row < min_pallet_length:
        available_area_bottom_row = 0
    else:
        available_area_bottom_row = remaining_length_bottom_row * remaining_width_bottom_row

    if remaining_length_top_row < min_pallet_width or remaining_width_top_row < min_pallet_length:
        available_area_top_row = 0
    else:
        available_area_top_row = remaining_length_top_row * remaining_width_top_row


    result[load_id] = {
            'pallets_requested': int(total_pallets),
            'pallets_packed': int(packed_pallets),
            'feasible': packed_pallets == total_pallets,
            'remaining_length_bottom_row': remaining_length_bottom_row,
            'remaining_length_top_row': remaining_length_top_row,
            'remaining_width_bottom_row': remaining_width_bottom_row,
            'remaining_width_top_row': remaining_width_top_row,
            'available_area_top_row': available_area_top_row,
            'available_area_bottom_row': available_area_bottom_row
        }
    # Plot setup
    fig, ax = plt.subplots(figsize=(20, 3))
    ax.set_xlim(0, truck_length)
    ax.set_ylim(0, truck_width)
    ax.set_aspect('equal')
    ax.set_title(f"Packed Layout for Shipment {load_id}")

    # Draw each packed pallet
    for b, x, y, w, h, rid in rectangles:
        # print(x, y, w, h, rid )
        material, plen, pwid = rect_map[rid]
        ax.add_patch(plt.Rectangle((x, y), w, h, edgecolor='black', facecolor='skyblue', alpha=0.6))
        ax.text(x + w/2, y + h/2, f"{material}\n{plen}×{pwid}", ha='center', va='center', fontsize=8)

    plt.xlabel("Truck Length (cm)")
    plt.ylabel("Truck Width (cm)")
    plt.grid(True)
    plt.savefig(f"{result_path}{tag}_{load_id}_pre_opt.png")
    plt.show()

result = pd.DataFrame.from_dict(result, orient='index')

In [81]:
result

""


In [82]:
##testing end for extract_free_area

In [83]:
print(min_pallet_length)
print(min_pallet_width)

120
80


In [84]:
#changes 5/12 rectpack
load_level_rectpack_responses = extract_free_area(open_sto_out,ROTATABLE_DIMS, truck_length, truck_width, min_pallet_length, min_pallet_width, result_path, tag)

In [85]:
print('% of loads with original load plan as rectpack feasible', load_level_rectpack_responses.loc[load_level_rectpack_responses['feasible'] == True, 'load_id'].nunique() / open_sto_out['load_id'].nunique() * 100.00)

% of loads with original load plan as rectpack feasible 100.0


In [86]:
#changes 5/12 rectpack
load_level_rectpack_responses['available_area'] = load_level_rectpack_responses['available_area_top_row'] + load_level_rectpack_responses['available_area_bottom_row']
load_level_rectpack_responses = load_level_rectpack_responses[['load_id','feasible','available_area_top_row','available_area_bottom_row','available_area']]

In [87]:
### Getting load level details of the original SKUs 
#changes 4/20 : Area based optimization
#changes 5/12 rectpack
load_level_feasible_order_details = open_sto_out.groupby(['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination', 'origin_shipping_location_sk',
    'destination_shipping_location_sk', 'Slot Booked From', 'Slot Booked To'], as_index=False).agg({'total_feasible_order_qty_hl': 'sum','total_feasible_order_qty_pc': 'sum','total_feasible_order_qty_pal': 'sum', 'total_feasible_order_qty_weight': 'sum','total_feasible_order_area': 'sum'})

# changes 5/27 : Pre-load changes
main_load_details = main_load_details[(main_load_details['actual_loading_start_ts']=='1900-01-01 00:00:00')|(main_load_details['actual_loading_start_ts'].isna())]
main_load_details = pd.merge(main_load_details, load_level_feasible_order_details[['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination', 'total_feasible_order_qty_hl', 'total_feasible_order_qty_pc','total_feasible_order_qty_pal', 'total_feasible_order_qty_weight','total_feasible_order_area']],\
            on = ['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source', 'Destination'], how = 'left')
main_load_details = pd.merge(main_load_details, load_level_rectpack_responses,on = ['load_id'], how = 'left')

In [88]:
main_load_details

,RFRC_NUM12,load_id,trailer_equipment_type_code,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,actual_loading_start_ts,planned_movement_ts,original_quantity_ordered_hl,original_quantity_ordered_pal,original_quantity_ordered_kg,total_area_in_cm2,total_feasible_order_qty_hl,total_feasible_order_qty_pc,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area,feasible,available_area_top_row,available_area_bottom_row,available_area
0,4.508681e+09,34798240,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0
1,4.508681e+09,34798241,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0
2,4.508681e+09,34798242,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0
3,4.508683e+09,34800871,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,NaT,2025-06-06,138.9960,26,24713.00,312000.0,0.0000,0.0,0,0.00,0.0,True,163200.0,163200.0,326400.0
4,4.508683e+09,34800872,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,NaT,2025-06-06,138.9960,26,24713.00,312000.0,0.0000,0.0,0,0.00,0.0,True,163200.0,163200.0,326400.0
5,4.508683e+09,34801610,ZGBTR26,STO,12,GB02,GB01,1365010,526605,2025-06-06 20:00:00,2025-06-06 20:30:00,2025-06-07 03:00:00,2025-06-07 03:30:00,NaT,2025-06-06,204.7536,26,27651.92,312000.0,204.7536,2405.0,26,27651.92,312000.0,True,0.0,0.0,0.0
6,4.508683e+09,34801622,ZGBTR26,STO,12,GB02,GB01,1365010,526605,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,NaT,2025-06-06,237.6768,23,27554.40,276000.0,237.6768,1894.0,23,27554.40,276000.0,True,31200.0,19200.0,50400.0
7,4.508683e+09,34800876,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 07:00:00,2025-06-06 07:30:00,NaT,NaT,NaT,2025-06-06,140.0652,26,24889.58,312000.0,17.1072,216.0,3,3028.08,36000.0,True,151200.0,139200.0,290400.0
8,4.508683e+09,34800877,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 04:00:00,2025-06-06 04:30:00,2025-06-06 05:00:00,2025-06-06 06:00:00,NaT,2025-06-06,244.8000,26,27297.10,312000.0,244.8000,1854.0,26,27297.10,312000.0,True,0.0,0.0,0.0
9,4.508683e+09,34801568,ZGBTR26,STO,12,GB01,GB02,526605,1365010,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,NaT,2025-06-06,212.0000,15,23839.20,237600.0,212.0000,440.0,15,23839.20,237600.0,True,52320.0,36480.0,88800.0


In [89]:
any(main_load_details['original_quantity_ordered_pal']>26)==True

True

In [90]:
#modifications needed for pallet differentiation in GB
#Please note that ordered PAL exceeds 32 for GB, is some cases as high as 53. AI to talk to Yuliia for clarification
# main_load_details['available_pal'] = np.where(main_load_details['sorted_match']==True, (32 - main_load_details['total_feasible_order_qty_pal']).astype(int), (26 - main_load_details['total_feasible_order_qty_pal']).astype(int))

#changes 4/20 : Area based optimization
#changes 5/12 rectpack
#commenting below line below top vs bottom split cannot be calculated for infeasible loads
# main_load_details['available_area'] = np.where(main_load_details['feasible']!=True,MAX_AVAILABLE_AREA - main_load_details['total_feasible_order_area'],main_load_details['available_area'])

#changes 4/14 : keg conversion and rounding

#changes 4/20 : Area based optimization
#below statement needs to be changed as available_pal takes into consideration standard pallet area
main_load_details['available_pal'] = main_load_details['available_area'] / STANDARD_PALLET_AREA
main_load_details['available_pal'] = np.where(main_load_details['available_pal'] < 0, 0, main_load_details['available_pal'])
main_load_details['available_pal'] = main_load_details['available_pal'].apply(custom_round)
main_load_details['available_weight'] = standard_weights.loc[standard_weights['Country'] == 'GB', 'weight_limit'].values[0] - (main_load_details['total_feasible_order_qty_weight']/1000)

In [91]:
load_cols_to_show = ['load_id','original_quantity_ordered_hl','original_quantity_ordered_pal','total_feasible_order_qty_hl', 'total_feasible_order_qty_pal','available_area','available_pal','available_weight']

In [92]:
(main_load_details['total_feasible_order_area'] + main_load_details['available_area']).unique()

array([326400., 312000., 319200.])

In [93]:
#AI to check why this can't be zero - must be floating point operations with PALs
tolerance = 1e-5
main_load_details['Action'] = ''
# Apply 'Load not at risk' condition
main_load_details.loc[
    (main_load_details['original_quantity_ordered_pal'] - main_load_details['total_feasible_order_qty_pal']).abs() < tolerance, 
    'Action'] = 'Load not at risk'

# Apply 'At risk' condition
main_load_details.loc[
    (main_load_details['total_feasible_order_qty_pal'] < main_load_details['original_quantity_ordered_pal']) & 
    (main_load_details['Action'] == ''),
    'Action'] = 'At risk'

# Apply 'Light load' condition only if the other two are not satisfied
# check if available_PAL should also be > 0

# this needs to be dynamic ie. the min available weight criteria for the load to be classified as a light load

#changes 4/14 : keg conversion and rounding
#changes 4/20 : Area based optimization
main_load_details.loc[
    (main_load_details['available_pal'] >= MIN_AVAILABLE_STD_PAL_FOR_LIGHT_LOAD_CLASS) & 
    (main_load_details['available_weight'] > MIN_AVAILABLE_WEIGHT_IN_TONNES_FOR_LIGHT_LOAD_CLASS) & 
    (main_load_details['Action'] == 'Load not at risk'), 
    'Action'] = 'Light load'

In [94]:
load_cols_to_show.append('Action')

In [95]:
main_load_details['Action'].value_counts()

Action
Light load          9
Load not at risk    5
At risk             3
Name: count, dtype: int64

In [96]:
(main_load_details['original_quantity_ordered_pal']>26).sum()


3

In [97]:
# open_sto_out.loc[open_sto_out.load_id.isin(list(main_load_details[main_load_details['original_quantity_ordered_pal']>26].load_id.unique())),]

In [98]:
#changes 4/14 : keg conversion and rounding
loads_at_risk_or_light = main_load_details[main_load_details['Action'].isin(['Light load', 'At risk'])]
#changes 4/20 : Area based optimization
loads_at_risk_or_light = loads_at_risk_or_light[(loads_at_risk_or_light['available_pal']>=MIN_AVAILABLE_STD_PAL_FOR_LIGHT_LOAD_CLASS)&(loads_at_risk_or_light['available_weight']>MIN_AVAILABLE_WEIGHT_IN_TONNES_FOR_LIGHT_LOAD_CLASS)]
print('## Loads that are at risk or have a light load: ', loads_at_risk_or_light['load_id'].nunique())
kpi_results['Loads that are at risk or have a light load'] = {
    'Value': loads_at_risk_or_light['load_id'].nunique(),
    'Percentage': loads_at_risk_or_light['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
}

print('## Loads that are at risk: ', loads_at_risk_or_light[loads_at_risk_or_light['Action']=='At risk']['load_id'].nunique())
kpi_results['Loads that are at risk'] = {
    'Value': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='At risk']['load_id'].nunique(),
    'Percentage': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='At risk']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
}

print('## Loads that are a light load: ', loads_at_risk_or_light[loads_at_risk_or_light['Action']=='Light load']['load_id'].nunique())
kpi_results['Loads that are a light load'] = {
    'Value': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='Light load']['load_id'].nunique(),
    'Percentage': loads_at_risk_or_light[loads_at_risk_or_light['Action']=='Light load']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
}

## Loads that are at risk or have a light load:  12
## Loads that are at risk:  3
## Loads that are a light load:  9


In [99]:
kpi_results

{'Total number of loads': {'Value': 18, 'Percentage': 100},
 'Loads that are at risk or have a light load': {'Value': 12,
  'Percentage': 66.66666666666666},
 'Loads that are at risk': {'Value': 3, 'Percentage': 16.666666666666664},
 'Loads that are a light load': {'Value': 9, 'Percentage': 50.0}}

In [100]:
loads_at_risk_or_light[loads_at_risk_or_light['Action']=='Light load']['load_id'].unique()

array([34798240, 34798241, 34798242, 34801622, 34801568, 34800880,
       34801613, 34800720, 34808282], dtype=int64)

In [101]:
print("LCP data combinations before removing zero-forecast SKUs with hist. loads: ", lcp_data.shape[0])
## Dropping those combinations where SKUs have zero forecast at the source but have been
lcp_data = pd.merge(lcp_data, no_forecast_df[['Source', 'material_sk', 'material_code', 'actual_quantity']], on=['Source', 'material_sk', 'material_code'], how = 'left').fillna(0)
lcp_data = lcp_data[lcp_data['actual_quantity']==0].reset_index(drop=True).drop(columns=['actual_quantity'])
print("LCP data combinations after removing zero-forecast SKUs with hist. loads: ", lcp_data.shape[0])

LCP data combinations before removing zero-forecast SKUs with hist. loads:  11224
LCP data combinations after removing zero-forecast SKUs with hist. loads:  11168


In [102]:
loads_at_risk_or_light_lcp_enriched = pd.merge(loads_at_risk_or_light.loc[:,~loads_at_risk_or_light.columns.str.contains('_shipping_location_sk')], lcp_data.loc[:,~lcp_data.columns.str.contains('_shipping_location_sk')], on=['Source', 'Destination'], how='inner')
loads_at_risk_or_light_lcp_enriched.rename(columns={'origin_slot_arrival':'Slot Booked From', 'origin_slot_departure':'Slot Booked To'}, inplace=True)

In [103]:
loads_at_risk_or_light_lcp_enriched_backup = loads_at_risk_or_light_lcp_enriched.copy()
# loads_at_risk_or_light_lcp_enriched = loads_at_risk_or_light_lcp_enriched_backup.copy()

In [104]:
#now we don't need overall load level available/feasible/etc in df data as it is in proposed SKU level
# changes 5/27 : Pre-load changes
loads_at_risk_or_light_lcp_enriched[['stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl']] = \
        loads_at_risk_or_light_lcp_enriched.apply(lambda row: pd.Series(calculate_stock_available_sr(row, stock, open_so, open_sto_in, open_sto_out, pre_load_sto_out_df, production, actual_production, inventory_policy, run_time, 'TOP-UP')), axis=1)
loads_at_risk_or_light_lcp_enriched = loads_at_risk_or_light_lcp_enriched[loads_at_risk_or_light_lcp_enriched['stock_on_hand_sr_hl'] > 0]

In [105]:
#testing start for calculate_stock_available_dest
#def calculate_stock_available_dest(load, stock, open_so, open_sto_in, open_sto_out,  production,actual_production, inventory_policy, run_time):

In [106]:
load = loads_at_risk_or_light_lcp_enriched.iloc[0,:]
# load = loads_at_risk_or_light_lcp_enriched[loads_at_risk_or_light_lcp_enriched['Source']=='GB80'].iloc[2,:]

In [107]:
load

RFRC_NUM12                                4508680509.0
load_id                                       34798240
trailer_equipment_type_code                        NaN
movement_type                                      STO
Priority Flag                                       11
Source                                            GB28
Destination                                       IT12
Slot Booked From                   2025-06-06 01:00:00
Slot Booked To                     2025-06-06 02:00:00
destination_slot_arrival                           NaT
destination_slot_departure                         NaT
actual_loading_start_ts                            NaT
planned_movement_ts                2025-06-06 00:00:00
original_quantity_ordered_hl                  141.1344
original_quantity_ordered_pal                       33
original_quantity_ordered_kg                   25773.0
total_area_in_cm2                             316800.0
total_feasible_order_qty_hl                   141.1344
total_feas

In [108]:
sku = load['material_sk']
destination = load['Destination']
priority_flag = load['Priority Flag']

try:
    # Stock at hand at the destination
    stock_at_hand = stock.loc[
        (stock['material_sk'] == sku) & (stock['Source'] == destination), 'opening_stock_hl'].values[0]
except IndexError:
    stock_at_hand = 0

In [109]:
stock_at_hand

8578.6272

In [110]:
outgoing_so = open_so[
    (open_so['material_sk'] == sku) &
    (open_so['Source'] == destination) &
    (open_so['Delivery Date'] == run_time.normalize())
]['open_so_out_hl'].sum()

In [111]:
outgoing_so

0.0

In [112]:
incoming_sto = open_sto_in[
    (open_sto_in['material_sk'] == sku) &
    (open_sto_in['Destination'] == destination) &
    (open_sto_in['Slot Booked From'].dt.normalize() == run_time.normalize())
]['total_quantity_hl'].sum()

In [113]:
incoming_sto

0.0

In [114]:
outgoing_sto = open_sto_out[
    (open_sto_out['material_sk'] == sku) &
    (open_sto_out['Source'] == destination) &
    (open_sto_out['Slot Booked From'].dt.normalize() == run_time.normalize())
]['total_feasible_order_qty_hl'].sum()

In [115]:
outgoing_sto

0.0

In [116]:

# Planned production for the whole day
#release_ts considered instead of end_outflow_ts
planned_production = production[
    (production['material_sk'] == sku) &
    (production['plant_code'] == destination) &
    (production['release_ts'].dt.normalize() == run_time.normalize())
]['production_hl'].sum()


In [117]:
planned_production

0.0

In [118]:

    # Actual production for the whole day
actual_prod = actual_production[
    (actual_production['material_sk'] == sku) &
    (actual_production['plant_code'] == destination)
]['production_hl'].sum()


In [119]:
actual_prod

0.0

In [120]:
sku

1869530

In [121]:
destination

'IT12'

In [122]:
stock_at_hand

8578.6272

In [123]:
# Calculate the stock available at the destination
stock_available = (stock_at_hand - outgoing_so + incoming_sto + planned_production + actual_prod - outgoing_sto)

try:
    # Maximum stock (inventory policy)
    max_stock = inventory_policy.loc[
        (inventory_policy['material_sk'] == sku) & 
        (inventory_policy['Source'] == destination), 
        'max_stock_hl'
    ].values[0]
except IndexError:
    #why 0
    #In such cases, max_stock should be equal to safety stock handled in pre-processing
    max_stock = 0

# Calculate the demand at the destination
#This can make demand negative based on above inventory policy calculation
#also demand should include forecast
demand = max_stock - stock_available

In [124]:
max_stock

25568.26

In [125]:
stock_available

8578.6272

In [126]:
demand

16989.6328

In [127]:
try:
    # safety stock (inventory policy)
    safety_stock = inventory_policy.loc[
        (inventory_policy['material_sk'] == sku) & 
        (inventory_policy['Source'] == destination), 
        'safety_stock_hl'
    ].values[0]
except IndexError:
    safety_stock = 0

In [128]:
oos_qty = (safety_stock - stock_available) if (safety_stock - stock_available>=0) else 0
oos_per = oos_qty / demand
oos_per = oos_per if oos_per>0 else 0

In [129]:
oos_per

0.6790966547552457

In [130]:
#testing end for calculate_stock_available_dest
# return stock_available, demand, oos_per

In [131]:
kpi_results

{'Total number of loads': {'Value': 18, 'Percentage': 100},
 'Loads that are at risk or have a light load': {'Value': 12,
  'Percentage': 66.66666666666666},
 'Loads that are at risk': {'Value': 3, 'Percentage': 16.666666666666664},
 'Loads that are a light load': {'Value': 9, 'Percentage': 50.0}}

In [132]:
# changes 5/27 : Pre-load changes
loads_at_risk_or_light_lcp_enriched[['stock_on_hand_dt_hl', 'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl']] = loads_at_risk_or_light_lcp_enriched.apply(lambda row: pd.Series(calculate_stock_available_dest(row, stock, open_so, open_sto_in, open_sto_out, pre_load_sto_out_df, production, actual_production, inventory_policy, run_time)), axis=1)
loads_at_risk_or_light_lcp_enriched = loads_at_risk_or_light_lcp_enriched[(loads_at_risk_or_light_lcp_enriched['demand_at_dt_hl'] > 0.0)&(loads_at_risk_or_light_lcp_enriched['stock_on_hand_sr_hl'] > 0.0)]

In [133]:
print('## Loads with LCP swaps available: ', loads_at_risk_or_light_lcp_enriched['load_id'].nunique())
kpi_results['Loads with LCP swaps available'] = {
    'Value':  loads_at_risk_or_light_lcp_enriched['load_id'].nunique(),
    'Percentage':  loads_at_risk_or_light_lcp_enriched['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
}

## Loads with LCP swaps available:  12


In [134]:
kpi_results

{'Total number of loads': {'Value': 18, 'Percentage': 100},
 'Loads that are at risk or have a light load': {'Value': 12,
  'Percentage': 66.66666666666666},
 'Loads that are at risk': {'Value': 3, 'Percentage': 16.666666666666664},
 'Loads that are a light load': {'Value': 9, 'Percentage': 50.0},
 'Loads with LCP swaps available': {'Value': 12,
  'Percentage': 66.66666666666666}}

In [135]:
loads_at_risk_or_light_lcp_enriched['container_type_description'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(container_type_dict)
loads_at_risk_or_light_lcp_enriched['PC_HL'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_hl_dict)
mapped_values = loads_at_risk_or_light_lcp_enriched.apply(
    lambda row: pal_stacking_factor_map.get(row['Destination'][:2], {}).get(row['PC_HL'], np.nan),
    axis=1
)
loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',mapped_values, 1)
#changes 4/20 : Area based optimization
loads_at_risk_or_light_lcp_enriched['pal_length'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_length_dict)
loads_at_risk_or_light_lcp_enriched['pal_width'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_width_dict)
loads_at_risk_or_light_lcp_enriched['pal_height'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_height_dict)
loads_at_risk_or_light_lcp_enriched['area_per_pal'] = loads_at_risk_or_light_lcp_enriched['pal_length'] * loads_at_risk_or_light_lcp_enriched['pal_width']

In [136]:
#removing aggregated ordered vs feasible columns at load level . Retaining only available PAL/weight
#changes 4/20 : Area based optimization
#changes 5/12 rectpack
loads_at_risk_or_light_lcp_enriched = loads_at_risk_or_light_lcp_enriched[['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Slot Booked From',
                'Slot Booked To', 'Priority Flag', 'available_pal', 'available_weight','available_area_top_row','available_area_bottom_row','available_area','material_sk', 'material_code', 'lcp_rank',
                'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl','actual_production_sr_hl',
                'outgoing_so_sto_sr_hl','incoming_sto_sr_hl', 'safety_stock_sr_hl', 'stock_on_hand_dt_hl',
                'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl', 'Action','pal_length', 'pal_width', 'pal_height', 'area_per_pal']]
#changes 4/14 : keg conversion and rounding
loads_at_risk_or_light_lcp_enriched['container_type_description'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(container_type_dict)
loads_at_risk_or_light_lcp_enriched['PC_PAL'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_pal_dict)
loads_at_risk_or_light_lcp_enriched['PC_HL'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_hl_dict)
#changes 4/15 pal_stacking_factor_map
#changes 4/20 : Area based optimization
mapped_values = loads_at_risk_or_light_lcp_enriched.apply(lambda row: pal_stacking_factor_map.get(row['Destination'][:2], {}).get(row['PC_HL'], np.nan),axis=1)
loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',mapped_values, 1)
loads_at_risk_or_light_lcp_enriched.dropna(subset = ['PAL_STACKING_FACTOR'], inplace=True)
loads_at_risk_or_light_lcp_enriched['HL_PAL'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',loads_at_risk_or_light_lcp_enriched['PC_PAL'] / (loads_at_risk_or_light_lcp_enriched['PC_HL'] * loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR']),loads_at_risk_or_light_lcp_enriched['PC_PAL'] / loads_at_risk_or_light_lcp_enriched['PC_HL'])
loads_at_risk_or_light_lcp_enriched['PAL_HL'] = 1 / loads_at_risk_or_light_lcp_enriched['HL_PAL']
loads_at_risk_or_light_lcp_enriched['HL_weight'] = np.where(loads_at_risk_or_light_lcp_enriched['container_type_description'].str.upper() == 'KEG',loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_weight_dict) * loads_at_risk_or_light_lcp_enriched['PAL_STACKING_FACTOR'] * loads_at_risk_or_light_lcp_enriched['HL_PAL'] / 1000,loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_weight_dict) * loads_at_risk_or_light_lcp_enriched['HL_PAL'] / 1000)


# loads_at_risk_or_light_lcp_enriched['HL_PAL'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_pal_dict) / loads_at_risk_or_light_lcp_enriched['material_sk'].map(pc_to_hl_dict)
# #check the division by 1000
# loads_at_risk_or_light_lcp_enriched['HL_weight'] = loads_at_risk_or_light_lcp_enriched['material_sk'].map(pal_weight_dict) * loads_at_risk_or_light_lcp_enriched['HL_PAL'] / 1000

In [137]:
# loads_at_risk_or_light_lcp_enriched.loc[loads_at_risk_or_light_lcp_enriched['container_type_description']=='KEG',:]

In [138]:
# print('% LCP SKUs with missing HL_PAL',(loads_at_risk_or_light_lcp_enriched['HL_PAL'].isnull().sum() / len(loads_at_risk_or_light_lcp_enriched['HL_PAL']))*100.00)
# print('% LCP SKUs with missing HL_weight',(loads_at_risk_or_light_lcp_enriched['HL_weight'].isnull().sum() / len(loads_at_risk_or_light_lcp_enriched['HL_weight']))*100.00)

In [139]:
loads_at_risk_or_light_lcp_obs_enriched = pd.merge(loads_at_risk_or_light_lcp_enriched, obs_df[['material_code', 'Source', 'material_sk', 'perc_obsolescence_risk_at_sr_hl']], on=['material_sk', 'material_code', 'Source'], how='left').fillna(0)

In [140]:
loads_at_risk_or_light_lcp_obs_enriched_backup = loads_at_risk_or_light_lcp_obs_enriched.copy()
# loads_at_risk_or_light_lcp_obs_enriched = loads_at_risk_or_light_lcp_obs_enriched_backup.copy()

In [141]:
loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'] = loads_at_risk_or_light_lcp_obs_enriched['Slot Booked From'] - run_time
#code for normalization
loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'] = loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'] / loads_at_risk_or_light_lcp_obs_enriched['Waiting_time'].max()

In [142]:
loads_at_risk_or_light_lcp_obs_enriched['Priority Flag'].unique()

array([11, 12,  0], dtype=int64)

In [143]:
priority_mapping = {x: (15 - x) * 3 + 1 for x in range(17)}

In [144]:
loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled'] = loads_at_risk_or_light_lcp_obs_enriched['Priority Flag'].map(priority_mapping)

In [145]:
#This is % OOS at destination. Check if scale is <1 or <100
loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] = loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl'] * 100.00
# Check why this is 200
loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] = np.where(loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] >= 100, 100, loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'])
# % At Risk refers to obsolescence risk - already scaled to 100
loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'] = np.where(loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'] >= 100, 100, loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'])

In [146]:
lcp_rank_mapping = {
    1: 5,
    2: 4,  # You can assign 2 to other values if needed
    3: 3,
    4: 2,  # Example mapping for the remaining values
    5: 1
}
loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] = loads_at_risk_or_light_lcp_obs_enriched['lcp_rank'].map(lcp_rank_mapping)

In [147]:
print("max %OOS",loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'].max())
print("max % At Risk",loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'].max())

max %OOS 67.90966547552458
max % At Risk 0.0


In [148]:
print("LCP:",loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'].unique())
print("Priority:",loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled'].unique())

LCP: [5]
Priority: [13 10 46]


In [149]:
loads_at_risk_or_light_lcp_obs_enriched['Weights'] = ((loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled']) + (1 * loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] + 1 * loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) + (loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled']))

In [150]:
print("Weights:",loads_at_risk_or_light_lcp_obs_enriched['Weights'].unique())

Weights: [85.90966548 19.43475637 15.         17.09994737 24.43366076 26.50014743
 25.45038594 21.37708382 36.65198772 31.22748032 25.625208   31.62826764
 36.60863554 21.70240467 24.72037649 73.93507486 15.13952701 69.1941024
 26.37234219 32.74785829 30.80686334 37.66883318 41.65468578 43.9428071
 18.0191884  28.89435465 29.95259862 26.08684646 26.51854194 22.8773605
 29.01096787 15.24094278 31.22146575 17.04552952 46.07030005 36.0735386
 27.52951634 17.12196281 15.34677731 26.83152233 19.9642936  26.54763781
 29.83136012 23.72835913 52.79092891 51.16286714 26.35277979 17.36923694
 25.42719458 31.95772059 19.84822425 34.17457383 17.85475655 26.62729264
 31.55899251 29.86741907 21.82760523 26.50487191 19.24465723 31.37954155
 39.11799877 54.50060109 51.53542772 26.43596517 16.09844069 19.8221292
 21.66422402 31.5522541  28.68226402 29.51867712 18.08399915 24.47571772
 18.20138234 26.39806798 46.0680977  53.99929971 16.50559182 25.10086528
 30.19345087 26.77020489 43.71930425 18.       

In [151]:
#loads_at_risk_or_light_lcp_obs_enriched.to_excel(f"{result_path}{tag}_pre_opti_model.xlsx", index=False)
# loads_at_risk_or_light_lcp_obs_enriched = pd.read_excel(f"{result_path}{tag}_pre_opti_model.xlsx")

In [152]:
temp = loads_at_risk_or_light[loads_at_risk_or_light['load_id'].isin(loads_at_risk_or_light_lcp_obs_enriched['load_id'].unique())]

In [153]:
data_pre_opti = loads_at_risk_or_light_lcp_obs_enriched.copy()

In [154]:
# loads_at_risk_or_light_lcp_obs_enriched = data_pre_opti.copy()

In [155]:
#testing start for optimise_loads
# problem = optimise_loads(data) 

In [156]:
problem = LpProblem('Load Exchanging', LpMaximize)

In [157]:
loads_at_risk_or_light_lcp_obs_enriched.groupby('load_id')[['available_area','available_area_top_row','available_area_bottom_row']].max()

,available_area,available_area_top_row,available_area_bottom_row
load_id,,,
34798240,9600.0,9600.0,0.0
34798241,9600.0,9600.0,0.0
34798242,9600.0,9600.0,0.0
34800720,19200.0,19200.0,0.0
34800871,326400.0,163200.0,163200.0
34800872,326400.0,163200.0,163200.0
34800876,290400.0,151200.0,139200.0
34800880,60000.0,31200.0,28800.0
34801568,88800.0,52320.0,36480.0


In [158]:
loads_at_risk_or_light_lcp_obs_enriched.loc[loads_at_risk_or_light_lcp_obs_enriched['load_id']==34762906,:]

,load_id,RFRC_NUM12,movement_type,Source,Destination,Slot Booked From,Slot Booked To,Priority Flag,available_pal,available_weight,available_area_top_row,available_area_bottom_row,available_area,material_sk,material_code,lcp_rank,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,stock_on_hand_dt_hl,demand_at_dt_hl,perc_oos_risk_at_dt_hl,Action,pal_length,pal_width,pal_height,area_per_pal,container_type_description,PC_PAL,PC_HL,PAL_STACKING_FACTOR,HL_PAL,PAL_HL,HL_weight,perc_obsolescence_risk_at_sr_hl,Waiting_time,priority_flag_rescaled,perc_oos_risk_at_dt_hl_rescaled,lcp_rank_rescaled,Weights


In [159]:
# Decision Variable for Quantity
#changes 4/20 : Area based optimization
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_Name'] = 'qty_' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Name'] = 'qty_pal_' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Continuous"))
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))
#changes 5/12 rectpack
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Top_Name'] = 'qty_pal_top' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Bottom_Name'] = 'qty_pal_bottom' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_TOP'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Top_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))
loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_BOTTOM'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Bottom_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))



In [160]:
print('lcp rank unique : ',loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'].unique())
print('OOS % unique : ',loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'].unique())
print('Obsolescence % unique : ',loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl'].unique())
print('Priority Flag unique : ',loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled'].unique())

lcp rank unique :  [5]
OOS % unique :  [67.90966548  1.43475637  0.          2.09994737  9.43366076 11.50014743
 10.45038594  6.37708382 21.65198772 16.22748032 10.625208   16.62826764
 21.60863554  6.70240467  9.72037649 58.93507486  0.13952701 54.1941024
 11.37234219 17.74785829 15.80686334 22.66883318 26.65468578 28.9428071
  3.0191884  13.89435465 14.95259862 11.08684646 11.51854194  7.8773605
 14.01096787  0.24094278 16.22146575  2.04552952 31.07030005 21.0735386
 12.52951634  2.12196281  0.34677731 11.83152233  4.9642936  11.54763781
 14.83136012  8.72835913 37.79092891 36.16286714 11.35277979  2.36923694
 10.42719458 16.95772059  4.84822425 19.17457383  2.85475655 11.62729264
 16.55899251 14.86741907  6.82760523 11.50487191  4.24465723 16.37954155
 24.11799877 39.50060109 36.53542772 11.43596517  1.09844069  4.8221292
  6.66422402 16.5522541  13.68226402 14.51867712  3.08399915  9.47571772
  3.20138234 11.39806798 31.0680977  38.99929971  1.50559182 10.10086528
 15.19345087 11.7

In [161]:
# OBJECTIVE FUNCTION
# shipment_value = lpSum((data['lcp_rank_1'] + (0.5 * data['%_OOS_1'] + 0.5 * data['%_At_Risk_1'])/100 + data['Priority Flag_1'] + (1 - data['Waiting_time'])) * data['Qty_LPVar'] * data['HL_weight'])
#shipment_value = lpSum((3* loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] + 1 + (2 * loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] + 2 * loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) + 2*loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled'] + weight_6 * loads_at_risk_or_light_lcp_obs_enriched['truck_utilization_score_rescaled']) * loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'])

shipment_value = lpSum((loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] + 1 + (1 * loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] + 1 * loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) + loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled']) * loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'])


In [162]:
#problem +=shipment_value
problem += lpSum(loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'])


In [163]:
# CONSTRAINT: Sum of all HL quantity recommendations for a material_sk should be less than total HL stock on hand at the source
# lpSum on Qty_LPVar as multiple loads can have same material_sk from same source where iloc[0] on Stock_on_hand_sr(HL) to avoid double counting stock_on_hand
for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['material_sk', 'Source']):
    problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['stock_on_hand_sr_hl'].iloc[0]

In [164]:
# CONSTRAINT: Sum of all HL quantity recommendations for a material_sk should be less than Demand at the Destination
for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['material_sk', 'Destination']):
    problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['demand_at_dt_hl'].iloc[0]

In [165]:
# CONSTRAINT: Sum of all recommended weights for a load should be less than the weight left on the truck(load)
# Check what happens when HL_weight is NA for any material_sk
for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id']):
    problem += lpSum(qty * conv for qty, conv in zip(grp_df['Qty_LPVar'], grp_df['HL_weight'])) <= grp_df['available_weight'].iloc[0]

In [166]:
# CONSTRAINT: Should be less than the area left on the truck
# Check what happens when HL_PAL is NA for any material_sk
#changes 4/20 : Area based optimization
#changes 5/12 rectpack
for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id']):
    problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL_TOP'], grp_df['area_per_pal'])) <= grp_df['available_area_top_row'].iloc[0]
    problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL_BOTTOM'], grp_df['area_per_pal'])) <= grp_df['available_area_bottom_row'].iloc[0]
    problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL'], grp_df['area_per_pal'])) <= grp_df['available_area'].iloc[0]

In [167]:
#changes 4/20 : Area based optimization
# individual constraint for each load_id and shipment
for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id','material_sk']):
    problem += lpSum(grp_df['Qty_LPVar']) == lpSum(pal_qty * hl_per_pal for pal_qty, hl_per_pal in zip(grp_df['Qty_LPVar_PAL'], grp_df['PAL_HL']))
    #constraint: sum of suggested pallets in top and bottom row should be less than the total suggested pallets
    problem += lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL_TOP']) + lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL_BOTTOM']) == lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL'])

In [168]:
problem.solve(PULP_CBC_CMD(timeLimit = 300, threads = None, msg = 0))

1

In [169]:
print(LpStatus[problem.status])

Optimal


In [170]:
#testing end for optimise loads
#problem = optimise_loads(data)

In [171]:
#changes 4/20 : Area based optimization
loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_hl'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'].apply(lambda x: x.value())
loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_pal'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL'].apply(lambda x: x.value())
loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_pal_top'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_TOP'].apply(lambda x: x.value())
loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_pal_bottom'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_BOTTOM'].apply(lambda x: x.value())

loads_at_risk_or_light_lcp_obs_enriched['LP_Result_Status'] = LpStatus[problem.status]

In [172]:
loads_at_risk_or_light_lcp_obs_enriched

,load_id,RFRC_NUM12,movement_type,Source,Destination,Slot Booked From,Slot Booked To,Priority Flag,available_pal,available_weight,available_area_top_row,available_area_bottom_row,available_area,material_sk,material_code,lcp_rank,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,stock_on_hand_dt_hl,demand_at_dt_hl,perc_oos_risk_at_dt_hl,Action,pal_length,pal_width,pal_height,area_per_pal,container_type_description,PC_PAL,PC_HL,PAL_STACKING_FACTOR,HL_PAL,PAL_HL,HL_weight,perc_obsolescence_risk_at_sr_hl,Waiting_time,priority_flag_rescaled,perc_oos_risk_at_dt_hl_rescaled,lcp_rank_rescaled,Weights,Qty_LPVar_Name,Qty_LPVar_PAL_Name,Qty_LPVar,Qty_LPVar_PAL,Qty_LPVar_PAL_Top_Name,Qty_LPVar_PAL_Bottom_Name,Qty_LPVar_PAL_TOP,Qty_LPVar_PAL_BOTTOM,suggested_deployment_sr_hl,suggested_deployment_sr_pal,suggested_deployment_sr_pal_top,suggested_deployment_sr_pal_bottom,LP_Result_Status
0,34798240,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.727,9600.0,0.0,9600.0,1869530,105531,1,830.253600,830.2536,0.0,0.0,0.0,0.0,0.000000,8578.6272,16989.632800,0.679097,Light load,120.0,80.0,167.5,9600.0,BOTTLE,0.018519,0.0792,1.0,0.233820,4.2768,0.180593,0.0,0.377933,13,67.909665,5,85.909665,qty_1869530_34798240,qty_pal_1869530_34798240,qty_1869530_34798240,qty_pal_1869530_34798240,qty_pal_top1869530_34798240,qty_pal_bottom1869530_34798240,qty_pal_top1869530_34798240,qty_pal_bottom1869530_34798240,0.0000,0.0,0.0,0.0,Optimal
1,34798240,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.727,9600.0,0.0,9600.0,2073737,99923,1,218.354400,218.3544,0.0,0.0,0.0,0.0,0.000000,1202.7312,4568.148388,0.014348,Light load,120.0,80.0,164.5,9600.0,BOTTLE,0.018519,0.0792,1.0,0.233820,4.2768,0.182613,0.0,0.377933,13,1.434756,5,19.434756,qty_2073737_34798240,qty_pal_2073737_34798240,qty_2073737_34798240,qty_pal_2073737_34798240,qty_pal_top2073737_34798240,qty_pal_bottom2073737_34798240,qty_pal_top2073737_34798240,qty_pal_bottom2073737_34798240,4.2768,1.0,1.0,0.0,Optimal
2,34798241,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.727,9600.0,0.0,9600.0,1869530,105531,1,830.253600,830.2536,0.0,0.0,0.0,0.0,0.000000,8578.6272,16989.632800,0.679097,Light load,120.0,80.0,167.5,9600.0,BOTTLE,0.018519,0.0792,1.0,0.233820,4.2768,0.180593,0.0,0.377933,13,67.909665,5,85.909665,qty_1869530_34798241,qty_pal_1869530_34798241,qty_1869530_34798241,qty_pal_1869530_34798241,qty_pal_top1869530_34798241,qty_pal_bottom1869530_34798241,qty_pal_top1869530_34798241,qty_pal_bottom1869530_34798241,0.0000,0.0,0.0,0.0,Optimal
3,34798241,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.727,9600.0,0.0,9600.0,2073737,99923,1,218.354400,218.3544,0.0,0.0,0.0,0.0,0.000000,1202.7312,4568.148388,0.014348,Light load,120.0,80.0,164.5,9600.0,BOTTLE,0.018519,0.0792,1.0,0.233820,4.2768,0.182613,0.0,0.377933,13,1.434756,5,19.434756,qty_2073737_34798241,qty_pal_2073737_34798241,qty_2073737_34798241,qty_pal_2073737_34798241,qty_pal_top2073737_34798241,qty_pal_bottom2073737_34798241,qty_pal_top2073737_34798241,qty_pal_bottom2073737_34798241,4.2768,1.0,1.0,0.0,Optimal
4,34798242,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.727,9600.0,0.0,9600.0,1869530,105531,1,830.253600,830.2536,0.0,0.0,0.0,0.0,0.000000,8578.6272,16989.632800,0.679097,Light load,120.0,80.0,167.5,9600.0,BOTTLE,0.018519,0.0792,1.0,0.233820,4.2768,0.180593,0.0,0.377933,13,67.909665,5,85.909665,qty_1869530_34798242,qty_pal_1869530_34798242,qty_1869530_34798242,qty_pal_1869530_34798242,qty_pal_top1869530_34798242,qty_pal_bottom1869530_34798242,qty_pal_top1869530_34798242,qty_pal_bottom1869530_34798242,4.2768,1.0,1.0,0.0,Optimal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [173]:
loads_at_risk_or_light_lcp_obs_enriched.loc[loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_pal']>0,]

,load_id,RFRC_NUM12,movement_type,Source,Destination,Slot Booked From,Slot Booked To,Priority Flag,available_pal,available_weight,available_area_top_row,available_area_bottom_row,available_area,material_sk,material_code,lcp_rank,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,stock_on_hand_dt_hl,demand_at_dt_hl,perc_oos_risk_at_dt_hl,Action,pal_length,pal_width,pal_height,area_per_pal,container_type_description,PC_PAL,PC_HL,PAL_STACKING_FACTOR,HL_PAL,PAL_HL,HL_weight,perc_obsolescence_risk_at_sr_hl,Waiting_time,priority_flag_rescaled,perc_oos_risk_at_dt_hl_rescaled,lcp_rank_rescaled,Weights,Qty_LPVar_Name,Qty_LPVar_PAL_Name,Qty_LPVar,Qty_LPVar_PAL,Qty_LPVar_PAL_Top_Name,Qty_LPVar_PAL_Bottom_Name,Qty_LPVar_PAL_TOP,Qty_LPVar_PAL_BOTTOM,suggested_deployment_sr_hl,suggested_deployment_sr_pal,suggested_deployment_sr_pal_top,suggested_deployment_sr_pal_bottom,LP_Result_Status
1,34798240,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.72700,9600.0,0.0,9600.0,2073737,99923,1,218.354400,218.35440,0.00000,0.0000,0.0,0.0,0.000000,1202.73120,4568.148388,0.014348,Light load,120.0,80.0,164.50,9600.0,BOTTLE,0.018519,0.07920,1.0,0.233820,4.2768,0.182613,0.0,0.377933,13,1.434756,5,19.434756,qty_2073737_34798240,qty_pal_2073737_34798240,qty_2073737_34798240,qty_pal_2073737_34798240,qty_pal_top2073737_34798240,qty_pal_bottom2073737_34798240,qty_pal_top2073737_34798240,qty_pal_bottom2073737_34798240,4.2768,1.0,1.0,0.0,Optimal
3,34798241,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.72700,9600.0,0.0,9600.0,2073737,99923,1,218.354400,218.35440,0.00000,0.0000,0.0,0.0,0.000000,1202.73120,4568.148388,0.014348,Light load,120.0,80.0,164.50,9600.0,BOTTLE,0.018519,0.07920,1.0,0.233820,4.2768,0.182613,0.0,0.377933,13,1.434756,5,19.434756,qty_2073737_34798241,qty_pal_2073737_34798241,qty_2073737_34798241,qty_pal_2073737_34798241,qty_pal_top2073737_34798241,qty_pal_bottom2073737_34798241,qty_pal_top2073737_34798241,qty_pal_bottom2073737_34798241,4.2768,1.0,1.0,0.0,Optimal
4,34798242,4.508681e+09,STO,GB28,IT12,2025-06-06 01:00:00,2025-06-06 02:00:00,11,1,2.72700,9600.0,0.0,9600.0,1869530,105531,1,830.253600,830.25360,0.00000,0.0000,0.0,0.0,0.000000,8578.62720,16989.632800,0.679097,Light load,120.0,80.0,167.50,9600.0,BOTTLE,0.018519,0.07920,1.0,0.233820,4.2768,0.180593,0.0,0.377933,13,67.909665,5,85.909665,qty_1869530_34798242,qty_pal_1869530_34798242,qty_1869530_34798242,qty_pal_1869530_34798242,qty_pal_top1869530_34798242,qty_pal_bottom1869530_34798242,qty_pal_top1869530_34798242,qty_pal_bottom1869530_34798242,4.2768,1.0,1.0,0.0,Optimal
20,34800871,4.508683e+09,STO,GB01,GB28,2025-06-06 01:00:00,2025-06-06 01:30:00,12,34,28.50000,163200.0,163200.0,326400.0,1903370,103436,1,189.240000,189.24000,0.00000,0.0000,0.0,0.0,0.000000,2519.04000,11207.013344,0.115001,At risk,120.0,100.0,168.75,12000.0,CAN,0.011111,0.12000,1.0,0.092593,10.8000,0.108593,0.0,0.377933,10,11.500147,5,26.500147,qty_1903370_34800871,qty_pal_1903370_34800871,qty_1903370_34800871,qty_pal_1903370_34800871,qty_pal_top1903370_34800871,qty_pal_bottom1903370_34800871,qty_pal_top1903370_34800871,qty_pal_bottom1903370_34800871,75.6000,7.0,4.0,3.0,Optimal
40,34800871,4.508683e+09,STO,GB01,GB28,2025-06-06 01:00:00,2025-06-06 01:30:00,12,34,28.50000,163200.0,163200.0,326400.0,1982919,99200,1,316.436875,600.96000,0.00000,0.0000,0.0,0.0,284.523125,0.00000,100.249300,0.589351,At risk,120.0,100.0,169.10,12000.0,CAN,0.011111,0.12000,1.0,0.092593,10.8000,0.109519,0.0,0.377933,10,58.935075,5,73.935075,qty_1982919_34800871,qty_pal_1982919_34800871,qty_1982919_34800871,qty_pal_1982919_34800871,qty_pal_top1982919_34800871,qty_pal_bottom1982919_34800871,qty_pal_top1982919_34800871,qty_pal_bottom1982919_34800871,32.4000,3.0,3.0,0.0,Optimal
42,34800871,4.508683e+09,STO,GB01,GB28,2025-06-06 01:00:00,2025-06-06 01:30:00,12,34,28.50000,163200.0,163200.0,326400.0,1870722,10100

In [174]:
#changes 4/20 : Area based optimization
optimization_results = loads_at_risk_or_light_lcp_obs_enriched[loads_at_risk_or_light_lcp_obs_enriched['suggested_deployment_sr_hl'] > 0]
optimization_results = optimization_results[['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination', 'Priority Flag', \
    'material_sk', 'material_code', 'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl'
    , 'Action', 'suggested_deployment_sr_hl','suggested_deployment_sr_pal', 'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl','pal_length','pal_width']]

In [175]:
# Swaps file creation
#changes 4/20 : Area based optimization
open_sto_out_swaps = open_sto_out[['load_id', 'RFRC_NUM12', 'movement_type', 'sales_document_item_code', 'Source', 'Destination', 'Priority Flag', 'material_sk', 'material_code',
                        'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl', 'qty_at_risk_hl', 'total_feasible_order_qty_hl', 'at_risk_flag','pal_length','pal_width']]
open_sto_out_swaps = open_sto_out_swaps[open_sto_out_swaps['at_risk_flag'] == True]
open_sto_out_swaps['Action'] = 'Swap-Out'
open_sto_out_swaps = open_sto_out_swaps.rename(columns={'qty_at_risk_hl': 'swap_out_qty_hl', 'total_feasible_order_qty_hl': 'suggested_deployment_sr_hl'})

In [176]:
open_sto_out_swaps[['load_id','material_sk','swap_out_qty_hl','suggested_deployment_sr_hl','at_risk_flag','Action']].sort_values(['load_id'])

,load_id,material_sk,swap_out_qty_hl,suggested_deployment_sr_hl,at_risk_flag,Action
11,34800871,1870515,138.996,0.0,True,Swap-Out
12,34800872,1870515,138.996,0.0,True,Swap-Out
23,34800876,1870515,122.958,0.0,True,Swap-Out


In [177]:
open_sto_out_swaps_combined = pd.concat([open_sto_out_swaps, optimization_results])

In [178]:
#changes 4/14 : keg conversion and rounding
# enriching metadata to enable correct HL forward conversion
open_sto_out_swaps_combined['container_type_description'] = open_sto_out_swaps_combined['material_sk'].map(container_type_dict)
open_sto_out_swaps_combined['PC_PAL'] = open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict)
open_sto_out_swaps_combined['PC_HL'] = open_sto_out_swaps_combined['material_sk'].map(pc_to_hl_dict)

#changes 4/15 pal_stacking_factor_map
mapped_values = open_sto_out_swaps_combined.apply(lambda row: pal_stacking_factor_map.get(row['Destination'][:2], {}).get(row['PC_HL'], np.nan),axis=1)
open_sto_out_swaps_combined['PAL_STACKING_FACTOR'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',mapped_values, 1)
#changes 4/20 : Area based optimization
open_sto_out_swaps_combined.dropna(subset = ['PAL_STACKING_FACTOR'], inplace=True)
open_sto_out_swaps_combined['HL_PAL'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',open_sto_out_swaps_combined['PC_PAL'] / (open_sto_out_swaps_combined['PC_HL'] * open_sto_out_swaps_combined['PAL_STACKING_FACTOR']),open_sto_out_swaps_combined['PC_PAL'] / open_sto_out_swaps_combined['PC_HL'])
open_sto_out_swaps_combined['HL_weight'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',open_sto_out_swaps_combined['material_sk'].map(pal_weight_dict) * open_sto_out_swaps_combined['PAL_STACKING_FACTOR'] * open_sto_out_swaps_combined['HL_PAL'] / 1000,open_sto_out_swaps_combined['material_sk'].map(pal_weight_dict) * open_sto_out_swaps_combined['HL_PAL'] / 1000)


In [179]:
#changes 4/17 : keg conversion and rounding
#changes 4/20 : Area based optimization
open_sto_out_swaps_combined['suggested_deployment_sr_pal'] = np.where(open_sto_out_swaps_combined['Action']=='Swap-Out',open_sto_out_swaps_combined['suggested_deployment_sr_hl'] * open_sto_out_swaps_combined['HL_PAL'],open_sto_out_swaps_combined['suggested_deployment_sr_pal'])
open_sto_out_swaps_combined['suggested_deployment_sr_weight'] = open_sto_out_swaps_combined['suggested_deployment_sr_hl'] * open_sto_out_swaps_combined['HL_weight']
open_sto_out_swaps_combined['suggested_deployment_sr_pc'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',(open_sto_out_swaps_combined['suggested_deployment_sr_pal'] * open_sto_out_swaps_combined['PAL_STACKING_FACTOR']) / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict),open_sto_out_swaps_combined['suggested_deployment_sr_pal'] / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict))
open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] = open_sto_out_swaps_combined['suggested_deployment_sr_pal'].apply(custom_round)
open_sto_out_swaps_combined['rounded_suggested_deployment_sr_hl'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] / open_sto_out_swaps_combined['HL_PAL']
open_sto_out_swaps_combined['rounded_suggested_deployment_sr_weight'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_hl'] * open_sto_out_swaps_combined['HL_weight']
open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pc'] = np.where(open_sto_out_swaps_combined['container_type_description'].str.upper() == 'KEG',(open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] * open_sto_out_swaps_combined['PAL_STACKING_FACTOR']) / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict),open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] / open_sto_out_swaps_combined['material_sk'].map(pc_to_pal_dict))
open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pc'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pc'].apply(custom_round)

#changes 4/20 : Area based optimization
open_sto_out_swaps_combined['rounded_suggested_deployment_sr_area'] = open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal'] * open_sto_out_swaps_combined['pal_length'] * open_sto_out_swaps_combined['pal_width']

In [180]:
#changes 4/20 : Area based optimization
open_sto_out_swaps_combined[['load_id','material_sk','material_code','Action','container_type_description','PC_PAL', 'PC_HL','PAL_STACKING_FACTOR',
       'HL_PAL', 'HL_weight','suggested_deployment_sr_hl','suggested_deployment_sr_pal','suggested_deployment_sr_weight', 'suggested_deployment_sr_pc','rounded_suggested_deployment_sr_hl',
       'rounded_suggested_deployment_sr_pal','rounded_suggested_deployment_sr_weight','rounded_suggested_deployment_sr_pc','rounded_suggested_deployment_sr_area']]

,load_id,material_sk,material_code,Action,container_type_description,PC_PAL,PC_HL,PAL_STACKING_FACTOR,HL_PAL,HL_weight,suggested_deployment_sr_hl,suggested_deployment_sr_pal,suggested_deployment_sr_weight,suggested_deployment_sr_pc,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_pc,rounded_suggested_deployment_sr_area
11,34800871,1870515,105537,Swap-Out,BOTTLE,0.011111,0.05940,1.0,0.187056,0.177796,0.0000,0.0,0.00000,0.0,0.0000,0,0.00000,0,0.0
12,34800872,1870515,105537,Swap-Out,BOTTLE,0.011111,0.05940,1.0,0.187056,0.177796,0.0000,0.0,0.00000,0.0,0.0000,0,0.00000,0,0.0
23,34800876,1870515,105537,Swap-Out,BOTTLE,0.011111,0.05940,1.0,0.187056,0.177796,0.0000,0.0,0.00000,0.0,0.0000,0,0.00000,0,0.0
1,34798240,2073737,99923,Light load,BOTTLE,0.018519,0.07920,1.0,0.233820,0.182613,4.2768,1.0,0.78100,54.0,4.2768,1,0.78100,54,9600.0
3,34798241,2073737,99923,Light load,BOTTLE,0.018519,0.07920,1.0,0.233820,0.182613,4.2768,1.0,0.78100,54.0,4.2768,1,0.78100,54,9600.0
4,34798242,1869530,105531,Light load,BOTTLE,0.018519,0.07920,1.0,0.233820,0.180593,4.2768,1.0,0.77236,54.0,4.2768,1,0.77236,54,9600.0
20,34800871,1903370,103436,At risk,CAN,0.011111,0.12000,1.0,0.092593,0.108593,75.6000,7.0,8.20960,630.0,75.6000,7,8.20960,630,84000.0
40,34800871,1982919,99200,At risk,CAN,0.011111,0.12000,1.0,0.092593,0.109519,32.4000,3.0,3.54840,270.0,32.4000,3,3.54840,270,36000.0
42,34800871,1870722,101002,At risk,CAN,0.012500,0.12000,1.0,0.104167,0.108813,9.6000,1.0,1.04460,80.0,9.6000,1,1.04460,80,12000.0
60,34800871,1886284,101371,At risk,CAN,0.011111,0.12000,1.0,0.092593,0.108593,10.8000,1.0,1.17280,90.0,10.8000,1,1.17280,90,12000.0


In [181]:
#changes 4/20 : Area based optimization
swapped_load_details = open_sto_out_swaps_combined.loc[open_sto_out_swaps_combined['Action']!= 'Swap-Out',:].groupby(['load_id'], as_index = False).agg({'rounded_suggested_deployment_sr_hl':'sum', 'rounded_suggested_deployment_sr_pal':'sum', 'rounded_suggested_deployment_sr_weight':'sum', 'rounded_suggested_deployment_sr_area':'sum'})

In [182]:
swapped_load_details

,load_id,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_area
0,34798240,4.2768,1,0.78100,9600.0
1,34798241,4.2768,1,0.78100,9600.0
2,34798242,4.2768,1,0.77236,9600.0
3,34800720,10.8000,1,1.18280,12000.0
4,34800871,261.9936,26,28.49340,312000.0
5,34800872,262.0512,26,28.48000,312000.0
6,34800876,231.3936,23,25.47170,273600.0
7,34800880,50.8000,5,6.53340,55200.0
8,34801568,42.7152,4,4.65226,48000.0
9,34801613,10.5600,1,1.15200,12000.0


In [183]:
#changes 4/20 : Area based optimization
main_load_details = pd.merge(main_load_details, swapped_load_details[['load_id', 'rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_weight', 'rounded_suggested_deployment_sr_area']], on='load_id', how='left')

In [184]:
main_load_details

,RFRC_NUM12,load_id,trailer_equipment_type_code,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,actual_loading_start_ts,planned_movement_ts,original_quantity_ordered_hl,original_quantity_ordered_pal,original_quantity_ordered_kg,total_area_in_cm2,total_feasible_order_qty_hl,total_feasible_order_qty_pc,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area,feasible,available_area_top_row,available_area_bottom_row,available_area,available_pal,available_weight,Action,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_area
0,4.508681e+09,34798240,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0,1,2.72700,Light load,4.2768,1.0,0.78100,9600.0
1,4.508681e+09,34798241,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0,1,2.72700,Light load,4.2768,1.0,0.78100,9600.0
2,4.508681e+09,34798242,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0,1,2.72700,Light load,4.2768,1.0,0.77236,9600.0
3,4.508683e+09,34800871,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,NaT,2025-06-06,138.9960,26,24713.00,312000.0,0.0000,0.0,0,0.00,0.0,True,163200.0,163200.0,326400.0,34,28.50000,At risk,261.9936,26.0,28.49340,312000.0
4,4.508683e+09,34800872,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,NaT,2025-06-06,138.9960,26,24713.00,312000.0,0.0000,0.0,0,0.00,0.0,True,163200.0,163200.0,326400.0,34,28.50000,At risk,262.0512,26.0,28.48000,312000.0
5,4.508683e+09,34801610,ZGBTR26,STO,12,GB02,GB01,1365010,526605,2025-06-06 20:00:00,2025-06-06 20:30:00,2025-06-07 03:00:00,2025-06-07 03:30:00,NaT,2025-06-06,204.7536,26,27651.92,312000.0,204.7536,2405.0,26,27651.92,312000.0,True,0.0,0.0,0.0,0,0.84808,Load not at risk,NaN,NaN,NaN,NaN
6,4.508683e+09,34801622,ZGBTR26,STO,12,GB02,GB01,1365010,526605,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,NaT,2025-06-06,237.6768,23,27554.40,276000.0,237.6768,1894.0,23,27554.40,276000.0,True,31200.0,19200.0,50400.0,5,0.94560,Light load,5.7024,1.0,0.92756,12000.0
7,4.508683e+09,34800876,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 07:00:00,2025-06-06 07:30:00,NaT,NaT,NaT,2025-06-06,140.0652,26,24889.58,312000.0,17.1072,216.0,3,3028.08,36000.0,True,151200.0,139200.0,290400.0,30,25.47192,At risk,231.3936,23.0,25.47170,273600.0
8,4.508683e+09,34800877,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 04:00:00,2025-06-06 04:30:00,2025-06-06 05:00:00,2025-06-06 06:00:00,NaT,2025-06-06,244.8000,26,27297.10,312000.0,244.8000,1854.0,26,27297.10,312000.0,True,0.0,0.0,0.0,0,1.20290,Load not at risk,NaN,NaN,NaN,NaN
9,4.508683e+09,34801568,ZGBTR26,STO,12,GB01,GB02,526605,1365010,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,NaT,2025-06-06,212.0000,15,23839.20,237600.0,212.0000,440.0,15,23839.20,237600.0,True,52320.0,36480.0,88800.0,9,4.66080,Light load,42.7152,4.0,4.65226,48000.0


In [185]:
#changes 4/20 : Area based optimization
main_load_details[['rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_weight', 'rounded_suggested_deployment_sr_area']] = main_load_details[['rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_weight','rounded_suggested_deployment_sr_area']].fillna(0)
main_load_details['rounded_suggested_deployment_sr_hl'] = main_load_details['rounded_suggested_deployment_sr_hl'] + main_load_details['total_feasible_order_qty_hl']
main_load_details['rounded_suggested_deployment_sr_pal'] = main_load_details['rounded_suggested_deployment_sr_pal'] + main_load_details['total_feasible_order_qty_pal']
# x 1000 for rounded weight because STO grain df is in tonnes
main_load_details['rounded_suggested_deployment_sr_weight'] = main_load_details['rounded_suggested_deployment_sr_weight']*1000 + main_load_details['total_feasible_order_qty_weight']
main_load_details['rounded_suggested_deployment_sr_area'] = main_load_details['rounded_suggested_deployment_sr_area'] + main_load_details['total_feasible_order_area']
main_load_details['Cancel_load'] = np.where((26-main_load_details['total_feasible_order_qty_pal']-main_load_details['rounded_suggested_deployment_sr_pal'])/26 > cancel_load_threshold, 'Yes', 'No')

In [186]:
main_load_details['Agreement to Recommendation(Yes/No)']= ''
main_load_details['Recommendation Executed(Yes/No)']= ''
main_load_details['Reason for non-agreement/non-execution']= ''

In [187]:
main_load_details.drop_duplicates(subset=['load_id'], inplace=True)

In [188]:
main_load_details.loc[main_load_details['Action']!='Load not at risk',:]

,RFRC_NUM12,load_id,trailer_equipment_type_code,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,actual_loading_start_ts,planned_movement_ts,original_quantity_ordered_hl,original_quantity_ordered_pal,original_quantity_ordered_kg,total_area_in_cm2,total_feasible_order_qty_hl,total_feasible_order_qty_pc,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area,feasible,available_area_top_row,available_area_bottom_row,available_area,available_pal,available_weight,Action,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_area,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution
0,4.508681e+09,34798240,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0,1,2.72700,Light load,145.4112,34.0,26554.00,326400.0,No,,,
1,4.508681e+09,34798241,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0,1,2.72700,Light load,145.4112,34.0,26554.00,326400.0,No,,,
2,4.508681e+09,34798242,NaN,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,NaT,2025-06-06,141.1344,33,25773.00,316800.0,141.1344,1782.0,33,25773.00,316800.0,True,9600.0,0.0,9600.0,1,2.72700,Light load,145.4112,34.0,26545.36,326400.0,No,,,
3,4.508683e+09,34800871,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,NaT,2025-06-06,138.9960,26,24713.00,312000.0,0.0000,0.0,0,0.00,0.0,True,163200.0,163200.0,326400.0,34,28.50000,At risk,261.9936,26.0,28493.40,312000.0,No,,,
4,4.508683e+09,34800872,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,NaT,2025-06-06,138.9960,26,24713.00,312000.0,0.0000,0.0,0,0.00,0.0,True,163200.0,163200.0,326400.0,34,28.50000,At risk,262.0512,26.0,28480.00,312000.0,No,,,
6,4.508683e+09,34801622,ZGBTR26,STO,12,GB02,GB01,1365010,526605,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,NaT,2025-06-06,237.6768,23,27554.40,276000.0,237.6768,1894.0,23,27554.40,276000.0,True,31200.0,19200.0,50400.0,5,0.94560,Light load,243.3792,24.0,28481.96,288000.0,No,,,
7,4.508683e+09,34800876,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 07:00:00,2025-06-06 07:30:00,NaT,NaT,NaT,2025-06-06,140.0652,26,24889.58,312000.0,17.1072,216.0,3,3028.08,36000.0,True,151200.0,139200.0,290400.0,30,25.47192,At risk,248.5008,26.0,28499.78,309600.0,No,,,
9,4.508683e+09,34801568,ZGBTR26,STO,12,GB01,GB02,526605,1365010,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,NaT,2025-06-06,212.0000,15,23839.20,237600.0,212.0000,440.0,15,23839.20,237600.0,True,52320.0,36480.0,88800.0,9,4.66080,Light load,254.7152,19.0,28491.46,285600.0,No,,,
10,4.508683e+09,34800880,ZGBTR26,STO,12,GB01,GB28,526605,1537973,2025-06-06 03:00:00,2025-06-06 03:30:00,2025-06-06 04:00:00,2025-06-06 05:00:00,NaT,2025-06-06,130.0116,26,21891.20,266400.0,130.0116,1875.0,26,21891.20,266400.0,True,31200.0,28800.0,60000.0,6,6.60880,Light load,180.8116,31.0,28424.60,321600.0,No,,,
11,4.508683e+09,34801613,ZGBTR26,STO,12,GB01,GB02,526605,1365010,2025-06-06 20:00:00,2025-06-06 20:30:00,2025-06-07 03:00:00,2025-06-07 03:30:00,NaT,2025-06-06,204.6080,19,27342.80,293280.0,204.6080,800.0,19,27342.80,293280.0,True,12480.0,20640.0,33120.0,3,1.15720,Light load,215.1680,20.0,28494.80,305280.0,No,,,


In [189]:
#changes 4/20 : Area based optimization
main_load_details = main_load_details[['RFRC_NUM12', 'load_id', 'movement_type', 'Priority Flag', 'Source',
    'Destination', 'origin_shipping_location_sk',
    'destination_shipping_location_sk', 'origin_slot_arrival',
    'origin_slot_departure', 'destination_slot_arrival',
    'destination_slot_departure', 'Action', 'original_quantity_ordered_hl',
    'original_quantity_ordered_pal', 'original_quantity_ordered_kg','total_area_in_cm2',
    'total_feasible_order_qty_hl','total_feasible_order_qty_pal', 'total_feasible_order_qty_weight','total_feasible_order_area',
    'available_pal', 'available_weight', 'available_area',
    'rounded_suggested_deployment_sr_hl', 'rounded_suggested_deployment_sr_pal',
    'rounded_suggested_deployment_sr_weight','rounded_suggested_deployment_sr_area', 'Cancel_load',
    'Agreement to Recommendation(Yes/No)',
    'Recommendation Executed(Yes/No)',
    'Reason for non-agreement/non-execution']]

In [190]:
main_load_details['available_pal'] = np.where(main_load_details['available_pal']<0, 0, main_load_details['available_pal'])
main_load_details['available_weight'] = np.where(main_load_details['available_weight']<0, 0, main_load_details['available_weight'])
main_load_details['Day_tag'] = tag

In [191]:
#In the rounded sugg columns, entires contain aggregation of total feasible order qty in PAL for already existing SKUs + additional swap ins together for loads that are at risk. Only delta for loads that are light loads and 0 for not at risk
main_load_details.loc[:,load_cols_to_show + ['rounded_suggested_deployment_sr_hl','rounded_suggested_deployment_sr_pal','rounded_suggested_deployment_sr_area','Cancel_load']].sort_values(['load_id'])

,load_id,original_quantity_ordered_hl,original_quantity_ordered_pal,total_feasible_order_qty_hl,total_feasible_order_qty_pal,available_area,available_pal,available_weight,Action,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_area,Cancel_load
0,34798240,141.1344,33,141.1344,33,9600.0,1,2.72700,Light load,145.4112,34.0,326400.0,No
1,34798241,141.1344,33,141.1344,33,9600.0,1,2.72700,Light load,145.4112,34.0,326400.0,No
2,34798242,141.1344,33,141.1344,33,9600.0,1,2.72700,Light load,145.4112,34.0,326400.0,No
12,34800698,154.4400,26,154.4400,26,0.0,0,3.17600,Load not at risk,154.4400,26.0,312000.0,No
13,34800720,148.5000,25,148.5000,25,19200.0,2,4.15000,Light load,159.3000,26.0,312000.0,No
3,34800871,138.9960,26,0.0000,0,326400.0,34,28.50000,At risk,261.9936,26.0,312000.0,No
4,34800872,138.9960,26,0.0000,0,326400.0,34,28.50000,At risk,262.0512,26.0,312000.0,No
7,34800876,140.0652,26,17.1072,3,290400.0,30,25.47192,At risk,248.5008,26.0,309600.0,No
8,34800877,244.8000,26,244.8000,26,0.0,0,1.20290,Load not at risk,244.8000,26.0,312000.0,No
10,34800880,130.0116,26,130.0116,26,60000.0,6,6.60880,Light load,180.8116,31.0,321600.0,No


In [192]:
# main_load_details.to_excel(f"{result_path}{tag}_load_level_report.xlsx", index=False)

In [193]:
open_sto_out_swaps_combined_backup = open_sto_out_swaps_combined.copy()

In [194]:
open_sto_out_swaps_combined

,load_id,RFRC_NUM12,movement_type,sales_document_item_code,Source,Destination,Priority Flag,material_sk,material_code,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,swap_out_qty_hl,suggested_deployment_sr_hl,at_risk_flag,pal_length,pal_width,Action,suggested_deployment_sr_pal,demand_at_dt_hl,perc_oos_risk_at_dt_hl,container_type_description,PC_PAL,PC_HL,PAL_STACKING_FACTOR,HL_PAL,HL_weight,suggested_deployment_sr_weight,suggested_deployment_sr_pc,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_pc,rounded_suggested_deployment_sr_area
11,34800871,4.508683e+09,STO,10.0,GB01,GB28,12,1870515,105537,0.000000,0.00000,0.00000,0.0000,0.000,0.0,0.000000,138.996,0.0000,True,120.0,100.0,Swap-Out,0.0,NaN,NaN,BOTTLE,0.011111,0.05940,1.0,0.187056,0.177796,0.00000,0.0,0,0.0000,0.00000,0,0.0
12,34800872,4.508683e+09,STO,10.0,GB01,GB28,12,1870515,105537,0.000000,0.00000,0.00000,0.0000,0.000,0.0,0.000000,138.996,0.0000,True,120.0,100.0,Swap-Out,0.0,NaN,NaN,BOTTLE,0.011111,0.05940,1.0,0.187056,0.177796,0.00000,0.0,0,0.0000,0.00000,0,0.0
23,34800876,4.508683e+09,STO,20.0,GB01,GB28,12,1870515,105537,0.000000,0.00000,0.00000,0.0000,277.992,0.0,0.000000,122.958,0.0000,True,120.0,100.0,Swap-Out,0.0,NaN,NaN,BOTTLE,0.011111,0.05940,1.0,0.187056,0.177796,0.00000,0.0,0,0.0000,0.00000,0,0.0
1,34798240,4.508681e+09,STO,NaN,GB28,IT12,11,2073737,99923,218.354400,218.35440,0.00000,0.0000,0.000,0.0,0.000000,NaN,4.2768,NaN,120.0,80.0,Light load,1.0,4568.148388,0.014348,BOTTLE,0.018519,0.07920,1.0,0.233820,0.182613,0.78100,54.0,1,4.2768,0.78100,54,9600.0
3,34798241,4.508681e+09,STO,NaN,GB28,IT12,11,2073737,99923,218.354400,218.35440,0.00000,0.0000,0.000,0.0,0.000000,NaN,4.2768,NaN,120.0,80.0,Light load,1.0,4568.148388,0.014348,BOTTLE,0.018519,0.07920,1.0,0.233820,0.182613,0.78100,54.0,1,4.2768,0.78100,54,9600.0
4,34798242,4.508681e+09,STO,NaN,GB28,IT12,11,1869530,105531,830.253600,830.25360,0.00000,0.0000,0.000,0.0,0.000000,NaN,4.2768,NaN,120.0,80.0,Light load,1.0,16989.632800,0.679097,BOTTLE,0.018519,0.07920,1.0,0.233820,0.180593,0.77236,54.0,1,4.2768,0.77236,54,9600.0
20,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1903370,103436,189.240000,189.24000,0.00000,0.0000,0.000,0.0,0.000000,NaN,75.6000,NaN,120.0,100.0,At risk,7.0,11207.013344,0.115001,CAN,0.011111,0.12000,1.0,0.092593,0.108593,8.20960,630.0,7,75.6000,8.20960,630,84000.0
40,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1982919,99200,316.436875,600.96000,0.00000,0.0000,0.000,0.0,284.523125,NaN,32.4000,NaN,120.0,100.0,At risk,3.0,100.249300,0.589351,CAN,0.011111,0.12000,1.0,0.092593,0.109519,3.54840,270.0,3,32.4000,3.54840,270,36000.0
42,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1870722,101002,48.720000,48.72000,0.00000,0.0000,0.000,0.0,0.000000,NaN,9.6000,NaN,120.0,100.0,At risk,1.0,2480.581000,0.000000,CAN,0.012500,0.12000,1.0,0.104167,0.108813,1.04460,80.0,1,9.6000,1.04460,80,12000.0
60,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1886284,101371,64.800000,64.80000,0.00000,0.0000,0.000,0.0,0.000000,NaN,10.8000,NaN,120.0,100.0,At risk,1.0,1341.153200,0.000000,CAN,0.011111,0.12000,1.0,0.092593,0.108593,1.17280,90.0,1,10.8000,1.17280,90,12000.0


In [195]:
#check if this merge at RFRCNUM12 is correct
open_sto_out_swaps_combined = pd.merge(open_sto_out_swaps_combined, main_load_details[['RFRC_NUM12', 'origin_slot_arrival', 'origin_slot_departure', 'destination_slot_arrival', 'destination_slot_departure', 'total_feasible_order_qty_pal', 'total_feasible_order_qty_weight','origin_shipping_location_sk', 'destination_shipping_location_sk', 'Cancel_load']], on = ['RFRC_NUM12'], how= 'left')
#converting feasible weight at STO level to tonnes
open_sto_out_swaps_combined['total_feasible_order_qty_weight'] = open_sto_out_swaps_combined['total_feasible_order_qty_weight']/1000
open_sto_out_swaps_combined['swap_out_qty_hl'] = open_sto_out_swaps_combined['swap_out_qty_hl'].fillna(0)

In [196]:
open_sto_out_swaps_combined.sort_values(['load_id'])

,load_id,RFRC_NUM12,movement_type,sales_document_item_code,Source,Destination,Priority Flag,material_sk,material_code,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,swap_out_qty_hl,suggested_deployment_sr_hl,at_risk_flag,pal_length,pal_width,Action,suggested_deployment_sr_pal,demand_at_dt_hl,perc_oos_risk_at_dt_hl,container_type_description,PC_PAL,PC_HL,PAL_STACKING_FACTOR,HL_PAL,HL_weight,suggested_deployment_sr_weight,suggested_deployment_sr_pc,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_pc,rounded_suggested_deployment_sr_area,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,total_feasible_order_qty_pal,total_feasible_order_qty_weight,origin_shipping_location_sk,destination_shipping_location_sk,Cancel_load
3,34798240,4.508681e+09,STO,NaN,GB28,IT12,11,2073737,99923,218.354400,218.35440,0.00000,0.0000,0.000,0.0,0.000000,0.000,4.2768,NaN,120.0,80.0,Light load,1.0,4568.148388,0.014348,BOTTLE,0.018519,0.07920,1.0,0.233820,0.182613,0.78100,54.0,1,4.2768,0.78100,54,9600.0,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,33,25.77300,1537973,2967078,No
4,34798241,4.508681e+09,STO,NaN,GB28,IT12,11,2073737,99923,218.354400,218.35440,0.00000,0.0000,0.000,0.0,0.000000,0.000,4.2768,NaN,120.0,80.0,Light load,1.0,4568.148388,0.014348,BOTTLE,0.018519,0.07920,1.0,0.233820,0.182613,0.78100,54.0,1,4.2768,0.78100,54,9600.0,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,33,25.77300,1537973,2967078,No
5,34798242,4.508681e+09,STO,NaN,GB28,IT12,11,1869530,105531,830.253600,830.25360,0.00000,0.0000,0.000,0.0,0.000000,0.000,4.2768,NaN,120.0,80.0,Light load,1.0,16989.632800,0.679097,BOTTLE,0.018519,0.07920,1.0,0.233820,0.180593,0.77236,54.0,1,4.2768,0.77236,54,9600.0,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,33,25.77300,1537973,2967078,No
31,34800720,4.508683e+09,STO,NaN,GB01,GB67,11,1982919,99200,316.436875,600.96000,0.00000,0.0000,0.000,0.0,284.523125,0.000,10.8000,NaN,120.0,100.0,Light load,1.0,112.990303,0.000000,CAN,0.011111,0.12000,1.0,0.092593,0.109519,1.18280,90.0,1,10.8000,1.18280,90,12000.0,2025-06-06 03:00:00,2025-06-06 03:30:00,2025-06-06 14:00:00,2025-06-06 15:00:00,25,24.35000,526605,1565895,No
0,34800871,4.508683e+09,STO,10.0,GB01,GB28,12,1870515,105537,0.000000,0.00000,0.00000,0.0000,0.000,0.0,0.000000,138.996,0.0000,True,120.0,100.0,Swap-Out,0.0,NaN,NaN,BOTTLE,0.011111,0.05940,1.0,0.187056,0.177796,0.00000,0.0,0,0.0000,0.00000,0,0.0,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,0,0.00000,526605,1537973,No
10,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1896054,107451,2044.273123,3791.60448,0.00000,0.0000,0.000,0.0,1747.331357,0.000,133.5936,NaN,120.0,100.0,At risk,14.0,6759.407386,0.000000,CAN,0.014286,0.13632,1.0,0.104795,0.108673,14.51800,980.0,14,133.5936,14.51800,980,168000.0,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,0,0.00000,526605,1537973,No
8,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1870722,101002,48.720000,48.72000,0.00000,0.0000,0.000,0.0,0.000000,0.000,9.6000,NaN,120.0,100.0,At risk,1.0,2480.581000,0.000000,CAN,0.012500,0.12000,1.0,0.104167,0.108813,1.04460,80.0,1,9.6000,1.04460,80,12000.0,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,0,0.00000,526605,1537973,No
9,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1886284,101371,64.800000,64.80000,0.00000,0.0000,0.000,0.0,0.000000,0.000,10.8000,NaN,120.0,100.0,At risk,1.0,1341.153200,0.000000,CAN,0.011111,0.12000,1.0,0.092593,0.108593,1.17280,90.0,1,10.8000,1.17280,90,12000.0,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,0,0.00000,526605,1537973,No
6,34800871,4.508683e+09,STO,NaN,GB01,GB28,12,1903370,103436,189.240000,189.24000,0.00000,0.0000,0.000,0.0,0.000000,0.000,75.6000,NaN,120.0,100.0,At risk,7.0,1120

In [197]:
open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['swap_out_qty_hl']>0)&(open_sto_out_swaps_combined['suggested_deployment_sr_hl']>0), 'Swap-out (Update)', open_sto_out_swaps_combined['Action'])
open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['swap_out_qty_hl']>0)&(open_sto_out_swaps_combined['suggested_deployment_sr_hl']==0), 'Swap-out (Delete)', open_sto_out_swaps_combined['Action'])

In [198]:
open_sto_out_swaps_combined = pd.merge(open_sto_out_swaps_combined, open_sto_out[['load_id', 'material_sk', 'total_quantity_hl']], on = ['load_id', 'material_sk'], how = 'left').fillna(0)

In [199]:
open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['Action']=='Light load')&(open_sto_out_swaps_combined['total_quantity_hl']==0), 'Top-up (New)', open_sto_out_swaps_combined['Action'])
open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['Action']=='Light load')&(open_sto_out_swaps_combined['total_quantity_hl']!=0), 'Top-up (Update)', open_sto_out_swaps_combined['Action'])
open_sto_out_swaps_combined['Action'] = np.where((open_sto_out_swaps_combined['Action']=='At risk'), 'Swap-in', open_sto_out_swaps_combined['Action'])

In [200]:
open_sto_out_swaps_combined['Agreement to Recommendation(Yes/No)']= ''
open_sto_out_swaps_combined['Recommendation Executed(Yes/No)']= ''
open_sto_out_swaps_combined['Reason for non-agreement/non-execution']= ''
open_sto_out_swaps_combined = open_sto_out_swaps_combined.fillna(0)

In [201]:
open_sto_out_swaps_combined = open_sto_out_swaps_combined[['RFRC_NUM12', 'load_id', 'movement_type', 'sales_document_item_code', 'Priority Flag', 'origin_slot_arrival', 'origin_slot_departure', 'Source',
    'Destination', 'origin_shipping_location_sk', 'destination_shipping_location_sk', 
    'material_sk', 'material_code', 'Action', 'stock_on_hand_sr_hl', 'stock_sr_hl', 'planned_production_sr_hl', 'actual_production_sr_hl', 'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl', 
    'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl', 'swap_out_qty_hl', 'suggested_deployment_sr_hl',
    'suggested_deployment_sr_pal', 'suggested_deployment_sr_pc',
    'suggested_deployment_sr_weight', 'rounded_suggested_deployment_sr_hl',
    'rounded_suggested_deployment_sr_pal', 'rounded_suggested_deployment_sr_pc',
    'rounded_suggested_deployment_sr_weight', 'Cancel_load',
    'Agreement to Recommendation(Yes/No)',
    'Recommendation Executed(Yes/No)',
    'Reason for non-agreement/non-execution']]

open_sto_out_swaps_combined['Day_tag'] = tag
    
open_sto_out_swaps_combined.rename(columns={'RFRC_NUM12':'STO Number'}, inplace=True)

In [202]:
### Dropping rows where the rounded recommendations for Swap-in or top-up are equal to zero
open_sto_out_swaps_combined = open_sto_out_swaps_combined[(~open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Top-up (Update)', 'Swap-in']))|(open_sto_out_swaps_combined['rounded_suggested_deployment_sr_pal']!=0)]

In [203]:
kpi_results

{'Total number of loads': {'Value': 18, 'Percentage': 100},
 'Loads that are at risk or have a light load': {'Value': 12,
  'Percentage': 66.66666666666666},
 'Loads that are at risk': {'Value': 3, 'Percentage': 16.666666666666664},
 'Loads that are a light load': {'Value': 9, 'Percentage': 50.0},
 'Loads with LCP swaps available': {'Value': 12,
  'Percentage': 66.66666666666666}}

In [204]:
print('### Number of loads with swap-ins or top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique())
kpi_results['Number of loads with swap-ins or top-ups'] = {
    'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique(),
    'Percentage':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])]['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
}

### Number of loads with swap-ins or top-ups:  12


In [205]:
print('### Number of swap-ins or top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])].shape[0])
kpi_results['Number of swap-ins or top-ups'] = {
    'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)', 'Swap-in', 'Top-up (Update)'])].shape[0],
    'Percentage':  '-'
}

### Number of swap-ins or top-ups:  31


In [206]:
print('### Number of loads with swap-ins: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in']['load_id'].nunique())
kpi_results['Number of loads with swap-ins'] = {
    'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in']['load_id'].nunique(),
    'Percentage':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in']['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
}

### Number of loads with swap-ins:  3


In [207]:
print('### Number of swap-ins: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in'].shape[0])
kpi_results['Number of swap-ins'] = {
    'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action']=='Swap-in'].shape[0],
    'Percentage':  '-'
}

### Number of swap-ins:  16


In [208]:
print('### Number of loads with top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])]['load_id'].nunique())
kpi_results['Number of loads with top-ups'] = {
    'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])]['load_id'].nunique(),
    'Percentage':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])]['load_id'].nunique()/main_outbound_df['load_id'].nunique() * 100
}

### Number of loads with top-ups:  9


In [209]:
print('### Number of top-ups: ', open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])].shape[0])
kpi_results['Number of top-ups'] = {
    'Value':  open_sto_out_swaps_combined[open_sto_out_swaps_combined['Action'].isin(['Top-up (New)','Top-up (Update)'])].shape[0],
    'Percentage':  '-'
}

### Number of top-ups:  15


In [210]:
main_load_details.loc[main_load_details['Action']!= 'Load not at risk',:]

,RFRC_NUM12,load_id,movement_type,Priority Flag,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,origin_slot_arrival,origin_slot_departure,destination_slot_arrival,destination_slot_departure,Action,original_quantity_ordered_hl,original_quantity_ordered_pal,original_quantity_ordered_kg,total_area_in_cm2,total_feasible_order_qty_hl,total_feasible_order_qty_pal,total_feasible_order_qty_weight,total_feasible_order_area,available_pal,available_weight,available_area,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_weight,rounded_suggested_deployment_sr_area,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag
0,4.508681e+09,34798240,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,Light load,141.1344,33,25773.00,316800.0,141.1344,33,25773.00,316800.0,1,2.72700,9600.0,145.4112,34.0,26554.00,326400.0,No,,,,D0
1,4.508681e+09,34798241,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,Light load,141.1344,33,25773.00,316800.0,141.1344,33,25773.00,316800.0,1,2.72700,9600.0,145.4112,34.0,26554.00,326400.0,No,,,,D0
2,4.508681e+09,34798242,STO,11,GB28,IT12,1537973,2967078,2025-06-06 01:00:00,2025-06-06 02:00:00,NaT,NaT,Light load,141.1344,33,25773.00,316800.0,141.1344,33,25773.00,316800.0,1,2.72700,9600.0,145.4112,34.0,26545.36,326400.0,No,,,,D0
3,4.508683e+09,34800871,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,At risk,138.9960,26,24713.00,312000.0,0.0000,0,0.00,0.0,34,28.50000,326400.0,261.9936,26.0,28493.40,312000.0,No,,,,D0
4,4.508683e+09,34800872,STO,12,GB01,GB28,526605,1537973,2025-06-06 01:00:00,2025-06-06 01:30:00,2025-06-06 02:00:00,2025-06-06 03:00:00,At risk,138.9960,26,24713.00,312000.0,0.0000,0,0.00,0.0,34,28.50000,326400.0,262.0512,26.0,28480.00,312000.0,No,,,,D0
6,4.508683e+09,34801622,STO,12,GB02,GB01,1365010,526605,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,Light load,237.6768,23,27554.40,276000.0,237.6768,23,27554.40,276000.0,5,0.94560,50400.0,243.3792,24.0,28481.96,288000.0,No,,,,D0
7,4.508683e+09,34800876,STO,12,GB01,GB28,526605,1537973,2025-06-06 07:00:00,2025-06-06 07:30:00,NaT,NaT,At risk,140.0652,26,24889.58,312000.0,17.1072,3,3028.08,36000.0,30,25.47192,290400.0,248.5008,26.0,28499.78,309600.0,No,,,,D0
9,4.508683e+09,34801568,STO,12,GB01,GB02,526605,1365010,2025-06-06 16:00:00,2025-06-06 16:30:00,2025-06-06 20:00:00,2025-06-06 20:30:00,Light load,212.0000,15,23839.20,237600.0,212.0000,15,23839.20,237600.0,9,4.66080,88800.0,254.7152,19.0,28491.46,285600.0,No,,,,D0
10,4.508683e+09,34800880,STO,12,GB01,GB28,526605,1537973,2025-06-06 03:00:00,2025-06-06 03:30:00,2025-06-06 04:00:00,2025-06-06 05:00:00,Light load,130.0116,26,21891.20,266400.0,130.0116,26,21891.20,266400.0,6,6.60880,60000.0,180.8116,31.0,28424.60,321600.0,No,,,,D0
11,4.508683e+09,34801613,STO,12,GB01,GB02,526605,1365010,2025-06-06 20:00:00,2025-06-06 20:30:00,2025-06-07 03:00:00,2025-06-07 03:30:00,Light load,204.6080,19,27342.80,293280.0,204.6080,19,27342.80,293280.0,3,1.15720,33120.0,215.1680,20.0,28494.80,305280.0,No,,,,D0


In [211]:
main_load_details[['total_feasible_order_qty_weight','rounded_suggested_deployment_sr_weight']] =  np.round(main_load_details[['total_feasible_order_qty_weight','rounded_suggested_deployment_sr_weight']]/1000,3)


In [212]:
# changes 5/29 : scorecard customization
wps_improvement_summary_overall = main_load_details.loc[main_load_details['Action']!='Load not at risk',['RFRC_NUM12', 'load_id','Day_tag', 'Action','Cancel_load','movement_type','Source','Destination','total_feasible_order_qty_weight','rounded_suggested_deployment_sr_weight']]
wps_improvement_summary_overall['market'] = 'GB'
wps_improvement_summary_overall['max_payload_weight'] = standard_weights.loc[standard_weights['Country'] == 'GB', 'weight_limit'].values[0]
wps_improvement_summary_overall['max_vs_deployment_weight'] = round(wps_improvement_summary_overall['max_payload_weight'] - wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'],3)
wps_improvement_summary_overall['deployment_weight_percent_of_max_weight'] = round((wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'] / wps_improvement_summary_overall['max_payload_weight'])*100.00,2)

wps_improvement_summary_overall['deployment_vs_feasible_weight'] = round(wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'] - wps_improvement_summary_overall['total_feasible_order_qty_weight'],3)
wps_improvement_summary_overall['deployment_weight_percent_of_feasible_weight'] = round((wps_improvement_summary_overall['rounded_suggested_deployment_sr_weight'] / wps_improvement_summary_overall['total_feasible_order_qty_weight'])*100.00,2)
wps_improvement_summary_overall['wps_savings_per_tonne_in_usd'] = wps_savings_per_tonne_in_usd
wps_improvement_summary_overall['wps_cost_savings_in_usd'] = round(wps_improvement_summary_overall['deployment_vs_feasible_weight'] * wps_improvement_summary_overall['wps_savings_per_tonne_in_usd'],2)

wps_improvement_summary_light_loads = wps_improvement_summary_overall.loc[wps_improvement_summary_overall['Action']=='Light load',:]

In [213]:
# changes 5/29 : scorecard customization
if tag != 'D0':
    cancellation_cost_potential_summary = main_load_details.loc[(main_load_details['total_feasible_order_qty_pal']==0) & (main_load_details['rounded_suggested_deployment_sr_pal']>0),['RFRC_NUM12', 'load_id','Day_tag', 'Action','Cancel_load','movement_type','Source','Destination','total_feasible_order_qty_pal','rounded_suggested_deployment_sr_pal']]
    cancellation_cost_potential_summary['average_shipment_price'] = average_shipment_price
    cancellation_cost_potential_summary['perc_trip_price_as_cancellation_cost'] = perc_trip_price_as_cancellation_cost
    cancellation_cost_potential_summary['average_cancellation_cost'] = round(cancellation_cost_potential_summary['average_shipment_price'] * cancellation_cost_potential_summary['perc_trip_price_as_cancellation_cost'],2)
    cancellation_cost_potential_summary['market'] = 'GB'

In [214]:
### initial volume:
metric = main_load_details['original_quantity_ordered_pal'].sum()
perc_metric = main_load_details['original_quantity_ordered_pal'].sum()/main_load_details['original_quantity_ordered_pal'].sum() * 100
print('### Total planned initial volume in PAL: ', )
kpi_results['Total planned initial volume in PAL'] = {
    'Value':  metric,
    'Percentage':  perc_metric
}
### Volumne at risk
metric = main_load_details['original_quantity_ordered_pal'].sum() - main_load_details['total_feasible_order_qty_pal'].sum()
perc_metric = (main_load_details['original_quantity_ordered_pal'].sum() - main_load_details['total_feasible_order_qty_pal'].sum())/main_load_details['original_quantity_ordered_pal'].sum() * 100
print('### Volume at risk in PAL due to insufficient stock at source: ', metric)
kpi_results['Volume at risk in PAL due to insufficient stock at source (% of total planned volume)'] = {
    'Value':  metric,
    'Percentage':  perc_metric
}
### Total optimised volume:
#check if the denominator base should be unavailable volume - yes
metric = main_load_details.loc[main_load_details['Action']=='At risk','rounded_suggested_deployment_sr_pal'].sum() - main_load_details.loc[main_load_details['Action']=='At risk','total_feasible_order_qty_pal'].sum()
perc_metric = (main_load_details.loc[main_load_details['Action']=='At risk','rounded_suggested_deployment_sr_pal'].sum() - main_load_details.loc[main_load_details['Action']=='At risk','total_feasible_order_qty_pal'].sum())/(main_load_details['original_quantity_ordered_pal'].sum() - main_load_details['total_feasible_order_qty_pal'].sum()) * 100
print('### Volume at risk optimized in PAL due to swap-ins', metric)
kpi_results['Volume at risk optimized in PAL due to swap-ins (% of volume at risk)'] = {
    'Value':  metric,
    'Percentage':  perc_metric
}

metric = main_load_details[main_load_details['Action']=='Light load']['rounded_suggested_deployment_sr_pal'].sum() - main_load_details[main_load_details['Action']=='Light load']['total_feasible_order_qty_pal'].sum()
perc_metric = ((main_load_details[main_load_details['Action']=='Light load']['rounded_suggested_deployment_sr_pal'].sum() - main_load_details[main_load_details['Action']=='Light load']['total_feasible_order_qty_pal'].sum())/main_load_details[main_load_details['Action']=='Light load']['available_pal'].sum()) * 100
print('### Light load top-ups performed in PAL: ', metric)
kpi_results['Light load top-ups performed in PAL (% of available PAL for light loads)'] = {
    'Value':  metric,
    'Percentage':  perc_metric
}

metric = np.round(main_load_details['total_feasible_order_qty_weight'].mean(),3)
print('### Average feasible weight per shipment in Tonnes pre optimization: ', metric)
kpi_results['Average feasible weight per shipment in Tonnes pre optimization'] = {
    'Value':  metric,
    'Percentage':  '-'
}
metric = np.round(main_load_details['rounded_suggested_deployment_sr_weight'].mean(),3)
print('### Average weight per shipment in Tonnes post optimization: ', metric)
kpi_results['Average weight per shipment in Tonnes post optimization'] = {
    'Value':  metric,
    'Percentage':  '-'
}
metric = np.round((main_load_details['rounded_suggested_deployment_sr_weight'].mean()) - (main_load_details['total_feasible_order_qty_weight'].mean()),3)
print('### Improvement in average weight per shipment in Tonnes due to optimization: ', metric)
kpi_results['Improvement in average weight per shipment in Tonnes due to optimization'] = {
    'Value': metric,
    'Percentage':  '-'
}
metric = np.round((main_load_details.loc[main_load_details['Action']=='Light load','total_feasible_order_qty_weight'].mean()),3)
print('### Average feasible weight per shipment in Tonnes pre optimization (only for light loads): ', metric)
kpi_results['Average feasible weight per shipment in Tonnes pre optimization (only for light loads)'] = {
    'Value': metric,
    'Percentage':  '-'
}
metric = np.round((main_load_details.loc[main_load_details['Action']=='Light load','rounded_suggested_deployment_sr_weight'].mean()),3)
print('### Average weight per shipment in Tonnes post optimization (only for light loads): ',  metric)
kpi_results['Average weight per shipment in Tonnes post optimization (only for light loads)'] = {
    'Value': metric,
    'Percentage':  '-'
}
metric = np.round(((main_load_details.loc[main_load_details['Action']=='Light load','rounded_suggested_deployment_sr_weight']) - (main_load_details.loc[main_load_details['Action']=='Light load','total_feasible_order_qty_weight'])).mean(),3)
print('### Improvement in average weight per shipment in Tonnes due to optimization (only for light loads): ', metric)
kpi_results['Improvement in average weight per shipment in Tonnes due to optimization (only for light loads)'] = {
    'Value': metric,
    'Percentage':  '-'
}
# changes 5/29 : scorecard customization
metric = round(wps_improvement_summary_overall['wps_cost_savings_in_usd'].sum(),2)
print('### USD Benefits generated from WPS improvement across all loads: ', metric)
kpi_results['USD Benefits generated from WPS improvement across all loads:'] = {
    'Value': round(metric,2),
    'Percentage':  '-'
}
metric = round(wps_improvement_summary_light_loads['wps_cost_savings_in_usd'].sum(),2)
print('### USD Benefits generated from WPS improvement across light loads: ', metric)
kpi_results['USD Benefits generated from WPS improvement across light loads:'] = {
    'Value': round(metric,2),
    'Percentage':  '-'
}
if tag!='D0':
    metric = round(cancellation_cost_potential_summary['average_cancellation_cost'].sum(),2)
    print('### Potential D1 cancellation avoidance cost savings across all loads: ', metric)
    kpi_results['Potential D1 cancellation avoidance cost savings across all loads:'] = {
        'Value': round(metric,2),
        'Percentage':  '-'
    }


### Total planned initial volume in PAL: 
### Volume at risk in PAL due to insufficient stock at source:  75
### Volume at risk optimized in PAL due to swap-ins 75.0
### Light load top-ups performed in PAL:  18.0
### Average feasible weight per shipment in Tonnes pre optimization:  21.361
### Average weight per shipment in Tonnes post optimization:  27.399
### Improvement in average weight per shipment in Tonnes due to optimization:  6.039
### Average feasible weight per shipment in Tonnes pre optimization (only for light loads):  25.263
### Average weight per shipment in Tonnes post optimization (only for light loads):  27.508
### Improvement in average weight per shipment in Tonnes due to optimization (only for light loads):  2.246
### USD Benefits generated from WPS improvement across all loads:  1026.55
### USD Benefits generated from WPS improvement across light loads:  202.1


In [215]:
kpi_df = pd.DataFrame.from_dict(kpi_results, orient='index')

# Save to excel file (or any other format)
#kpi_df.to_excel(f"{result_path}{tag}_Score_card.xlsx")

In [216]:
## Need to recheck the open_sto_in and open_sto_out part.
# updated_stock = calculate_end_of_day_stock(stock, open_so, open_sto_out, production, actual_production, swaps_df, run_time)

In [217]:
open_sto_out_swaps_combined

,STO Number,load_id,movement_type,sales_document_item_code,Priority Flag,origin_slot_arrival,origin_slot_departure,Source,Destination,origin_shipping_location_sk,destination_shipping_location_sk,material_sk,material_code,Action,stock_on_hand_sr_hl,stock_sr_hl,planned_production_sr_hl,actual_production_sr_hl,outgoing_so_sto_sr_hl,incoming_sto_sr_hl,safety_stock_sr_hl,demand_at_dt_hl,perc_oos_risk_at_dt_hl,swap_out_qty_hl,suggested_deployment_sr_hl,suggested_deployment_sr_pal,suggested_deployment_sr_pc,suggested_deployment_sr_weight,rounded_suggested_deployment_sr_hl,rounded_suggested_deployment_sr_pal,rounded_suggested_deployment_sr_pc,rounded_suggested_deployment_sr_weight,Cancel_load,Agreement to Recommendation(Yes/No),Recommendation Executed(Yes/No),Reason for non-agreement/non-execution,Day_tag
0,4.508683e+09,34800871,STO,10.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.00000,0.0000,0.000,0.0,0.000000,0.000000,0.000000,138.996,0.0000,0.0,0.0,0.00000,0.0000,0,0,0.00000,No,,,,D0
1,4.508683e+09,34800872,STO,10.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.00000,0.0000,0.000,0.0,0.000000,0.000000,0.000000,138.996,0.0000,0.0,0.0,0.00000,0.0000,0,0,0.00000,No,,,,D0
2,4.508683e+09,34800876,STO,20.0,12,2025-06-06 07:00:00,2025-06-06 07:30:00,GB01,GB28,526605,1537973,1870515,105537,Swap-out (Delete),0.000000,0.00000,0.00000,0.0000,277.992,0.0,0.000000,0.000000,0.000000,122.958,0.0000,0.0,0.0,0.00000,0.0000,0,0,0.00000,No,,,,D0
3,4.508681e+09,34798240,STO,0.0,11,2025-06-06 01:00:00,2025-06-06 02:00:00,GB28,IT12,1537973,2967078,2073737,99923,Top-up (Update),218.354400,218.35440,0.00000,0.0000,0.000,0.0,0.000000,4568.148388,0.014348,0.000,4.2768,1.0,54.0,0.78100,4.2768,1,54,0.78100,No,,,,D0
4,4.508681e+09,34798241,STO,0.0,11,2025-06-06 01:00:00,2025-06-06 02:00:00,GB28,IT12,1537973,2967078,2073737,99923,Top-up (Update),218.354400,218.35440,0.00000,0.0000,0.000,0.0,0.000000,4568.148388,0.014348,0.000,4.2768,1.0,54.0,0.78100,4.2768,1,54,0.78100,No,,,,D0
5,4.508681e+09,34798242,STO,0.0,11,2025-06-06 01:00:00,2025-06-06 02:00:00,GB28,IT12,1537973,2967078,1869530,105531,Top-up (New),830.253600,830.25360,0.00000,0.0000,0.000,0.0,0.000000,16989.632800,0.679097,0.000,4.2768,1.0,54.0,0.77236,4.2768,1,54,0.77236,No,,,,D0
6,4.508683e+09,34800871,STO,0.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1903370,103436,Swap-in,189.240000,189.24000,0.00000,0.0000,0.000,0.0,0.000000,11207.013344,0.115001,0.000,75.6000,7.0,630.0,8.20960,75.6000,7,630,8.20960,No,,,,D0
7,4.508683e+09,34800871,STO,0.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1982919,99200,Swap-in,316.436875,600.96000,0.00000,0.0000,0.000,0.0,284.523125,100.249300,0.589351,0.000,32.4000,3.0,270.0,3.54840,32.4000,3,270,3.54840,No,,,,D0
8,4.508683e+09,34800871,STO,0.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1870722,101002,Swap-in,48.720000,48.72000,0.00000,0.0000,0.000,0.0,0.000000,2480.581000,0.000000,0.000,9.6000,1.0,80.0,1.04460,9.6000,1,80,1.04460,No,,,,D0
9,4.508683e+09,34800871,STO,0.0,12,2025-06-06 01:00:00,2025-06-06 01:30:00,GB01,GB28,526605,1537973,1886284,101371,Swap-in,64.800000,64.80000,0.00000,0.0000,0.000,0.0,0.000000,1341.153200,0.000000,0.000,10.8000,1.0,90.0,1.17280,10.8000,1,90,1.17280,No,,,,D0


In [218]:
#open_sto_out_swaps_combined.to_excel(f"{result_path}{tag}_Swaps.xlsx", index=False)

In [219]:
#testing start for calculate end of day stocks : calculate_end_of_day_stock(stock, open_so, open_sto_out, production, actual_production, swaps_df, run_time)

In [220]:
run_time

Timestamp('2025-06-05 13:27:24')

In [221]:
stock_backup = stock.copy()
open_so_backup = open_so.copy()
open_sto_out_backup = open_sto_out.copy()
production_backup = production.copy()
actual_production_backup = actual_production.copy()
open_sto_out_swaps_combined_backup = open_sto_out_swaps_combined.copy()


In [222]:
updated_stock = stock.copy()

In [223]:
idx = 0
row = updated_stock.iloc[idx]

In [224]:
row

material_code        100004
Source                 IT12
material_sk         1874495
plant_sk              13688
opening_stock_hl    46.0152
Name: 0, dtype: object

In [225]:
sku = row['material_sk']
source = row['Source']

# Stock at hand
stock_at_hand = row['opening_stock_hl']

In [226]:
stock_at_hand

46.01519999999999

In [227]:
# Outgoing SO for the whole day
outgoing_so = open_so[
    (open_so['material_sk'] == sku) &
    (open_so['Source'] == source) &
    (open_so['Delivery Date'] == run_time.normalize())
]['open_so_out_hl'].sum()
print(outgoing_so)

0.0


In [228]:
# Updated incoming Open STO
#why is it only for loads at risk/swaps df
incoming_sto = open_sto_out_swaps_combined[
    (open_sto_out_swaps_combined['material_sk'] == sku) &
    (open_sto_out_swaps_combined['Destination'] == source) &
    (open_sto_out_swaps_combined['suggested_deployment_sr_hl'] >0)
]['suggested_deployment_sr_hl'].sum()
print(incoming_sto)

0.0


In [229]:
incoming_og_sto = open_sto_out[
    (open_sto_out['material_sk'] == sku) &
    (open_sto_out['Destination']==source)&
    (open_sto_out['at_risk_flag']==False)
]['total_quantity_hl'].sum()
print(incoming_og_sto)

0.0


In [230]:
# Updated outgoing Open STO
outgoing_sto = open_sto_out_swaps_combined[
    (open_sto_out_swaps_combined['material_sk'] == sku) &
    (open_sto_out_swaps_combined['Source'] == source) &
    (open_sto_out_swaps_combined['suggested_deployment_sr_hl'] >0)
]['suggested_deployment_sr_hl'].sum()
print(outgoing_sto)

0.0


In [231]:
outgoing_og_sto = open_sto_out[
    (open_sto_out['material_sk'] == sku) &
    (open_sto_out['Source']==source)&
    (open_sto_out['at_risk_flag']==False)
]['total_quantity_hl'].sum()
print(outgoing_og_sto)

0.0


In [232]:
#AI to see if buffer needs to be added for planned production for all plants
try:
    # Planned production for the whole day
    planned_production = production[
        (production['material_sk'] == sku) &
        (production['plant_code'] == source) &
        (production['release_ts'].dt.normalize() == run_time.normalize())
    ]['production_hl'].sum()
except:
    planned_production = 0
print(planned_production)

0.0


In [233]:
# Calculate the stock at hand at the end of the day
stock_at_hand_end_of_day = stock_at_hand - outgoing_so + incoming_sto + incoming_og_sto - outgoing_sto - outgoing_og_sto + planned_production
print(stock_at_hand_end_of_day)

46.01519999999999


In [234]:
updated_stock.at[idx, 'Closing_Stock'] = stock_at_hand_end_of_day

In [235]:
#testing end for calculate end of day stocks : calculate_end_of_day_stock(stock, open_so, open_sto_out, production, actual_production, swaps_df, run_time)
# updated_stock = calculate_end_of_day_stock(stock, open_so, open_sto_out, production, actual_production, swaps_df, run_time)

In [236]:
# changes 5/27 : Pre-load changes
updated_stock = calculate_end_of_day_stock(stock, open_so, open_sto_out, pre_load_sto_out_df, production, actual_production, open_sto_out_swaps_combined, run_time)

In [237]:
#testing end for process_loads
# swaps_df_d0, main_load_details_d0, updated_stock = process_loads(main_outbound_df, main_inbound_df, main_load_details, stock, open_so, open_sto, production, actual_production, inventory_policy, lcp_data, load_details_df, run_time, result_path, 'D0')

In [238]:
#general testing end

testing end

In [239]:
# # Call the function and capture the problem object
# problem = optimise_loads(loads_at_risk_or_light_lcp_obs_enriched)

# # Print only variables that have been assigned a non-zero value
# for var in problem.variables():
#     if var.varValue != 0:
#         print(var.name, var.varValue)


In [240]:
print("Objective value:", value(problem.objective))


Objective value: 920.3463999999998


In [241]:
# loads_at_risk_or_light_lcp_obs_enriched['obj_coeff'] = (
#     loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] +
#     1 +
#     (1 * loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] +
#      1 * loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) +
#     loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled']
# )
# print(loads_at_risk_or_light_lcp_obs_enriched[['material_sk', 'load_id', 'obj_coeff']].head(10))


In [242]:
# result_df = loads_at_risk_or_light_lcp_obs_enriched[['material_sk', 'load_id', 'Qty_LPVar']].copy()
# result_df['optimized_qty'] = result_df['Qty_LPVar'].apply(lambda x: x.varValue if hasattr(x, 'varValue') else 0)
# result_agg = result_df.groupby('load_id')['optimized_qty'].sum()


In [243]:
# for name, constraint in problem.constraints.items():
#     print(f"{name}: {constraint.value()} (Slack: {constraint.slack})")


In [244]:
# for w1 in [0.1, 0.5, 1, 2]:
#     for w2 in [0.1, 0.5, 1, 2]:
#         print(f"Trying w1={w1}, w2={w2}")
#         loads_at_risk_or_light_lcp_obs_enriched['obj_coeff'] = (
#             loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] +
#             1 +
#             (w1 * loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled'] +
#              w2 * loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) +
#             loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled']
#         )
#         # define and solve optimization here
#         # collect KPI


In [245]:
# def optimise_loads(loads_at_risk_or_light_lcp_obs_enriched, w1=1, w2=1):
#     problem = LpProblem('Load Exchanging', LpMaximize)

#     # Create LP variables (same as before)
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_Name'] = 'qty_' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Name'] = 'qty_pal_' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Continuous"))
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))

#     # Rectpack vars
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Top_Name'] = 'qty_pal_top' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Bottom_Name'] = 'qty_pal_bottom' + loads_at_risk_or_light_lcp_obs_enriched['material_sk'].astype(str) + '_' + loads_at_risk_or_light_lcp_obs_enriched['load_id'].astype(str)
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_TOP'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Top_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))
#     loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_BOTTOM'] = loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar_PAL_Bottom_Name'].apply(lambda x : LpVariable(x, lowBound=0, cat="Integer"))

#     # 🧠 Update objective with weights
#     shipment_value = lpSum(
#         (
#             loads_at_risk_or_light_lcp_obs_enriched['lcp_rank_rescaled'] + 
#             1 + 
#             (w1 * loads_at_risk_or_light_lcp_obs_enriched['perc_oos_risk_at_dt_hl_rescaled']) +
#             (w2 * loads_at_risk_or_light_lcp_obs_enriched['perc_obsolescence_risk_at_sr_hl']) +
#             loads_at_risk_or_light_lcp_obs_enriched['priority_flag_rescaled']
#         ) * loads_at_risk_or_light_lcp_obs_enriched['Qty_LPVar']
#     )
#     problem += shipment_value

#     # 🧱 Constraints (same as before, keep untouched)
#     for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['material_sk', 'Source']):
#         problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['stock_on_hand_sr_hl'].iloc[0]

#     for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['material_sk', 'Destination']):
#         problem += lpSum(grp_df['Qty_LPVar']) <= grp_df['demand_at_dt_hl'].iloc[0]

#     for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id']):
#         problem += lpSum(qty * conv for qty, conv in zip(grp_df['Qty_LPVar'], grp_df['HL_weight'])) <= grp_df['available_weight'].iloc[0]
#         problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL_TOP'], grp_df['area_per_pal'])) <= grp_df['available_area_top_row'].iloc[0]
#         problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL_BOTTOM'], grp_df['area_per_pal'])) <= grp_df['available_area_bottom_row'].iloc[0]
#         problem += lpSum(pal_qty * area_conv for pal_qty, area_conv in zip(grp_df['Qty_LPVar_PAL'], grp_df['area_per_pal'])) <= grp_df['available_area'].iloc[0]

#     for grp_name, grp_df in loads_at_risk_or_light_lcp_obs_enriched.groupby(['load_id','material_sk']):
#         problem += lpSum(grp_df['Qty_LPVar']) == lpSum(pal_qty * hl_per_pal for pal_qty, hl_per_pal in zip(grp_df['Qty_LPVar_PAL'], grp_df['PAL_HL']))
#         problem += lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL_TOP']) + lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL_BOTTOM']) == lpSum(pal_qty for pal_qty in grp_df['Qty_LPVar_PAL'])

#     print('Started optimization')
#     problem.solve(PULP_CBC_CMD(timeLimit = 600, threads = None, msg = 0))
#     print('completed optimization')
#     print(LpStatus[problem.status])
#     return problem


In [246]:
df_copy = loads_at_risk_or_light_lcp_obs_enriched.copy()


In [247]:
#problem = optimise_loads(df_copy, w1=w1, w2=w2)


In [248]:
for name, constraint in problem.constraints.items():
    if "available_weight" in name:
        print(name, ":", constraint.slack)


In [249]:
for load_id, grp in df.groupby("load_id"):
    used_weight = sum(var.varValue * wt for var, wt in zip(grp["Qty_LPVar"], grp["HL_weight"]))
    print(f"Load {load_id} used weight: {used_weight} / limit: {grp['available_weight'].iloc[0]}")


In [250]:
loads_at_risk_or_light_lcp_obs_enriched.columns

Index(['load_id', 'RFRC_NUM12', 'movement_type', 'Source', 'Destination',
       'Slot Booked From', 'Slot Booked To', 'Priority Flag', 'available_pal',
       'available_weight', 'available_area_top_row',
       'available_area_bottom_row', 'available_area', 'material_sk',
       'material_code', 'lcp_rank', 'stock_on_hand_sr_hl', 'stock_sr_hl',
       'planned_production_sr_hl', 'actual_production_sr_hl',
       'outgoing_so_sto_sr_hl', 'incoming_sto_sr_hl', 'safety_stock_sr_hl',
       'stock_on_hand_dt_hl', 'demand_at_dt_hl', 'perc_oos_risk_at_dt_hl',
       'Action', 'pal_length', 'pal_width', 'pal_height', 'area_per_pal',
       'container_type_description', 'PC_PAL', 'PC_HL', 'PAL_STACKING_FACTOR',
       'HL_PAL', 'PAL_HL', 'HL_weight', 'perc_obsolescence_risk_at_sr_hl',
       'Waiting_time', 'priority_flag_rescaled',
       'perc_oos_risk_at_dt_hl_rescaled', 'lcp_rank_rescaled', 'Weights',
       'Qty_LPVar_Name', 'Qty_LPVar_PAL_Name', 'Qty_LPVar', 'Qty_LPVar_PAL',
       'Qt

In [251]:
# Debug View – Analyze decision flexibility per load
debug_sku_df = (
    loads_at_risk_or_light_lcp_obs_enriched
    .groupby('load_id')
    .agg(
        total_skus=('material_sk', 'nunique'),
        total_decision_vars=('Qty_LPVar', 'count'),
        total_suggested_qty=('suggested_deployment_sr_hl', 'sum'),
        total_available_pal=('available_pal', 'first'),
        total_available_weight=('available_weight', 'first'),
        total_available_area=('available_area', 'first')
    )
    .reset_index()
)

# Add a flag for optimization flexibility
debug_sku_df['has_flexibility'] = debug_sku_df['total_skus'] > 1

# Optional: Save to Excel
debug_sku_df.to_excel(f"{result_path}{tag}_sku_flexibility_debug.xlsx", index=False)

# Display top loads by SKU count
debug_sku_df.sort_values('total_skus', ascending=False).head(15)


,load_id,total_skus,total_decision_vars,total_suggested_qty,total_available_pal,total_available_weight,total_available_area,has_flexibility
8,34801568,85,85,42.7152,9,4.66080,88800.0,True
9,34801613,85,85,10.5600,3,1.15720,33120.0,True
6,34800876,81,81,231.3936,30,25.47192,290400.0,True
7,34800880,80,80,50.8000,6,6.60880,60000.0,True
4,34800871,79,79,261.9936,34,28.50000,326400.0,True
5,34800872,79,79,262.0512,34,28.50000,326400.0,True
10,34801622,63,63,5.7024,5,0.94560,50400.0,True
3,34800720,46,46,10.8000,2,4.15000,19200.0,True
11,34808282,13,13,31.5000,5,3.43000,50400.0,True
0,34798240,2,2,4.2768,1,2.72700,9600.0,True


In [252]:
# After the model has been solved
#selected_df = loads_at_risk_or_light_lcp_obs_enriched.copy()
selected_df = loads_at_risk_or_light_lcp_enriched.copy()
selected_df['Qty_Selected'] = selected_df['Qty_LPVar'].apply(lambda var: var.varValue if var.varValue is not None else 0)

# Check how many SKUs were actually selected
selected_summary = (
    selected_df.groupby('load_id')
    .agg(
        total_selected_qty=('Qty_Selected', 'sum'),
        num_skus_selected=('Qty_Selected', lambda x: (x > 0).sum()),
        num_total_skus=('Qty_Selected', 'count'),
        total_suggested_qty=('suggested_deployment_sr_hl', 'sum'),
        total_available_pal=('available_pal', 'first'),
        total_available_weight=('available_weight', 'first'),
        total_available_area=('available_area', 'first')
    )
    .reset_index()
)

selected_summary['selection_ratio'] = selected_summary['num_skus_selected'] / selected_summary['num_total_skus']
selected_summary.sort_values('selection_ratio', ascending=False).head(10)


,load_id,total_selected_qty,num_skus_selected,num_total_skus,total_suggested_qty,total_available_pal,total_available_weight,total_available_area,selection_ratio
0,34798240,4.2768,1,2,4.2768,1,2.72700,9600.0,0.500000
1,34798241,4.2768,1,2,4.2768,1,2.72700,9600.0,0.500000
2,34798242,4.2768,1,2,4.2768,1,2.72700,9600.0,0.500000
11,34808282,31.5000,2,13,31.5000,5,3.43000,50400.0,0.153846
6,34800876,231.3936,6,81,231.3936,30,25.47192,290400.0,0.074074
4,34800871,261.9936,5,79,261.9936,34,28.50000,326400.0,0.063291
5,34800872,262.0512,5,79,262.0512,34,28.50000,326400.0,0.063291
7,34800880,50.8000,4,80,50.8000,6,6.60880,60000.0,0.050000
8,34801568,42.7152,3,85,42.7152,9,4.66080,88800.0,0.035294
3,34800720,10.8000,1,46,10.8000,2,4.15000,19200.0,0.021739


In [ ]:
mat_id = "XYZ"
load_id = "34798240"

subset = loads_at_risk_or_light_lcp_enriched[
    (loads_at_risk_or_light_lcp_enriched["material_sk"] == mat_id) &
    (loads_at_risk_or_light_lcp_enriched["load_id"] == load_id)
]

print(subset[['stock_on_hand_sr_hl', 'demand_at_dt_hl', 'available_weight', 'available_area', 'area_per_pal', 'PAL_HL', 'Qty_LPVar_PAL', 'Qty_LPVar']])


In [1]:
loads_at_risk_or_light_lcp_enriched.columns

NameError: name 'loads_at_risk_or_light_lcp_enriched' is not defined